In [1]:
# CHANGE YOUR PATH HERE
midi_lib_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/midi_lib"

output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output"

log_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/logs/model_result_log"

In [2]:
# CHANGE YOUR PATH HERE
topk15_t1_0_ngram0_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0"

In [3]:
import sys

sys.path.append(midi_lib_path)

In [4]:
from datetime import datetime
import os

from converter.midi_dto_to_midi_file_object import midi_dto_to_midi_file_object_converter
from converter.musecoco_line_to_midi_dto import musecoco_line_to_midi_dto_converter
from converter.musecoco_line_str_to_token_list import musecoco_line_str_to_token_list_converter
from musecoco_utils import remove_structure_errors

from midi_file_utils import write_midi_file

In [5]:
# model_result_str = "s-7 o-0 t-35 d-27 p-57 d-14 d-12 v-12 i-65 p-76 d-6 v-16 p-64 d-12 v-20 p-59 i-0 p-57 d-12 v-20 b-1 s-9 o-0 t-34 i-48 t-35 i-0 p-76 t-34 d-6 v-20 o-12 t-35 i-0 p-69 d-12 v-20 o-24 t-35 i-0 p-65 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-12 v-24 i-42 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-12 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-12 v-20 p-69 d-6 v-20 p-69 d-6 v-20 p-60 d-6 v-20 o-42 t-37 p-67 d-12 v-16 p-57 d-6 v-20 p-57 d-6 v-27 p-64 d-6 v-20 o-42 t-35 i-0 p-65 d-12 v-20 p-60 d-6 v-12 i-56 t-35 i-0 p-81 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-24 i-67 i-42 d-6 v-20 p-50 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-67 d-6 v-20 o-6 t-35 i-0 p-65 d-23 d-12 v-20 p-52 d-11 v-20 o-30 t-35 i-0 p-77 d-6 v-24 i-57 p-48 d-6 v-20 i-43 v-20 b-1 s-8 o-0 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-0 p-82 o-6 t-35 i-0 p-55 d-12 v-20 o-36 t-34 i-0 p-50 d-6 v-20 p-67 d-12 v-20 p-60 d-6 v-20 o-6 t-30 i-0 p-67 d-12 v-20 i-128 p-187 p-60 d-12 v-20 i-56 t-40 d-12 v-20 p-47 d-6 v-12 p-41 d-6 v-24 o-0 t-35 i-40 p-50 d-6 v-20 i-60 p-69 d-6 v-20 p-69 d-6 v-20 p-43 d-6 v-20 o-30 t-35 i-38 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-12 v-20 p-57 d-6 v-20 p-48 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-12 v-20 o-18 t-35 i-0 p-74 d-12 v-20 p-50 d-6 v-20 o-24 t-35 i-0 p-59 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-24 d-12 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 i-43 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 i-128 p-170 d-6 v-20 i-48 p-47 i-0 p-60 d-6 v-20 p-50 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 p-65 d-6 v-20 p-55 d-6 v-20 p-41 d-6 v-20 o-12 t-35 i-0 d-12 v-20 p-55 d-6 v-20 i-56 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 p-60 d-12 v-16 d-6 v-20 p-55 d-12 v-20 p-62 d-11 v-20 o-6 t-35 i-0 p-60 d-6 v-20 i-56 p-59 d-3 v-20 p-45 d-12 d-6 v-20 p-41 d-6 v-20 o-6 t-30 i-0 p-69 d-3 v-20 p-48 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-69 d-3 v-20 p-53 d-18 v-20 o-18 t-30 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-48 d-11 v-20 p-52 d-6 v-20 o-30 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 p-48 d-11 v-20 o-42 t-35 i-0 p-41 d-6 v-20 p-55 d-6 v-20 p-36 d-12 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-12 v-20 o-6 t-26 i-48 t-34 i-43 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-76 d-12 v-20 p-41 p-52 d-6 v-20 o-24 t-35 i-52 p-48 d-6 v-20 o-9 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-24 d-12 v-20 p-48 d-11 v-20 o-36 t-35 i-0 t-35 i-0 p-67 d-17 v-20 p-60 d-12 v-20 p-55 d-6 v-20 p-64 d-6 v-24 b-1 s-9 o-0 t-35 i-0 p-67 d-18 v-20 p-57 d-6 v-20 p-59 d-12 v-20 o-12 t-35 i-0 p-57 d-6 v-20 o-12 t-30 i-0 d-6 v-20 i-60 p-65 d-6 v-20 d-12 v-20 o-24 t-35 i-0 d-11 v-20 o-12 t-38 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-57 d-6 v-20 p-48 d-14 d-6 v-20 o-30 t-35 i-0 p-76 d-6 v-20 o-18 t-35 i-0 p-52 d-6 v-20 i-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-4 d-6 v-20 p-64 d-6 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-40 d-11 v-20 i-48 t-35 i-0 p-74 d-6 v-20 p-41 t-35 i-0 p-64 d-12 v-20 b-1 s-3 v-20 b-1 s-8 o-0 t-35 i-0 p-65 d-6 v-20 i-52 p-60 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-84 d-6 v-20 p-53 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 p-47 d-6 v-20 p-59 d-6 v-20 o-12 t-35 i-0 p-84 d-3 v-20 o-18 t-35 i-0 p-62 d-3 v-20 i-60 d-6 v-20 p-57 d-6 v-20 o-45 t-35 i-0 p-67 d-6 v-20 i-46 d-6 v-20 i-65 p-55 d-26 d-6 v-20 o-12 t-35 i-0 p-67 d-3 v-20 i-52 p-65 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-69 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-60 d-11 v-20 o-12 t-35 i-0 p-60 d-6 v-20 i-60 t-35 i-0 p-65 d-12 v-20 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-24 o-36 t-35 i-0 p-74 d-12 v-20 o-24 t-35 i-0 p-77 d-6 v-20 i-57 t-35 i-0 p-74 d-6 v-20 p-59 d-6 v-20 o-12 t-35 i-0 p-64 d-11 v-20 o-12 t-35 i-0 p-62 v-20 b-1 s-8 o-0 t-42 d-6 v-20 i-60 p-64 d-6 v-20 o-12 t-35 i-0 p-88 p-65 d-6 v-20 b-1 </s> d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-58 d-6 v-20 o-12 t-35 i-0 p-59 d-3 v-20 p-57 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 p-69 d-6 v-20 o-12 t-35 i-0 p-45 d-6 v-20 p-50 d-6 v-20 p-43 d-6 v-20 p-64 d-6 v-20 o-6 t-35 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 i-52 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-52 t-35 i-0 p-62 d-6 v-20 i-128 p-187 d-12 v-20 p-43 d-6 v-20 p-52 d-6 v-20 i-60 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-18 t-35 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-86 d-6 v-20 o-12 d-6 v-20 o-36 t-35 i-0 p-60 d-9 v-20 p-64 d-3 v-20 o-18 t-35 i-0 p-81 p-67 d-12 v-20 p-50 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 d-6 v-20 p-62 d-6 v-20 p-60 d-3 v-20 o-6 t-35 i-0 p-59 d-11 v-20 p-163 d-6 v-20 d-6 v-20 o-30 t-26 i-0 p-62 d-12 v-20 t-35 i-0 p-62 d-6 v-20 p-57 d-12 v-20 p-52 d-6 v-20 i-52 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-15 d-12 v-20 o-36 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 p-59 d-6 v-20 i-58 d-12 v-20 o-18 t-35 i-0 p-62 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 p-48 d-6 v-20 p-48 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-19 o-12 t-35 i-0 p-48 d-11 v-20 o-6 t-30 i-0 p-60 d-6 v-20 o-42 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-24 o-12 t-35 i-0 p-60 d-6 v-20 p-57 d-6 v-20 o-30 d-6 v-20 b-1 s-8 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 d-6 v-20 o-12 t-38 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-43 d-6 v-20 o-24 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-38 d-6 v-20 p-57 d-6 v-20 p-64 d-6 v-20 p-48 d-6 v-20 t-35 i-0 p-40 d-5 d-18 v-20 b-1 d-18 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-3 v-20 p-64 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-76 d-11 v-20 o-15 d-6 v-20 p-64 d-6 v-20 p-48 d-6 v-20 p-45 d-6 v-20 p-57 d-6 v-20 d-6 v-20 i-52 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-40 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-48 d-6 v-20 p-52 d-12 v-20 o-36 t-35 i-0 p-57 d-6 v-20 o-30 t-35 i-0 p-76 d-6 v-20 i-58 v-20 o-6 t-34 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 i-56 p-50 d-6 v-20 p-41 t-35 d-3 v-20 p-64 d-6 v-20 o-30 t-35 i-0 p-65 d-6 v-20 v-20 i-5"
# model_result_str = "s-9 o-0 t-35 i-0 p-76 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-12 v-20 o-12 t-35 i-0 p-64 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 p-50 d-6 v-20 o-12 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 i-48 t-35 i-0 p-57 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-12 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-62 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 d-6 v-20 b-1 t-38 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 v-20 o-18 t-35 i-0 p-76 d-6 v-20 b-1 d-6 v-20 o-18 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 d-6 v-20 o-6 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-59 d-6 v-20 o-18 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 v-20 o-6 t-35 i-0 p-76 d-6 v-20 d-6 v-20 o-18 t-35 i-0 t-35 i-0 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 d-6 v-20 b-1 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 i-0 p-62 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 b-1 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-24 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 t-38 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 t-35 i-0 p-57 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 b-1 s-7 o-0 t-35 i-0 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 t-38 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-30 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 v-20 b-1 d-6 v-20 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 t-35 i-0 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 d-6 v-20 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 t-35 i-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 d-6 v-20 d-6 v-20 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 t-38 i-0 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1"
# model_result_str = "s-9 o-0 t-36 i-40 p-64 d-6 v-20 o-48 t-36 i-40 p-64 d-6 v-20 o-48 t-36 i-29 p-64 d-6 v-20 o-48 t-36 i-29 p-64 d-6 v-20 o-48 t-36 i-29 p-64 d-6 v-20 o-48 t-36 i-29 p-64 d-6 v-20 o-48 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-60 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-66 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 i-29 p-60 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-60 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-67 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-60 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-60 d-6 v-20 t-36 i-29 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-40 p-62 d-6 v-20 t-36 i-40 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-67 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-29 p-64 d-6 v-20 t-36 i-40 p-64 d-6"
# model_result_str = "o-0 i-30 p-67 d-6 v-20 o-72 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-57 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-172 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-71 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-65 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-70 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-40 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 t-18 d-6 v-20 t-39 i-30 p-61 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-71 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 t-47 i-30 p-53 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 v-20 t-39 i-30 p-59 d-6 v-20 t-39 i-30 s-48 v-20 t-39 i-30 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 s-48 p-65 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-77 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-68 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-65 d-6 v-20 t-39 i-30 s-128 d-6 v-20 t-39 i-30 d-1 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 t-18 d-6 v-20 t-39 i-30 p-65 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-61 d-6 v-20 t-39 i-30 p-68 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-77 d-6 v-20 t-39 i-30 p-77 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 ST1_13 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-61 d-6 v-20 t-39 i-30 p-57 d-6 v-20 t-39 i-30 ST1_1 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-59 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-65 d-6 v-20 t-39 i-30 p-71 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-59 d-6 v-20 t-39 i-30 t-17 p-58 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 d-6 v-20 t-39 i-30 d-19 v-20 t-39 i-30 d-6 v-20 t-39 i-30 p-77 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-60 d-6 v-20 t-39 i-30 p-66 d-6 v-20 t-39 i-30 p-68 d-6 v-20 t-39 i-30 p-62 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-63 d-6 v-20 t-39 i-30 p-57 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-76 d-6 v-20 t-39 i-30 p-67 d-6 v-20 t-39 i-30 p-79 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-55 d-6 v-20 t-39 i-30 t-18 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-57 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-56 d-6 v-20 t-39 i-30 p-61 d-6 v-20 t-39 i-30 s-48 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-65 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 d-6 v-20 t-39 i-30 s-174 d-6 v-20 t-39 i-30 p-74 d-6 v-20 t-39 i-30 p-57 d-6 v-20 t-39 i-30 p-69 d-6 v-20 t-39 i-30 p-64 d-6 v-20 t-39 i-30 p-77 d-6 v-20 t-39 v-20 t-39 i-30 p-60 d-6"
# model_result_str = "s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-34 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-76 t-34 i-0 p-67 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-12 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-65 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 v-20 b-1 s-8 d-6 v-20 o-12 t-35 i-0 p-74 d-12 v-20 o-12 t-35 i-0 p-65 d-12 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-11 v-20 o-12 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 i-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-72 t-35 i-0 p-65 d-6 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 i-0 p-72 d-6 v-20 p-62 d-6 v-20 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 p-52 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-11 v-20 p-52 d-6 v-20 p-64 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 p-62 d-6 v-20 o-12 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-62 d-6 v-20 p-50 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 p-53 d-6 v-20 p-62 d-6 v-20 p-52 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 p-60 d-6 v-20 p-69 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 t-35 i-0 p-67 d-6 v-20 p-65 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 i-0 p-71 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-71 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 i-52 v-20 o-24 t-35 i-0 p-71 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-6 t-35 i-0 p-65 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 p-64 d-6 v-20 o-18 v-20 p-62 d-6 v-20 o-6 d-6 v-20 p-52 d-6 v-20 i-52 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-48 d-6 v-20 o-30 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 v-20 p-62 d-6 v-20 p-57 d-6 v-20 p-48 d-6 v-20 i-52 v-20 o-24 t-35 i-0 p-71 d-6 v-20 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 t-34 i-0 p-67 d-6 v-20 p-53 d-6 v-20 p-62 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 p-62 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-60 v-20 o-18 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 d-6 v-20 o-6 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 p-64 d-6 v-20 p-60 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-60 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-62 d-6 v-20 p-53 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-52 d-6 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-64 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 d-6 v-20 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-8 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 i-52 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-71 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-62 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-65 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 p-52 d-6 v-20 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-50 d-6 v-20 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-71 t-35 i-0 p-71 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-79 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-59 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-16 b-1 s-8 o-0 t-35 i-0 p-67 d-11 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-24 t-30 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-48 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-30 t-35 i-0 p-74 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-55 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-18 t-30 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-30 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 o-6 t-30 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 p-48 d-6 v-20 b-1 s-7 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-6 t-30 i-0 p-62 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-74 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 d-6 v-20 b-1 d-6 v-20 b-1 s-7 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-6 t-30 o-0 t-35 i-0 p-62 d-6 v-20 o-6 t-30 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-12 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-18 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-6 t-30 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-59 d-6 v-20 o-12 t-26 i-0 p-74 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-30 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-6 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-7 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-50 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-24 t-35 i-0 p-55 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-52 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-34 i-0 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 o-12 t-30 i-0 p-65 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 o-36 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-62 d-6 v-20 o-6 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-57 d-6 v-20 b-1 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-18 t-30 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 p-53 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-34 i-0 p-55 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-6"
# model_result_str = "s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-79 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-59 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-16 b-1 s-8 o-0 t-35 i-0 p-67 d-11 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-24 t-30 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-48 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-74"
# model_result_str = "s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-3 v-20 o-24 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-69 d-6 v-16 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-55 d-6 v-20 p-53 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-6 t-30 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 o-6 t-30 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-52 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-7 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 p-60 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-50 d-6 v-20 p-57 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 i-48 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-36 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-55 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 p-52 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-64 d-6 v-20 o-24 t-34 i-0 p-67 d-6 v-20 p-52 d-6 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 o-6 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 o-12 t-30 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 s-7 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-26 i-0 p-62 d-6 v-20 p-62 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-6 t-30 i-0 p-64 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-6 t-30 i-0 p-60 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-30 i-0 p-64 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 d-6 v-20 p-60 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-34 i-0 p-74 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 p-55 d-6 v-20 i-48 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-6 t-30 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 p-62 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 o-30 t-35 i-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-62 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 p-57 o-0 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 b-1 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-12 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 p-55 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-30 t-30 i-0 p-65 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 o-0 t-35 i-0 p-72 d-6 v-20 p-53 d-6 v-20 p-59 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-30 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-79 d-6 v-20 i-48 d-6 v-20 o-6 t-30 i-0 p-64 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-52 d-6 v-20 o-6 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-6 t-30 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-76 d-6 v-20 o-36 t-35 i-0 p-60 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-74 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 t-30 i-0 p-76 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-30 t-30 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-62 d-6 v-20 p-60 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6"
# model_result_str = "s-9 o-0 t-35 i-0 p-67 d-3 v-20 o-36 t-30 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 I1s2_15_1 t-35 i-0 p-79 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-16 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-9 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-16 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-30 i-0 p-74 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-34 i-0 p-72 d-6 v-20 b-1 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-72 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-62 d-6 v-20 o-12 t-30 i-0 p-72 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-55 d-6 v-20 o-36 t-34 i-0 p-74 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 t-30 i-0 p-67 d-6 v-20 p-48 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-6 t-30 i-0 p-62 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-6 d-6 v-20 b-1 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-52 d-11 v-20 o-6 d-6 v-20 b-1 s-7 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-18 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 o-6 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 o-12 t-30 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-57 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 o-6 d-6 v-20 o-6 t-35 i-0 p-67 d-6 d-6 v-20 o-6 d-6 v-20 o-36 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-50 d-6 v-20 p-55 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 i-48 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 d-6 v-20 p-52 d-6 v-20 b-1 d-6 v-20 p-52 d-6 v-20 p-43 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 i-48 t-35 i-0 p-57 d-6 v-20 b-1 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-30 i-0 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-57 d-6 v-20 o-6 t-30 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-50 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-30 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 p-57 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 d-6 v-20 b-1 d-6 v-20 p-52 d-6 v-20 p-62 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 o-30 t-35 i-0 p-62 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 o-6 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-55 d-6 v-20 b-1 s-7 d-6 v-20 o-18 t-30 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-30 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 o-6 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 i-48 d-6 v-20 p-62 d-6 v-20 o-24 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-55 d-6 v-20 i-48 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-36 t-35 i-0 t-30 i-48 d-6 v-20 o-18 t-30 o-0 t-35 i-0 p-60 d-6 v-20 b-1 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-30 i-0 p-71 d-6 v-20 p-50 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 p-53 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-42 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 p-55 d-6 v-20 o-30 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 i-48 p-55 d-6 v-20 o-6 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-57 d-6 v-20 p-62 d-6 v-20 p-55 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-42 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-6 t-30 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55"
# model_result_str = "s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-9 v-20 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-6 v-20 p-52 d-6 v-20 p-48 d-3 v-24 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 p-48 d-3 v-24 p-33 d-6 v-20 o-33 t-33 i-57 p-48 d-3 v-24 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-12 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-12 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-50 d-9 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-24 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-50 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-30 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-33 t-33 i-57 p-52 d-6 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-48 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-43 d-3 v-20 o-42 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 p-45 d-3 v-20 o-45 t-33 i-57 p-50 d-13 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-12 t-33 i-57 p-53 d-9 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-21 t-33 i-57 p-53 d-3 v-24 o-24 t-33 i-57 p-53 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-39 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 p-42 d-3 v-20 o-42 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 p-41 d-3 v-20 p-41 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-47 d-3 v-20 p-47 d-3 v-20 o-9 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-15 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-21 t-33 i-57 p-48 d-6 v-20 p-48 d-6"
# model_result_str = "s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-62 d-3 v-20 o-24 t-35 i-0 p-65 d-3 v-20 o-18 t-30 i-0 p-77 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-59 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-65 d-11 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-59 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-74 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-59 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-77 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-36 t-35 i-0 p-60 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-59 d-6 v-20 o-12 t-30 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-7 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-59 d-6 v-20 o-36 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 t-30 i-0 p-60 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-62 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-53 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-6 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 p-50 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 o-18 t-35 i-48 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-18 t-30 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-12 d-6 v-20 p-60 d-6 v-20 p-52 d-6 v-20 p-53 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-55 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 i-48 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-59 d-6 v-20 p-53 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-53 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 p-55 d-6 v-20 o-30 t-35 i-0 p-64 d-6 v-20 p-53 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 o-6 t-35 i-0 p-71 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 o-12 t-30 i-0 p-62 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-65 d-6 v-20 o-24 t-35 i-0 p-62 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-57 d-6 v-20 p-57 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 p-50 d-6 v-20 p-62 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-65 d-6 v-20 p-52 d-6 v-20 b-1 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-57 d-6 v-20 p-55 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-57 d-6 v-20 o-6 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 d-6 v-20 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 p-55 d-6 v-20 b-1 d-6 v-20 p-53 d-6 v-20 b-1 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 p-57 d-6 v-20 p-50 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-48 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-6 d-6 v-20 b-1 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 i-48 t-35 i-0 d-6 v-20 p-55 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-59 d-6 v-20 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 d-6 v-20 b-1 s-7 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-50 d-6 v-20 o-12 t-35 i-0 t-30 i-48 t-35 i-0 p-67 d-6 v-20 b-1 d-6 v-20 p-60 d-6 v-20 p-52 v-20 p-52 d-6 v-20 p-52 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-30 t-30 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-62 d-6 v-20 p-52 d-6 v-20 p-62 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 p-52 d-6 v-20 p-50 d-6 v-20 p-64 d-6 v-20 o-12 t-30 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-6 t-30 d-6 v-20 b-1 d-6 v-20 o-36 t-35 i-0 p-76 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 p-53 d-6 v-20 i-48 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-52 d-6 v-20 p-57 d-6 v-20 p-60 d-6 v-20 p-55 d-6 v-20 d-6 v-20 p-53 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-36 t-35 d-6 v-20 p-57 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-55 d-6 v-20 p-48 d-6 v-20 o-18 o-0 t-35 i-0 p-67 d-6 v-20"
# model_result_str = "d-3 v-20 b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-24 b-1 s-9 o-0 o-0 d-3 v-20 o-12 v-20 o-12 v-20 o-24 d-6 v-20 b-1 t-35 i-0 d-3 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 o-18 v-20 b-1 s-9 o-0 d-3 v-20 o-24 i-0 d-6 v-20 b-1 s-9 o-0 d-3 d-3 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 d-3 v-20 b-1 s-9 o-0 d-3 v-20 o-12 v-20 o-12 v-20 b-1 v-20 b-1 s-9 o-0 d-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-36 d-6 v-20 o-12 v-20 o-24 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 d-6 v-20 d-6 v-20 b-1 s-8 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 v-20 o-24 v-20 o-24 d-6 v-20 o-12 v-20 b-1 v-20 d-6 v-20 o-12 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 i-128 i-0 p-76 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 o-12 v-20 o-12 v-20 o-18 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-64 d-3 v-20 o-12 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 p-74 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-6 v-20 v-20 o-12 v-20 b-1 v-20 d-3 v-20 b-1 s-9 o-0 d-3 v-20 o-12 i-0 d-6 v-20 o-12 v-20 d-3 v-20 i-128 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 s-9 o-0 i-0 s-9 o-0 i-0 d-6 v-20 o-24 d-6 v-20 i-128 d-6 v-20 o-12 v-20 o-12 i-0 i-0 t-35 i-0 p-72 d-6 v-20 o-12 v-20 p-60 d-3 v-20 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 d-6 v-20 v-20 d-3 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 i-0 p-64 d-6 v-20 o-30 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 i-0 d-6 v-20 p-64 d-6 v-20 o-24 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 i-0 d-6 v-20 d-3 v-20 v-20 b-1 s-9 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 i-0 p-71 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 o-12 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 v-20 v-20 p-64 i-0 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 d-6 v-20 o-18 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 v-20 b-1 v-20 i-128 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 d-3 v-20 d-6 v-20 d-3 v-20 d-6 v-20 i-128 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 o-12 v-20 p-64 d-6 v-20 d-6 v-20 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 d-3 v-20 o-24 v-20 o-12 v-20 b-1 v-20 b-1 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 d-12 v-20 p-64 v-20 o-12 v-20 o-30 v-20 b-1 v-20 d-3 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 d-6 v-20 o-12 v-20 o-12 d-6 v-20 o-12 v-20 p-64 d-6 v-20 o-12 v-20 i-128 v-20 d-12 v-20 d-3 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-3 v-20 o-12 v-20 i-128 d-6 v-20 b-1 v-24 d-6 v-20 b-1 d-6 v-20 d-6 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 p-60 i-0 d-6 v-20 o-12 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 v-20 p-64 d-3 v-20 b-1 s-9 o-0 v-20 o-12 v-20 p-64 v-20 d-6 v-20 d-6 v-20 v-20 d-6 v-20 b-1 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 p-60 d-6 v-20 b-1 s-9 o-0 v-20 o-24 v-20 o-12 v-20 o-12 v-20 b-1 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 b-1 v-20 d-6 v-20 o-0 i-0 p-76 d-6 v-20 b-1 v-20 b-1 v-20 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 i-128 d-6 v-20 b-1 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 b-1 v-20 d-3 v-20 b-1 v-20 v-20 o-12 v-20 b-1 v-20 b-1 v-20 b-1 v-20 p-64 i-0 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 v-20 i-128 i-0 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 i-0 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 d-6 v-20 b-1 v-20 d-3 v-20 p-64 d-6 v-20 o-12 v-20 d-3 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 o-24 d-6 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 v-20 o-12 v-20 d-3 v-20 p-60 d-6 v-20 i-128 v-20 i-128 d-6 v-20 p-64 v-20 o-24 d-6 v-20 o-12 v-20 o-12 v-20 p-64 i-0 d-6 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 b-1 v-20 i-128 v-20 o-12 v-20 i-128 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 p-64 d-3 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 v-20 p-64 d-6 v-20 o-12 v-20 p-64 d-6 v-20 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 o-24 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-60 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 v-20 d-3 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-3 v-20 o-12 v-20 b-1 s-9 i-0 d-6 v-20 p-64 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 d-3 v-20 o-24 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 p-71 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 i-128 i-128 d-6 v-20 p-64 d-6 v-20 o-12 v-20 p-64 v-20 o-24 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 o-24 v-20 d-6 v-20 d-12 v-20 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 d-6 v-20 d-3 v-20 b-1 s-9 o-0 v-20 d-6 v-20 v-20 o-24 v-20 v-20 b-1 s-9 o-0 v-20 i-128 v-20 o-12 v-20 i-128 i-0 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 p-48 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 v-20 o-12 v-20 b-1 s-9 d-6 v-20 o-12 v-20 i-128 v-20 p-64 v-20 d-3 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 i-128 i-0 v-20 i-128 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 i-128 i-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 i-128 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 d-6 v-20 o-24 d-6 v-20 d-3 v-20 o-24 i-0 v-20 i-128 i-0 p-64 v-20 i-128 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-12 v-20 o-12 v-20 p-64 d-3 v-20 o-24 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-24 v-20 o-12 i-0 d-6 v-20 p-64 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 p-64 v-20 p-64 d-6 v-20 p-64 d-6 v-20 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 o-12 v-20 b-1 v-20 o-12 v-20 p-64 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 i-128 d-12 v-20 d-6 v-20 o-12 v-20 p-57 i-0 p-64 d-6 v-20 o-12 d-12 v-20 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 i-128 d-6 v-20 o-12 v-20 b-1 s-9 o-0 i-0 p-71 v-20 i-128 d-12 v-20 i-128 d-12 v-20 b-1 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 d-12 v-20 b-1 s-9 o-0 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 b-1 v-20 d-6 v-20 b-1 s-9 v-20 d-3 v-20 o-24 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 d-6 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 d-3 v-20 o-12 v-20 i-128 d-6 v-20 o-12 v-20 o-24 v-20 i-128 d-6 v-20 b-1 v-20 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 d-6 v-20 o-12 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 i-128 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 o-12 v-20 b-1 d-6 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 i-128 i-0 p-67 i-0 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 v-20 d-6 v-20 o-12 v-20 o-12 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 o-12 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 i-128 i-128 i-0 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 v-20 b-1 v-20 o-12 v-20 p-64 d-6 v-20 i-128 i-0 v-20 o-12 v-20 b-1 v-20 o-12 v-20 o-24 d-6 v-20 v-20 p-64 d-12 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 i-128 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 d-6 v-20 d-6 v-20 p-64 i-0 p-71 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-0 d-6 v-20 p-60 d-6 v-20 d-6 v-20 p-59 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 i-128 i-128 d-12 v-20 i-128 i-0 d-6 v-20 i-128 d-3 v-20 o-12 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 d-6 v-20 p-64 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 p-60 d-6 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 v-20 b-1 v-20 v-20 o-12 v-20 v-20 o-12 v-20 b-1 v-20 b-1 d-12 v-20 d-6 v-20 i-128 i-0 i-0 p-64 d-6 v-20 o-12 v-20 i-128 d-6 v-20 b-1 s-9 o-0 i-0 p-76 d-6 v-20 b-1 v-20 b-1 s-9 o-0 i-0 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 b-1 v-20 b-1"
# model_result_str = "b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-20 o-12 v-24 b-1 s-9 o-0 o-0 d-3 v-20 o-12 v-20 o-12 v-20 o-24 d-6 v-20 b-1 t-35 i-0 d-3 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 o-18 v-20 b-1 s-9 o-0 d-3 v-20 o-24 i-0 d-6 v-20 b-1 s-9 o-0 d-3 d-3 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 d-3 v-20 b-1 s-9 o-0 d-3 v-20 o-12 v-20 o-12 v-20 b-1 v-20 b-1 s-9 o-0 d-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-36 d-6 v-20 o-12 v-20 o-24 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 d-6 v-20 d-6 v-20 b-1 s-8 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 v-20 o-24 v-20 o-24 d-6 v-20 o-12 v-20 b-1 v-20 d-6 v-20 o-12 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 i-128 i-0 p-76 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 o-12 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 o-12 v-20 o-12 v-20 o-18 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 p-64 d-3 v-20 o-12 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 p-74 d-3 v-20 o-12 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-6 v-20 v-20 o-12 v-20 b-1 v-20 d-3 v-20 b-1 s-9 o-0 d-3 v-20 o-12 i-0 d-6 v-20 o-12 v-20 d-3 v-20 i-128 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 b-1 s-9 o-0 i-0 s-9 o-0 i-0 d-6 v-20 o-24 d-6 v-20 i-128 d-6 v-20 o-12 v-20 o-12 i-0 i-0 t-35 i-0 p-72 d-6 v-20 o-12 v-20 p-60 d-3 v-20 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 d-6 v-20 v-20 d-3 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 i-0 p-64 d-6 v-20 o-30 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 i-0 d-6 v-20 p-64 d-6 v-20 o-24 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 i-0 d-6 v-20 d-3 v-20 v-20 b-1 s-9 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 i-0 p-71 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 o-12 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 v-20 v-20 p-64 i-0 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 d-6 v-20 o-18 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 v-20 b-1 v-20 i-128 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 d-3 v-20 d-6 v-20 d-3 v-20 d-6 v-20 i-128 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 o-12 v-20 p-64 d-6 v-20 d-6 v-20 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 d-3 v-20 o-24 v-20 o-12 v-20 b-1 v-20 b-1 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 d-12 v-20 p-64 v-20 o-12 v-20 o-30 v-20 b-1 v-20 d-3 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 d-6 v-20 o-12 v-20 o-12 d-6 v-20 o-12 v-20 p-64 d-6 v-20 o-12 v-20 i-128 v-20 d-12 v-20 d-3 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-3 v-20 o-12 v-20 i-128 d-6 v-20 b-1 v-24 d-6 v-20 b-1 d-6 v-20 d-6 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 p-60 i-0 d-6 v-20 o-12 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 v-20 p-64 d-3 v-20 b-1 s-9 o-0 v-20 o-12 v-20 p-64 v-20 d-6 v-20 d-6 v-20 v-20 d-6 v-20 b-1 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 p-60 d-6 v-20 b-1 s-9 o-0 v-20 o-24 v-20 o-12 v-20 o-12 v-20 b-1 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 o-12 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 o-12 v-20 b-1 s-9 o-0 v-20 b-1 v-20 d-6 v-20 o-0 i-0 p-76 d-6 v-20 b-1 v-20 b-1 v-20 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 i-128 d-6 v-20 b-1 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 b-1 v-20 d-3 v-20 b-1 v-20 v-20 o-12 v-20 b-1 v-20 b-1 v-20 b-1 v-20 p-64 i-0 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 v-20 i-128 i-0 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 i-0 v-20 d-6 v-20 o-24 d-6 v-20 o-12 v-20 d-6 v-20 b-1 v-20 d-3 v-20 p-64 d-6 v-20 o-12 v-20 d-3 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 o-24 d-6 v-20 d-6 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 v-20 o-12 v-20 d-3 v-20 p-60 d-6 v-20 i-128 v-20 i-128 d-6 v-20 p-64 v-20 o-24 d-6 v-20 o-12 v-20 o-12 v-20 p-64 i-0 d-6 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 b-1 v-20 i-128 v-20 o-12 v-20 i-128 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 p-64 d-3 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 v-20 p-64 d-6 v-20 o-12 v-20 p-64 d-6 v-20 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 o-24 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-60 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 v-20 d-3 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 d-3 v-20 o-12 v-20 b-1 s-9 i-0 d-6 v-20 p-64 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 p-64 d-6 v-20 p-64 d-6 v-20 i-128 d-6 v-20 d-3 v-20 o-24 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 i-0 p-71 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 i-128 i-128 d-6 v-20 p-64 d-6 v-20 o-12 v-20 p-64 v-20 o-24 v-20 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 o-24 v-20 d-6 v-20 d-12 v-20 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 b-1 v-20 b-1 v-20 d-6 v-20 d-3 v-20 b-1 s-9 o-0 v-20 d-6 v-20 v-20 o-24 v-20 v-20 b-1 s-9 o-0 v-20 i-128 v-20 o-12 v-20 i-128 i-0 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 p-48 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 v-20 o-12 v-20 b-1 s-9 d-6 v-20 o-12 v-20 i-128 v-20 p-64 v-20 d-3 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 i-128 i-0 v-20 i-128 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 i-128 i-0 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 i-128 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 d-6 v-20 o-24 d-6 v-20 d-3 v-20 o-24 i-0 v-20 i-128 i-0 p-64 v-20 i-128 d-6 v-20 b-1 s-9 o-0 d-6 v-20 o-12 v-20 o-12 v-20 p-64 d-3 v-20 o-24 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-24 v-20 o-12 i-0 d-6 v-20 p-64 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 p-64 v-20 p-64 d-6 v-20 p-64 d-6 v-20 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 o-12 v-20 b-1 v-20 o-12 v-20 p-64 v-20 d-6 v-20 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 v-20 o-12 d-3 v-20 i-128 d-12 v-20 d-6 v-20 o-12 v-20 p-57 i-0 p-64 d-6 v-20 o-12 d-12 v-20 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 i-128 d-6 v-20 o-12 v-20 b-1 s-9 o-0 i-0 p-71 v-20 i-128 d-12 v-20 i-128 d-12 v-20 b-1 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 b-1 d-12 v-20 b-1 s-9 o-0 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 b-1 v-20 d-6 v-20 b-1 s-9 v-20 d-3 v-20 o-24 d-6 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 d-6 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 o-12 v-20 o-12 v-20 d-6 v-20 i-128 d-6 v-20 i-128 d-6 v-20 d-3 v-20 o-12 v-20 i-128 d-6 v-20 o-12 v-20 o-24 v-20 i-128 d-6 v-20 b-1 v-20 d-6 v-20 b-1 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 d-6 v-20 o-12 d-6 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 i-128 d-6 v-20 o-12 v-20 b-1 s-9 o-0 i-0 v-20 i-128 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 o-12 v-20 b-1 d-6 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 i-128 i-0 p-67 i-0 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 v-20 d-6 v-20 o-12 v-20 o-12 v-20 o-12 v-20 i-128 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 o-12 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 b-1 v-20 i-128 i-128 i-0 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 v-20 b-1 v-20 o-12 v-20 p-64 d-6 v-20 i-128 i-0 v-20 o-12 v-20 b-1 v-20 o-12 v-20 o-24 d-6 v-20 v-20 p-64 d-12 v-20 d-6 v-20 b-1 v-20 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 i-0 d-6 v-20 i-128 d-6 v-20 d-6 v-20 i-128 i-0 d-6 v-20 d-6 v-20 d-6 v-20 p-64 i-0 p-71 d-6 v-20 d-6 v-20 b-1 s-9 o-0 v-20 i-0 d-6 v-20 p-60 d-6 v-20 d-6 v-20 p-59 d-6 v-20 d-6 v-20 d-6 v-20 b-1 v-20 p-64 d-6 v-20 d-6 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 o-12 v-20 i-128 i-128 d-12 v-20 i-128 i-0 d-6 v-20 i-128 d-3 v-20 o-12 v-20 i-128 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 v-20 d-6 v-20 b-1 s-9 v-20 i-128 d-6 v-20 d-6 v-20 d-6 v-20 o-12 v-20 i-128 d-6 v-20 p-64 d-6 v-20 b-1 v-20 d-6 v-20 i-128 d-6 v-20 d-6 v-20 p-60 d-6 v-20 o-12 v-20 o-12 v-20 b-1 s-9 o-0 v-20 d-6 v-20 o-12 v-20 b-1 v-20 v-20 o-12 v-20 v-20 o-12 v-20 b-1 v-20 b-1 d-12 v-20 d-6 v-20 i-128 i-0 i-0 p-64 d-6 v-20 o-12 v-20 i-128 d-6 v-20 b-1 s-9 o-0 i-0 p-76 d-6 v-20 b-1 v-20 b-1 s-9 o-0 i-0 v-20 o-12 v-20 d-6 v-20 b-1 s-9 o-0 v-20 p-64 d-6 v-20 o-12 v-20 d-6 v-20 b-1 v-20 b-1"
# model_result_str = "s-9 o-0 t-35 i-24 p-62 d-6 v-20 o-24 t-35 i-58 S4_20_2 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 v-25 d-3 v-20 p-48 d-6 v-24 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-40 t-35 i-58 t-35 i-0 p-72 d-3 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-69 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-60 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 i-128 p-170 d-3 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-76 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-3 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 v-20 o-12 t-35 i-0 p-76 d-12 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-69 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-12 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 i-0 p-60 d-3 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 d-6 v-20 i-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 t-35 i-0 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-52 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-69 d-6 v-20 b-1 s-8 o-0 t-35 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-64 v-20 o-24 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 v-20 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-24 t-35 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 i-0 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-48 d-6 v-20 o-24 t-35 d-6 v-20 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 t-35 i-0 p-76 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 v-20 o-24 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 i-52 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 t-35 i-0 t-35 i-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-48 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 i-0 p-67 d-6 v-20 o-12 t-35 i-0 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-24 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-59 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-57 t-35 i-0 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-12 v-20 o-24 t-35 i-0 p-74 d-12 v-20 b-1 s-9 o-0 t-35 i-0 S4_13_2 p-64 d-12 v-20 o-12 t-35 v-20 i-60 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-12 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 o-24 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-64 d-12 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-18 t-35 i-0 p-69 v-20 i-128 p-170 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-62 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 i-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-56 p-60 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 o-12 t-35 i-0 p-69 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-30 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-52 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-76 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 b-1 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-72 d-6 v-20 p-52 d-12 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-76 v-20 o-12 d-6 v-20 o-18 v-20 o-12 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 i-0 p-72 d-12 v-20 i-60 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-71 d-6 v-20 o-6 t-35 i-0 p-72 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-50 d-6 v-20 p-57 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 v-20 b-1 s-9 o-0 t-35 i-0 p-79 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 v-20 p-52 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 v-20 o-18 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-12 d-6 v-20 o-18 t-35 i-0 p-67 i-0 p-76 v-20 p-52 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-55 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 v-20 o-18 v-20 o-18 t-35 i-0 p-59 o-0 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 b-1 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 p-43 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-72 v-20 o-6 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-36 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 i-60 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-72 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-24 d-6 v-20 o-12 t-35 i-0 p-72 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 v-20 p-50 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-60 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-48 d-6 v-20 p-48 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-76 v-20 o-6 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-72 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-69 v-20 o-18 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-52 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-24 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-45 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 o-18 t-35 i-0 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-6 t-35 i-0 p-69 v-20 p-48 d-6 v-20 p-48 d-6 v-20 o-18 t-35 i-0 v-20 o-6 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 v-20 o-18 t-35 i-0 p-69 v-20 o-18 t-35 i-0 p-74 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 o-18 t-35 i-0 p-69 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 o-18 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-36 i-0 p-76 d-6 v-20 o-6 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-48 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-18 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-60 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-6 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 o-6 t-35 i-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 o-0 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0"
# model_result_str = "s-8 o-0 t-35 i-12 p-72 d-18 v-20 p-40 d-12 v-16 o-12 t-35 i-12 p-43 d-12 v-16 o-24 t-35 i-12 p-67 d-6 v-20 p-48 d-12 v-16 o-30 t-35 i-12 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-12 p-79 d-12 v-20 p-40 d-12 v-16 o-12 t-35 i-12 p-76 d-12 v-20 p-46 d-12 v-16 o-24 t-35 i-12 p-72 d-12 v-20 p-48 d-12 v-16 b-1 s-8 o-0 t-35 i-12 p-71 d-12 v-20 p-41 d-12 v-16 o-12 t-35 i-12 p-69 d-12 v-20 p-48 d-12 v-16 o-24 t-35 i-12 p-77 d-6 v-20 p-53 d-12 v-16 o-30 t-35 i-12 p-77 d-6 v-20 b-1 s-8 o-0 t-35 i-12 p-76 d-12 v-20 p-43 d-12 v-16 o-12 t-35 i-12 p-72 d-12 v-18 p-48 d-12 v-14 o-24 t-35 i-12 p-74 d-12 v-17 p-50 d-12 v-13 b-1 s-8 o-0 t-35 i-12 p-72 d-18 v-16 p-52 d-18 v-12 o-24 t-35 i-12 p-76 d-6 v-16 o-30 t-35 i-12 p-77 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-79 d-6 v-16 p-40 d-15 v-12 o-6 t-35 i-12 p-81 d-6 v-16 o-12 t-35 i-12 p-79 d-6 v-16 o-18 t-35 i-12 p-76 d-6 v-16 p-36 d-6 v-12 o-24 t-35 i-12 p-72 d-6 v-16 p-31 d-12 v-12 o-30 t-35 i-12 p-76 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-77 d-6 v-16 p-35 d-15 v-12 o-6 t-35 i-12 p-74 d-6 v-16 o-12 t-35 i-12 p-79 d-6 v-16 o-18 t-35 i-12 p-77 d-6 v-16 p-38 d-6 v-12 o-24 t-35 i-12 p-74 d-6 v-16 p-41 d-12 v-12 o-30 t-35 i-12 p-71 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-67 d-4 v-16 p-43 d-12 v-12 o-4 t-35 i-12 p-69 d-4 v-16 o-8 t-35 i-12 p-71 d-4 v-16 o-12 t-35 i-12 p-74 d-4 v-17 p-38 d-16 v-13 o-16 t-35 i-12 p-77 d-4 v-17 o-20 t-35 i-12 p-78 d-4 v-18 o-24 t-35 i-12 p-79 d-4 v-18 o-28 t-35 i-12 p-81 d-4 v-19 o-32 t-35 i-12 p-82 d-4 v-19 p-41 d-4 v-15 b-1 s-8 o-0 t-35 i-12 p-83 d-18 v-20 p-43 d-18 v-16 o-24 t-35 i-12 p-67 d-6 v-24 o-30 t-35 i-12 p-67 d-6 v-24 b-1 s-8 o-0 t-35 i-12 p-69 d-12 v-24 p-40 d-12 v-20 o-12 t-35 i-12 p-67 d-12 v-24 p-43 d-12 v-20 o-24 t-35 i-12 p-72 d-12 v-24 p-64 d-6 v-16 p-48 d-12 v-20 o-30 t-35 i-12 p-64 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-71 d-18 v-24 p-65 d-6 v-16 p-38 d-12 v-20 o-6 t-35 i-12 p-62 d-6 v-16 o-12 t-35 i-12 p-67 d-6 v-16 p-43 d-12 v-20 o-18 t-35 i-12 p-65 d-6 v-16 o-24 t-35 i-12 p-67 d-6 v-24 p-62 d-6 v-16 p-47 d-12 v-20 o-30 t-35 i-12 p-67 d-6 v-24 p-59 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-69 d-12 v-24 p-65 d-18 v-16 p-41 d-12 v-20 o-12 t-35 i-12 p-67 d-12 v-24 p-47 d-12 v-20 o-24 t-35 i-12 p-74 d-12 v-24 p-65 d-6 v-16 p-50 d-12 v-20 o-30 t-35 i-12 p-65 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-72 d-18 v-24 p-67 d-6 v-16 p-40 d-12 v-20 o-6 t-35 i-12 p-64 d-6 v-16 o-12 t-35 i-12 p-67 d-6 v-16 p-43 d-12 v-20 o-18 t-35 i-12 p-69 d-6 v-16 o-24 t-35 i-12 p-67 d-6 v-24 p-64 d-6 v-16 p-48 d-12 v-20 o-30 t-35 i-12 p-67 d-6 v-24 p-60 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-79 d-12 v-24 p-67 d-18 v-16 p-40 d-12 v-20 o-12 t-35 i-12 p-76 d-12 v-24 p-46 d-12 v-20 o-24 t-35 i-12 p-72 d-12 v-24 p-67 d-6 v-16 p-48 d-12 v-20 o-30 t-35 i-12 p-67 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-71 d-12 v-24 p-65 d-6 v-16 p-41 d-12 v-20 o-6 t-35 i-12 p-64 d-6 v-16 o-12 t-35 i-12 p-69 d-12 v-24 p-65 d-6 v-16 p-48 d-12 v-20 o-18 t-35 i-12 p-60 d-6 v-16 o-24 t-35 i-12 p-77 d-6 v-24 p-57 d-6 v-16 p-53 d-12 v-20 o-30 t-35 i-12 p-77 d-6 v-24 p-60 d-6 v-16 b-1 s-8 o-0 t-35 i-12 p-76 d-12 v-24 p-67 d-12 v-16 p-43 d-12 v-20 o-12 t-35 i-12 p-72 d-12 v-24 p-64 d-6 v-16 p-48 d-12 v-20 o-18 t-35 i-12 p-65 d-6 v-16 o-24 t-35 i-12 p-74 d-12 v-24 p-67 d-6 v-16 p-50 d-12 v-20 o-30 t-35 i-12 p-62 d-6 v-16 b-1"
# model_result_str = "d-6 v-20 o-48 t-25 p-68 d-27 t-36 i-40 p-72 i-24 p-64 d-6 v-20 o-48 t-39 i-25 p-57 p-67 d-6 v-20 o-36 i-0 p-62 d-6 v-20 t-29 d-6 v-20 t-38 i-90 i-22 i-52 p-69 d-3 v-20 t-25 i-16 d-6 v-20 t-36 i-0 p-57 d-12 v-20 t-39 i-52 t-36 i-16 p-69 d-6 v-20 t-26 p-62 d-27 v-17 o-48 t-33 i-40 p-68 d-6 v-20 t-36 i-3 p-60 v-17 o-48 t-33 i-29 p-64 d-6 v-20 t-38 t-30 i-2 p-74 t-29 p-71 d-6 v-20 t-31 d-6 v-20 t-41 i-24 p-58 i-22 p-62 d-6 v-20 t-35 i-41 p-67 d-18 v-17 o-0 t-35 i-24 p-53 d-16 i-3 p-65 d-3 v-20 t-28 i-0 i-0 p-62 d-27 v-17 t-39 i-0 p-71 d-9 v-20 t-36 i-0 p-27 d-27 v-17 o-0 t-29 i-41 p-55 i-16 d-27 v-17 o-96 d-12 v-17 t-36 i-27 t-36 i-52 d-18 v-17 t-30 i-25 p-62 p-62 d-18 v-20 t-36 i-40 p-35 d-27 v-17 t-35 i-24 p-60 d-6 v-20 t-38 d-18 v-17 t-36 i-2 p-70 p-69 d-6 v-20 t-36 i-0 p-57 d-18 i-24 p-51 d-27 v-17 t-36 i-61 p-62 d-6 v-17 t-41 i-3 d-6 v-20 t-38 i-21 p-74 v-17 t-36 i-2 p-72 v-20 t-41 d-6 v-20 t-38 i-16 p-70 d-3 v-20 t-38 i-27 p-70 t-33 i-90 p-28 d-6 v-20 t-41 i-25 i-16 p-66 p-55 d-18 v-17 t-36 i-61 p-62 d-6 v-17 o-0 t-35 p-59 p-60 v-17 t-36 i-52 t-38 d-21 v-17 t-29 i-16 i-53 p-55 d-27 v-17 t-31 d-27 v-17 o-69 t-35 d-15 v-17 t-38 p-62 d-15 d-27 v-17 t-48 i-61 p-71 v-17 t-35 i-41 p-71 d-6 v-17 t-36 i-3 d-12 v-17 d-12 d-27 v-17 t-29 i-8 p-62 d-27 v-17 t-27 p-60 d-27 v-17 t-34 i-0 p-65 d-18 v-17 o-72 t-36 i-4 p-67 d-27 v-17 t-38 i-90 d-6 v-17 t-36 i-3 i-41 p-66 d-18 v-17 t-36 i-3 p-67 d-27 v-17 t-37 i-53 p-62 d-12 v-17 t-42 i-90 p-59 d-18 v-17 t-35 i-8 t-34 i-61 i-0 p-68 d-18 v-17 t-40 i-61 d-6 v-17 t-39 p-65 d-27 v-17 t-35 i-0 v-17 t-36 i-41 p-55 d-27 v-17 t-35 i-90 p-26 v-17 t-35 p-64 d-18 v-17 t-42 d-15 v-17 t-38 p-67 d-18 v-17 t-39 i-90 p-34 d-27 v-17 t-38 i-90 d-9 v-20 t-24 i-21 p-55 i-16 p-67 d-21 v-17 t-32 i-90 p-69 d-24 p-61 d-18 v-17 t-24 i-90 d-18 v-17 t-35 i-61 p-48 d-12 v-20 t-35 i-0 p-31 d-27 v-17 t-35 i-8 p-60 d-27 v-17 t-29 p-77 p-63 p-63 d-6 v-17 t-33 i-21 p-55 v-17 t-23 v-17 t-36 i-0 p-60 d-27 v-17 t-36 i-2 p-59 d-12 v-17 d-27 v-17 d-12 v-17 t-32 i-4 p-56 d-27 v-17 t-33 p-74 d-18 v-17 t-33 d-27 v-17 t-36 i-2 t-35 i-90 p-59 d-27 v-17 t-36 i-8 p-29 d-18 v-17 t-36 i-90 p-55 d-27 v-17 t-35 i-8 p-58 p-55 d-27 v-17 t-36 i-61 p-62 d-18 v-17 t-38 i-21 i-21 p-37 d-27 v-17 t-36 i-0 p-59 d-12 v-17 t-36 i-61 p-52 d-27 v-17 t-35 i-41 p-32 v-17 t-35 i-3 p-59 d-18 v-17 t-36 i-3 d-12 v-17 d-27 v-17 t-38 i-90 d-18 i-90 i-90 p-60 d-18 v-17 t-35 d-18 v-17 t-35 i-2 d-27 v-17 t-36 i-90 p-68 d-27 v-17 t-35 i-4 v-17 t-36 i-41 p-57 v-17 t-33 d-27 v-17 t-32 i-90 d-27 v-17 t-32 i-90 p-51 d-27 v-17 t-33 d-18 v-17 t-35 i-8 p-60 d-27 v-17 t-37 v-17 t-36 i-3 p-67 d-18 i-4 d-18 v-17 t-35 i-4 p-65 d-27 v-17 t-40 i-4 d-27 v-17 t-36 i-0 p-61 d-18 v-17 t-35 d-15 v-17 t-33 i-61 p-63 d-18 v-17 d-27 v-17 t-33 i-8 p-70 d-12 v-17 t-39 d-27 v-17 t-33 d-37 v-17 t-36 i-21 p-52 d-18 v-17 t-35 i-4 i-21 p-67 d-27 v-17 t-36 d-27 v-17 t-33 d-18 v-17 t-33 i-3 i-0 v-17 t-36 v-17 t-43 d-18 v-17 t-29 i-41 i-8 d-6 v-17 t-36 i-41 p-62 v-17 t-35 i-21 i-3 i-41 p-63 d-27 v-17 t-35 i-41 p-74 v-17 t-36 i-61 p-59 d-12 v-17 t-33 v-17 t-41 i-41 t-37 i-21 p-62 d-27 v-17 t-32 i-8 p-65 i-21 i-21 p-71 d-18 i-3 t-30 i-2 i-61 p-57 p-57 d-18 v-17 t-34 i-2 d-27 v-17 t-33 i-3 d-18 v-17 d-27 v-17 t-36 i-3 p-65 d-18 v-17 t-41 d-18 v-17 t-33 i-90 p-61 i-3 p-70 d-27 v-17 t-37 i-3 p-31 i-8 p-36 d-27 v-17 t-36 p-76 d-27 v-17 t-35 d-18 v-17 t-37 i-3 p-59 p-59 p-33 d-27 v-17 t-37 i-41 p-56 d-18 v-17 t-35 i-16 d-27 v-17 t-36 i-41 p-61 d-18 v-17 i-61 p-30 v-17 d-27 v-17 t-33 i-21 p-67 d-27 v-17 t-35 i-8 p-68 d-27 v-17 t-36 v-17 t-36 i-8 p-61 d-15 v-17 t-36 i-0 p-67 d-18 v-17 t-33 i-3 p-57 d-18 v-17 t-38 d-18 v-17 t-38 i-3 p-38 d-18 v-17 t-36 i-16 p-71 d-27 v-17 t-43 d-18 v-17 t-35 v-17 v-17 t-24 i-8 i-8 p-59 d-27 v-17 t-41 d-18 v-17 t-33 p-31 p-66 d-12 v-17 i-61 d-27 v-17 t-36 i-90 v-17 t-33 i-61 v-17 t-36 i-41 d-18 v-17 i-61 d-12 v-17 t-33 v-17 t-33 i-61 p-65 v-17 t-36 i-61 d-12 v-17 t-33 i-3 i-2 p-62 d-27 v-17 t-35 i-2 i-2 p-67 d-27 v-17 t-38 i-2 p-60 d-18 v-17 t-41 i-2 d-27 v-17 t-23 i-3 p-76 d-27 v-17 t-33 d-27 v-17 t-35 i-2 p-28 d-37 v-17 t-33 d-27 v-17 t-43 i-16 p-35 d-27 v-17 t-43 d-18 v-17 t-33 i-8 d-27 v-17 t-33 v-17 t-36 i-61 p-35 d-27 v-17 t-36 i-41 p-33 d-18 v-17 v-17 t-29 i-2 p-31 d-6 v-20 t-36 i-41 p-65 d-27 v-17 i-4 d-27 v-17 d-27 v-17 t-30 i-16 p-68 d-12 v-17 t-23 v-17 t-41 i-4 p-39 p-63 p-60 d-27 v-17 t-37 i-8 p-36 d-18 v-17 t-33 i-90 p-59 d-27 v-17 t-36 i-3 p-52 d-27 v-17 t-30 v-17 t-36 i-3 d-27 v-17 t-36 i-3 v-17 t-33 i-61 v-17 t-35 i-4 p-59 d-27 v-17 v-17 t-33 d-18 v-17 t-35 i-3 p-55 d-18 v-17 t-33 i-8 d-27 v-17 t-36 i-90 p-60 d-27 v-17 t-30 i-61 i-8 p-67 d-18 v-17 t-33 i-90 p-33 d-27 v-17 t-37 i-8 p-62 d-18 v-17 t-36 i-4 i-90 d-18 v-17 t-37 v-17 t-41 p-64 d-27 v-17 t-35 i-2 p-60 d-18 v-17 t-36 i-16 d-12 v-17 t-33 i-61 v-17 t-29 v-17 t-38 v-17 t-33 i-3 d-12 v-20 t-28 i-2 p-37 t-37 i-41 p-33 d-18 v-17 d-27 v-17 t-33 d-27 v-17 t-36 i-0 i-21 p-55 d-27 v-17 t-33 d-25 v-17 t-33 d-18 v-17 t-35 i-41 d-18 v-17 t-33 v-17 t-35 i-21 p-53 p-65 d-27 v-17 t-33 p-28 i-41 v-17 v-17 t-36 i-2 v-17 t-33 i-90 p-38 d-18 v-17 t-40 i-21 d-18 v-17 t-33 i-61 v-17 t-42 d-18 v-17 t-35 i-4 p-57 d-27 v-17 t-36 d-27 v-17 d-15 d-27 v-17 t-36 i-3 p-69 d-18 v-17 t-33 v-17 t-33 v-17 t-36 i-61 p-34 d-27 v-17 t-33 v-17 t-33 v-17 t-33 i-61 p-31 d-27 v-17 t-34 i-21 d-18 d-18 v-17 t-33 d-18 v-17 t-41 i-16 p-69 d-27 v-17 t-35 i-8 p-38 d-18 v-17 t-33 i-21 p-38 v-17 t-33 i-3 p-53 d-12 v-17 t-33 i-8 p-31 d-27 v-17 i-41 p-66 d-27 v-17 t-37 i-3 d-27 v-17 t-33 i-41 p-65 d-27 v-17 d-18 v-17 t-33 i-21 i-21 d-27 v-17 d-18 v-17 t-36 i-16 p-35 i-3 v-17 t-29 d-18 v-17 t-33 i-21 d-18 v-17 t-31 v-17 t-33 d-6 v-17 t-33 i-3 p-27 v-17 t-38 i-41 v-17 i-8 p-60 v-17 t-36 i-41 p-70 d-18 v-17 d-27 v-17 t-33 i-41 p-69 d-18 v-17 t-33 d-18 v-17 t-30 i-21 p-38 d-18 v-17 t-36 i-4 d-27 v-17 t-36 i-21 p-63 i-21 v-17 t-41 i-3 d-27 v-17 t-33 i-8 d-27 v-17 t-28 i-3 p-63 d-15 v-17 t-32 i-3 p-68 d-27 v-17 t-36 i-3 p-28 i-4 p-62 v-17 t-42 i-3 p-55 d-12 v-17 t-33 i-61 v-17 t-35 i-41 i-41 d-27 v-17 t-36 i-2 v-17 t-33 i-16 p-56 v-17 t-36 i-2 p-62 d-18 v-17 t-36 i-41 d-27 v-17 t-33 v-17 i-61 p-39 i-90 v-17 t-41 d-18 v-17 i-61 d-27 v-17 t-23 v-17 t-33 d-18 v-17 t-30 i-21 p-59 d-18 v-17 t-33 i-90 d-18 v-17 t-42 i-41 p-57 v-17 t-35 i-21 p-57 d-18 v-17 t-29 d-27 v-17 t-33 i-90 d-27 v-17 t-36 i-61 v-17 t-36 i-2 v-17 t-36 i-4 p-66 i-90 d-18 v-17 v-17 t-33 d-18 v-17 t-33 i-4 p-68 d-27 v-17 t-41 i-8 d-27 v-17 t-36 i-3 p-73 i-3 p-65 d-27 v-17 t-33 i-90 p-67 i-2 i-2 p-60 d-18 v-17 t-33 i-21 d-6 p-77 v-17 t-33 i-2 i-2 t-33 i-90 p-61 d-18 v-17 t-43 v-17 t-42 d-27 v-17 i-4 p-57 d-18 v-17 t-33 i-61 v-17 t-33 i-21 d-27 v-17 t-33 i-3 i-3 d-18 v-17 i-21 p-36 d-27 v-17 t-36 i-8 p-33 d-27 v-17 t-33 i-8 p-59 d-27 v-17 t-33 i-90 d-18 v-17 t-33 d-18 v-17 t-35 i-3 p-57 d-12 v-17 i-90 p-69 d-18 d-18 p-64 d-18 v-17 t-36 d-27 v-17 t-33 i-90 d-18 d-18 v-17 t-33 i-4 p-31 d-18 i-3 i-21 p-34 v-17 i-2 p-36 v-17 t-31 d-15 v-17 t-36 i-90 d-27 v-17 d-27 v-17 t-39 i-2 v-17 t-33 d-18 v-17 t-41 i-21 p-31 v-17 t-31 i-90 d-27 v-17 t-36 i-21 i-61 d-27 v-17 t-32 i-21 p-33 d-18 v-17 t-36 i-3 t-33 i-90 p-60 d-18 v-17 t-29 i-16 p-34 d-18 v-17 t-45 d-27 v-17 t-36 i-21 p-62 d-27 v-17 t-33 v-17 t-36 i-0 v-17 t-36 d-27 v-17 d-27 v-17 d-12 v-17 t-35 i-8 p-57 i-4 v-17 t-25 i-3 p-61 d-12 v-17 t-42 i-21 i-4 v-17 t-29 d-27 v-17 t-33 d-27 v-17 t-39 i-3 p-57 d-18 v-17 i-21 p-57 d-27 v-17 t-33 i-3 i-21 p-31 d-27 v-17 d-27 v-17 t-36 i-2 v-17 t-33 i-21 d-27 v-17 t-33 d-18 v-17 t-33 i-3 d-27 v-17 t-32 d-18 v-17 t-38 v-17 t-33 i-3 p-70 d-18 v-17 t-36 i-3 p-67 d-27 v-17 d-27 v-17 t-36 i-4 p-64 d-27 v-17 d-27 v-17 t-33 v-17 t-33 i-4 p-32 d-18 v-17 t-36 v-17 t-33 i-3 p-67 d-27 v-17 t-35 i-41 d-27 v-17 t-33 v-17 i-0 p-42 d-18 v-17 t-33 i-2 p-52 p-55 d-12 v-17 t-36 i-4 p-39 d-18 v-17 t-30 p-62 d-18 v-17 d-27 v-17 t-35 i-3 p-66 d-6 v-17 t-33 d-18 v-17 t-39 i-90 p-64 d-18 v-17 t-33 d-18 p-60 d-27 v-17 t-33"
# model_result_str = "o-48 t-25 p-68 d-27 t-36 i-40 p-72 i-24 p-64 d-6 v-20 o-48 t-39 i-25 p-57 p-67 d-6 v-20 o-36 i-0 p-62 d-6 v-20 t-29 d-6 v-20 t-38 i-90 i-22 i-52 p-69 d-3 v-20 t-25 i-16 d-6 v-20 t-36 i-0 p-57 d-12 v-20 t-39 i-52 t-36 i-16 p-69 d-6 v-20 t-26 p-62 d-27 v-17 o-48 t-33 i-40 p-68 d-6 v-20 t-36 i-3 p-60 v-17 o-48 t-33 i-29 p-64 d-6 v-20 t-38 t-30 i-2 p-74 t-29 p-71 d-6 v-20 t-31 d-6 v-20 t-41 i-24 p-58 i-22 p-62 d-6 v-20 t-35 i-41 p-67 d-18 v-17 o-0 t-35 i-24 p-53 d-16 i-3 p-65 d-3 v-20 t-28 i-0 i-0 p-62 d-27 v-17 t-39 i-0 p-71 d-9 v-20 t-36 i-0 p-27 d-27 v-17 o-0 t-29 i-41 p-55 i-16 d-27 v-17 o-96 d-12 v-17 t-36 i-27 t-36 i-52 d-18 v-17 t-30 i-25 p-62 p-62 d-18 v-20 t-36 i-40 p-35 d-27 v-17 t-35 i-24 p-60 d-6 v-20 t-38 d-18 v-17 t-36 i-2 p-70 p-69 d-6 v-20 t-36 i-0 p-57 d-18 i-24 p-51 d-27 v-17 t-36 i-61 p-62 d-6 v-17 t-41 i-3 d-6 v-20 t-38 i-21 p-74 v-17 t-36 i-2 p-72 v-20 t-41 d-6 v-20 t-38 i-16 p-70 d-3 v-20 t-38 i-27 p-70 t-33 i-90 p-28 d-6 v-20 t-41 i-25 i-16 p-66 p-55 d-18 v-17 t-36 i-61 p-62 d-6 v-17 o-0 t-35 p-59 p-60 v-17 t-36 i-52 t-38 d-21 v-17 t-29 i-16 i-53 p-55 d-27 v-17 t-31 d-27 v-17 o-69 t-35 d-15 v-17 t-38 p-62 d-15 d-27 v-17 t-48 i-61 p-71 v-17 t-35 i-41 p-71 d-6 v-17 t-36 i-3 d-12 v-17 d-12 d-27 v-17 t-29 i-8 p-62 d-27 v-17 t-27 p-60 d-27 v-17 t-34 i-0 p-65 d-18 v-17 o-72 t-36 i-4 p-67 d-27 v-17 t-38 i-90 d-6 v-17 t-36 i-3 i-41 p-66 d-18 v-17 t-36 i-3 p-67 d-27 v-17 t-37 i-53 p-62 d-12 v-17 t-42 i-90 p-59 d-18 v-17 t-35 i-8 t-34 i-61 i-0 p-68 d-18 v-17 t-40 i-61 d-6 v-17 t-39 p-65 d-27 v-17 t-35 i-0 v-17 t-36 i-41 p-55 d-27 v-17 t-35 i-90 p-26 v-17 t-35 p-64 d-18 v-17 t-42 d-15 v-17 t-38 p-67 d-18 v-17 t-39 i-90 p-34 d-27 v-17 t-38 i-90 d-9 v-20 t-24 i-21 p-55 i-16 p-67 d-21 v-17 t-32 i-90 p-69 d-24 p-61 d-18 v-17 t-24 i-90 d-18 v-17 t-35 i-61 p-48 d-12 v-20 t-35 i-0 p-31 d-27 v-17 t-35 i-8 p-60 d-27 v-17 t-29 p-77 p-63 p-63 d-6 v-17 t-33 i-21 p-55 v-17 t-23 v-17 t-36 i-0 p-60 d-27 v-17 t-36 i-2 p-59 d-12 v-17 d-27 v-17 d-12 v-17 t-32 i-4 p-56 d-27 v-17 t-33 p-74 d-18 v-17 t-33 d-27 v-17 t-36 i-2 t-35 i-90 p-59 d-27 v-17 t-36 i-8 p-29 d-18 v-17 t-36 i-90 p-55 d-27 v-17 t-35 i-8 p-58 p-55 d-27 v-17 t-36 i-61 p-62 d-18 v-17 t-38 i-21 i-21 p-37 d-27 v-17 t-36 i-0 p-59 d-12 v-17 t-36 i-61 p-52 d-27 v-17 t-35 i-41 p-32 v-17 t-35 i-3 p-59 d-18 v-17 t-36 i-3 d-12 v-17 d-27 v-17 t-38 i-90 d-18 i-90 i-90 p-60 d-18 v-17 t-35 d-18 v-17 t-35 i-2 d-27 v-17 t-36 i-90 p-68 d-27 v-17 t-35 i-4 v-17 t-36 i-41 p-57 v-17 t-33 d-27 v-17 t-32 i-90 d-27 v-17 t-32 i-90 p-51 d-27 v-17 t-33 d-18 v-17 t-35 i-8 p-60 d-27 v-17 t-37 v-17 t-36 i-3 p-67 d-18 i-4 d-18 v-17 t-35 i-4 p-65 d-27 v-17 t-40 i-4 d-27 v-17 t-36 i-0 p-61 d-18 v-17 t-35 d-15 v-17 t-33 i-61 p-63 d-18 v-17 d-27 v-17 t-33 i-8 p-70 d-12 v-17 t-39 d-27 v-17 t-33 d-37 v-17 t-36 i-21 p-52 d-18 v-17 t-35 i-4 i-21 p-67 d-27 v-17 t-36 d-27 v-17 t-33 d-18 v-17 t-33 i-3 i-0 v-17 t-36 v-17 t-43 d-18 v-17 t-29 i-41 i-8 d-6 v-17 t-36 i-41 p-62 v-17 t-35 i-21 i-3 i-41 p-63 d-27 v-17 t-35 i-41 p-74 v-17 t-36 i-61 p-59 d-12 v-17 t-33 v-17 t-41 i-41 t-37 i-21 p-62 d-27 v-17 t-32 i-8 p-65 i-21 i-21 p-71 d-18 i-3 t-30 i-2 i-61 p-57 p-57 d-18 v-17 t-34 i-2 d-27 v-17 t-33 i-3 d-18 v-17 d-27 v-17 t-36 i-3 p-65 d-18 v-17 t-41 d-18 v-17 t-33 i-90 p-61 i-3 p-70 d-27 v-17 t-37 i-3 p-31 i-8 p-36 d-27 v-17 t-36 p-76 d-27 v-17 t-35 d-18 v-17 t-37 i-3 p-59 p-59 p-33 d-27 v-17 t-37 i-41 p-56 d-18 v-17 t-35 i-16 d-27 v-17 t-36 i-41 p-61 d-18 v-17 i-61 p-30 v-17 d-27 v-17 t-33 i-21 p-67 d-27 v-17 t-35 i-8 p-68 d-27 v-17 t-36 v-17 t-36 i-8 p-61 d-15 v-17 t-36 i-0 p-67 d-18 v-17 t-33 i-3 p-57 d-18 v-17 t-38 d-18 v-17 t-38 i-3 p-38 d-18 v-17 t-36 i-16 p-71 d-27 v-17 t-43 d-18 v-17 t-35 v-17 v-17 t-24 i-8 i-8 p-59 d-27 v-17 t-41 d-18 v-17 t-33 p-31 p-66 d-12 v-17 i-61 d-27 v-17 t-36 i-90 v-17 t-33 i-61 v-17 t-36 i-41 d-18 v-17 i-61 d-12 v-17 t-33 v-17 t-33 i-61 p-65 v-17 t-36 i-61 d-12 v-17 t-33 i-3 i-2 p-62 d-27 v-17 t-35 i-2 i-2 p-67 d-27 v-17 t-38 i-2 p-60 d-18 v-17 t-41 i-2 d-27 v-17 t-23 i-3 p-76 d-27 v-17 t-33 d-27 v-17 t-35 i-2 p-28 d-37 v-17 t-33 d-27 v-17 t-43 i-16 p-35 d-27 v-17 t-43 d-18 v-17 t-33 i-8 d-27 v-17 t-33 v-17 t-36 i-61 p-35 d-27 v-17 t-36 i-41 p-33 d-18 v-17 v-17 t-29 i-2 p-31 d-6 v-20 t-36 i-41 p-65 d-27 v-17 i-4 d-27 v-17 d-27 v-17 t-30 i-16 p-68 d-12 v-17 t-23 v-17 t-41 i-4 p-39 p-63 p-60 d-27 v-17 t-37 i-8 p-36 d-18 v-17 t-33 i-90 p-59 d-27 v-17 t-36 i-3 p-52 d-27 v-17 t-30 v-17 t-36 i-3 d-27 v-17 t-36 i-3 v-17 t-33 i-61 v-17 t-35 i-4 p-59 d-27 v-17 v-17 t-33 d-18 v-17 t-35 i-3 p-55 d-18 v-17 t-33 i-8 d-27 v-17 t-36 i-90 p-60 d-27 v-17 t-30 i-61 i-8 p-67 d-18 v-17 t-33 i-90 p-33 d-27 v-17 t-37 i-8 p-62 d-18 v-17 t-36 i-4 i-90 d-18 v-17 t-37 v-17 t-41 p-64 d-27 v-17 t-35 i-2 p-60 d-18 v-17 t-36 i-16 d-12 v-17 t-33 i-61 v-17 t-29 v-17 t-38 v-17 t-33 i-3 d-12 v-20 t-28 i-2 p-37 t-37 i-41 p-33 d-18 v-17 d-27 v-17 t-33 d-27 v-17 t-36 i-0 i-21 p-55 d-27 v-17 t-33 d-25 v-17 t-33 d-18 v-17 t-35 i-41 d-18 v-17 t-33 v-17 t-35 i-21 p-53 p-65 d-27 v-17 t-33 p-28 i-41 v-17 v-17 t-36 i-2 v-17 t-33 i-90 p-38 d-18 v-17 t-40 i-21 d-18 v-17 t-33 i-61 v-17 t-42 d-18 v-17 t-35 i-4 p-57 d-27 v-17 t-36 d-27 v-17 d-15 d-27 v-17 t-36 i-3 p-69 d-18 v-17 t-33 v-17 t-33 v-17 t-36 i-61 p-34 d-27 v-17 t-33 v-17 t-33 v-17 t-33 i-61 p-31 d-27 v-17 t-34 i-21 d-18 d-18 v-17 t-33 d-18 v-17 t-41 i-16 p-69 d-27 v-17 t-35 i-8 p-38 d-18 v-17 t-33 i-21 p-38 v-17 t-33 i-3 p-53 d-12 v-17 t-33 i-8 p-31 d-27 v-17 i-41 p-66 d-27 v-17 t-37 i-3 d-27 v-17 t-33 i-41 p-65 d-27 v-17 d-18 v-17 t-33 i-21 i-21 d-27 v-17 d-18 v-17 t-36 i-16 p-35 i-3 v-17 t-29 d-18 v-17 t-33 i-21 d-18 v-17 t-31 v-17 t-33 d-6 v-17 t-33 i-3 p-27 v-17 t-38 i-41 v-17 i-8 p-60 v-17 t-36 i-41 p-70 d-18 v-17 d-27 v-17 t-33 i-41 p-69 d-18 v-17 t-33 d-18 v-17 t-30 i-21 p-38 d-18 v-17 t-36 i-4 d-27 v-17 t-36 i-21 p-63 i-21 v-17 t-41 i-3 d-27 v-17 t-33 i-8 d-27 v-17 t-28 i-3 p-63 d-15 v-17 t-32 i-3 p-68 d-27 v-17 t-36 i-3 p-28 i-4 p-62 v-17 t-42 i-3 p-55 d-12 v-17 t-33 i-61 v-17 t-35 i-41 i-41 d-27 v-17 t-36 i-2 v-17 t-33 i-16 p-56 v-17 t-36 i-2 p-62 d-18 v-17 t-36 i-41 d-27 v-17 t-33 v-17 i-61 p-39 i-90 v-17 t-41 d-18 v-17 i-61 d-27 v-17 t-23 v-17 t-33 d-18 v-17 t-30 i-21 p-59 d-18 v-17 t-33 i-90 d-18 v-17 t-42 i-41 p-57 v-17 t-35 i-21 p-57 d-18 v-17 t-29 d-27 v-17 t-33 i-90 d-27 v-17 t-36 i-61 v-17 t-36 i-2 v-17 t-36 i-4 p-66 i-90 d-18 v-17 v-17 t-33 d-18 v-17 t-33 i-4 p-68 d-27 v-17 t-41 i-8 d-27 v-17 t-36 i-3 p-73 i-3 p-65 d-27 v-17 t-33 i-90 p-67 i-2 i-2 p-60 d-18 v-17 t-33 i-21 d-6 p-77 v-17 t-33 i-2 i-2 t-33 i-90 p-61 d-18 v-17 t-43 v-17 t-42 d-27 v-17 i-4 p-57 d-18 v-17 t-33 i-61 v-17 t-33 i-21 d-27 v-17 t-33 i-3 i-3 d-18 v-17 i-21 p-36 d-27 v-17 t-36 i-8 p-33 d-27 v-17 t-33 i-8 p-59 d-27 v-17 t-33 i-90 d-18 v-17 t-33 d-18 v-17 t-35 i-3 p-57 d-12 v-17 i-90 p-69 d-18 d-18 p-64 d-18 v-17 t-36 d-27 v-17 t-33 i-90 d-18 d-18 v-17 t-33 i-4 p-31 d-18 i-3 i-21 p-34 v-17 i-2 p-36 v-17 t-31 d-15 v-17 t-36 i-90 d-27 v-17 d-27 v-17 t-39 i-2 v-17 t-33 d-18 v-17 t-41 i-21 p-31 v-17 t-31 i-90 d-27 v-17 t-36 i-21 i-61 d-27 v-17 t-32 i-21 p-33 d-18 v-17 t-36 i-3 t-33 i-90 p-60 d-18 v-17 t-29 i-16 p-34 d-18 v-17 t-45 d-27 v-17 t-36 i-21 p-62 d-27 v-17 t-33 v-17 t-36 i-0 v-17 t-36 d-27 v-17 d-27 v-17 d-12 v-17 t-35 i-8 p-57 i-4 v-17 t-25 i-3 p-61 d-12 v-17 t-42 i-21 i-4 v-17 t-29 d-27 v-17 t-33 d-27 v-17 t-39 i-3 p-57 d-18 v-17 i-21 p-57 d-27 v-17 t-33 i-3 i-21 p-31 d-27 v-17 d-27 v-17 t-36 i-2 v-17 t-33 i-21 d-27 v-17 t-33 d-18 v-17 t-33 i-3 d-27 v-17 t-32 d-18 v-17 t-38 v-17 t-33 i-3 p-70 d-18 v-17 t-36 i-3 p-67 d-27 v-17 d-27 v-17 t-36 i-4 p-64 d-27 v-17 d-27 v-17 t-33 v-17 t-33 i-4 p-32 d-18 v-17 t-36 v-17 t-33 i-3 p-67 d-27 v-17 t-35 i-41 d-27 v-17 t-33 v-17 i-0 p-42 d-18 v-17 t-33 i-2 p-52 p-55 d-12 v-17 t-36 i-4 p-39 d-18 v-17 t-30 p-62 d-18 v-17 d-27 v-17 t-35 i-3 p-66 d-6 v-17 t-33 d-18 v-17 t-39 i-90 p-64 d-18 v-17 t-33 d-18 p-60 d-27 v-17 t-33"
# model_result_str = "d-12 v-20 o-30 t-35 i-60 p-62 o-30 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-56 p-72 v-24 b-1 s-9 o-0 t-35 i-40 p-81 d-11 v-20 i-60 p-64 d-6 v-20 o-6 t-35 i-58 p-64 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-55 d-6 v-20 i-128 p-170 v-24 p-59 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-18 v-16 i-57 v-20 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-58 p-64 d-6 v-20 o-24 t-35 i-71 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-43 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 v-20 i-57 p-48 v-20 o-24 d-6 v-20 i-43 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 i-128 p-170 d-6 v-20 p-52 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-18 t-35 i-48 d-12 v-20 b-1 s-9 o-0 t-35 i-52 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-58 p-48 t-35 i-0 p-69 d-6 v-24 i-67 t-35 i-0 p-67 d-6 v-20 p-64 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-38 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 o-18 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 p-64 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 p-57 d-6 v-20 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-52 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-48 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-60 d-12 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 t-35 i-0 p-69 d-12 v-20 b-1 v-20 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-60 d-6 v-20 i-128 p-170 d-6 v-20 p-60 d-6 v-20 p-62 d-6 v-20 p-48 d-6 v-20 o-6 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-6 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 p-74 i-0 p-67 d-6 v-20 p-164 d-6 v-20 o-30 t-35 i-0 p-62 v-20 p-60 d-6 v-20 b-1 t-33 i-128 p-170 d-6 v-20 i-128 p-173 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-76 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-64 p-64 d-6 v-20 o-6 t-35 i-0 p-77 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-48 t-35 i-0 p-64 d-6 v-20 b-1 d-6 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-76 d-6 v-24 i-42 t-35 i-0 p-76 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-60 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-0 d-6 v-20 o-18 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-173 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-164 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-60 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-128 p-170 d-6 v-20 o-36 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-64 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-64 d-6 v-20 i-128 p-170 v-20 o-30 t-35 i-0 v-20 b-1 v-20 o-12 t-35 i-0 p-67 d-6 v-20 i-0 p-60 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-64 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-71 v-20 b-1 s-9 o-0 t-35 i-40 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-76 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 i-128 p-170 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-38 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-18 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 d-6 v-20 p-60 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 i-128 t-35 i-0 p-67 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 i-128 p-170 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 p-170 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 o-24 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 i-128 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-59 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 o-36 t-35 i-0 p-74 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-79 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-64 d-6 v-20 i-0 p-67 d-6 v-20 o-18 t-35 i-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 d-6 v-20 p-62 d-6 v-20 i-48 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 i-128 p-173 t-35 i-0 d-6 v-20 i-128 p-170 t-30 i-0 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 o-0 t-35 i-0 p-57 d-6 v-20 v-20 i-48 t-35 i-0 t-30 i-0 d-6 v-20 o-18 t-35 t-35 i-0 d-6 v-20 t-35 i-0 d-6 v-20 p-43 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-55 d-6 v-20 p-55 d-11 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18"
# model_result_str = "o-30 t-35 i-60 p-62 o-30 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-56 p-72 v-24 b-1 s-9 o-0 t-35 i-40 p-81 d-11 v-20 i-60 p-64 d-6 v-20 o-6 t-35 i-58 p-64 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-55 d-6 v-20 i-128 p-170 v-24 p-59 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-18 v-16 i-57 v-20 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-58 p-64 d-6 v-20 o-24 t-35 i-71 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-43 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 v-20 i-57 p-48 v-20 o-24 d-6 v-20 i-43 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 i-128 p-170 d-6 v-20 p-52 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-18 t-35 i-48 d-12 v-20 b-1 s-9 o-0 t-35 i-52 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-58 p-48 t-35 i-0 p-69 d-6 v-24 i-67 t-35 i-0 p-67 d-6 v-20 p-64 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-38 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 o-18 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 p-64 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 p-57 d-6 v-20 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-52 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-48 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-60 d-12 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 t-35 i-0 p-69 d-12 v-20 b-1 v-20 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-60 d-6 v-20 i-128 p-170 d-6 v-20 p-60 d-6 v-20 p-62 d-6 v-20 p-48 d-6 v-20 o-6 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-6 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 p-74 i-0 p-67 d-6 v-20 p-164 d-6 v-20 o-30 t-35 i-0 p-62 v-20 p-60 d-6 v-20 b-1 t-33 i-128 p-170 d-6 v-20 i-128 p-173 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-76 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-64 p-64 d-6 v-20 o-6 t-35 i-0 p-77 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-48 t-35 i-0 p-64 d-6 v-20 b-1 d-6 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-76 d-6 v-24 i-42 t-35 i-0 p-76 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-60 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-0 d-6 v-20 o-18 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-173 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-164 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-60 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-128 p-170 d-6 v-20 o-36 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-64 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-64 d-6 v-20 i-128 p-170 v-20 o-30 t-35 i-0 v-20 b-1 v-20 o-12 t-35 i-0 p-67 d-6 v-20 i-0 p-60 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-64 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-71 v-20 b-1 s-9 o-0 t-35 i-40 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-76 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 i-128 p-170 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-38 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-18 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 d-6 v-20 p-60 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 i-128 t-35 i-0 p-67 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 i-128 p-170 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 p-170 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 o-24 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 i-128 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-59 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 o-36 t-35 i-0 p-74 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-79 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-64 d-6 v-20 i-0 p-67 d-6 v-20 o-18 t-35 i-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 d-6 v-20 p-62 d-6 v-20 i-48 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 i-128 p-173 t-35 i-0 d-6 v-20 i-128 p-170 t-30 i-0 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 o-0 t-35 i-0 p-57 d-6 v-20 v-20 i-48 t-35 i-0 t-30 i-0 d-6 v-20 o-18 t-35 t-35 i-0 d-6 v-20 t-35 i-0 d-6 v-20 p-43 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-55 d-6 v-20 p-55 d-11 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18"
# model_result_str = "s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-9 v-20 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-6 v-20 p-52 d-6 v-20 p-48 d-3 v-24 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 p-48 d-3 v-24 p-33 d-6 v-20 o-33 t-33 i-57 p-48 d-3 v-24 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-12 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-12 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-50 d-9 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-24 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-50 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-30 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-33 t-33 i-57 p-52 d-6 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-48 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-43 d-3 v-20 o-42 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 p-45 d-3 v-20 o-45 t-33 i-57 p-50 d-13 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-12 t-33 i-57 p-53 d-9 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-21 t-33 i-57 p-53 d-3 v-24 o-24 t-33 i-57 p-53 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-39 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 p-42 d-3 v-20 o-42 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 p-41 d-3 v-20 p-41 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-47 d-3 v-20 p-47 d-3 v-20 o-9 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-15 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-21 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-27 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 p-48 d-6 v-20 p-48 d-6 v-20 o-42 t-33 i-57 p-45 d-6 v-20 p-45 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-41 d-3 v-20 p-41 d-3 v-20 p-38 d-6 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-47 d-3 v-20 p-47 d-3 v-20 o-9 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-9 v-20 o-15 t-33 i-57 p-50 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 o-21 t-33 i-57 p-48 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-36 d-9 v-20 o-24 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-27 t-33 i-57 p-48 d-3 v-24 o-30 t-33 i-57 p-48 d-3 v-24 p-33 d-6 v-20 o-33 t-33 i-57 p-48 d-3 v-24 o-36 t-33 i-57 p-48 d-12 v-24 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-48 d-12 v-20 p-40 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-15 t-33 i-57 p-50 d-6 v-24 p-45 d-6 v-20 p-45 d-6 v-20 p-38 d-6 v-20 o-21 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-30 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-33 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-43 d-12 v-20 p-43 d-12 v-20 p-36 d-12 v-20 b-1"
# model_result_str = "s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-42 d-6 v-24 p-55 d-6 v-20 i-60 p-57 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-60 d-12 d-12 v-20 o-6 p-55 d-12 v-20 b-1 s-9 o-0 t-35 i-24 d-6 v-20 o-24 t-35 i-0 p-36 d-6 v-20 b-1 s-9 o-0 t-35 i-9 d-18 v-20 b-1 s-8 o-0 t-35 i-40 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-9 d-6 v-20 o-24 t-35 i-24 d-12 v-24 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 o-12 t-35 d-6 v-24 d-6 v-20 b-1 s-9 o-0 t-35 i-58 d-12 v-20 b-1 s-9 o-0 t-35 d-6 v-24 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-60 p-59 o-0 t-35 i-12 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-9 d-6 v-20 p-60 d-12 t-34 v-20 o-12 t-35 i-48 p-69 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-14 d-12 v-20 b-1 s-9 o-0 t-35 i-48 o-0 t-35 i-57 p-55 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 i-58 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 i-128 t-35 i-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-12 d-6 v-20 i-57 d-6 v-20 b-1 s-9 o-0 t-35 i-58 p-52 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-57 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-69 p-62 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 p-57 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-57 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-14 t-35 i-57 p-59 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-48 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-24 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-24 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-12 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-56 i-52 p-72 t-35 i-58 t-35 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-12 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-48 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-12 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-30 t-35 i-58 p-57 d-6 v-20 o-24 t-35 i-56 p-60 d-6 v-20 b-1 s-7 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-72 d-6 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-128 p-187 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-35 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-62 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-164 d-6 v-20 o-24 d-12 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 o-0 t-35 i-57 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 o-6 t-35 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-8 o-0 t-35 i-48 p-48 d-6 v-20 p-57 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-77 d-6 v-20 o-18 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-43 d-12 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-56 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-14 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 v-20 p-55 d-6 v-20 b-1 s-3 t-35 i-0 p-57 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 o-24 t-35 i-0 p-76 t-35 i-58 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-164 d-6 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 d-6 v-20 b-1 s-8 o-0 t-34 i-0 t-35 i-0 p-72 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 p-64 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 d-6 v-20 d-12 v-20 o-12 t-35 i-58 d-6 v-20 o-24 t-35 i-0 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-71 d-12 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 v-20 b-1 s-9 o-0 t-35 i-0 p-77 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 p-48 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-76 v-20 p-53 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 o-42 t-35 i-128 p-170 d-6 v-20 p-64 d-6 v-20 b-1 s-8 v-20 p-57 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-24 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-57 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 i-48 v-20 p-57 o-0 t-35 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-57 o-0 t-35 i-0 p-64 d-6 v-20 p-60 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-48 d-6 v-20 p-57 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 o-0 t-35 i-58 d-6 v-20 d-6 v-20 i-65 d-6 v-20 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-6 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-18 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 p-57 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 b-1 v-20 o-12 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 d-6 v-20 b-1 s-8 o-0 t-35 i-58 d-6 v-20 o-24 t-35 i-56 v-20 o-6 t-35 i-57 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 p-43 d-6 v-20 o-24 t-35 v-20 o-24 t-35 v-20 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 p-48 d-6 v-20 i-65 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-42 t-35 i-57 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-8 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-24 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-24 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 b-1 v-20 p-60 t-35 i-0 p-67 d-6 v-20 i-56 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-65 t-35 i-40 p-77 t-35 i-0 p-65 v-20 b-1 v-20 o-24 t-35 i-57 o-0 t-35 t-35 i-0 p-67 d-6 v-20 b-1 s-9 d-6 v-20 p-48 t-35 i-0 p-67 d-6 v-20 o-18 t-35 d-6 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 t-35 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-53 d-6 v-20 b-1 s-8 o-0 t-35 i-24 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-24 p-48 d-6 v-20 o-24 t-35 i-48 t-35 i-0 p-48 d-6 v-20 o-12 t-35 v-20 p-45 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 t-35 i-0 p-59 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-36 t-35 i-0 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-48 d-6 v-20 d-6 v-20 p-60 t-35 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 v-20 b-1 s-8 v-20 b-1 v-20 b-1 s-9 o-0 t-35 i-56 d-6 v-20 b-1 s-9 o-0 t-35 i-24 p-41 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-8 o-0 t-35 v-20 p-57 t-35 i-56 p-67 d-6 v-20 i-42 p-43 d-6 v-20 p-57 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-24 p-57 d-6 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 p-60 v-20 o-24 t-35 i-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-64 d-3 v-20 p-60 I1s2_7_1 v-20 b-1 s-9 o-0 t-35 i-40 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-16 p-60 d-11 v-20 i-42 p-48 p-48 d-12 v-20 o-24 t-35 i-0 p-64 t-35 i-40 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 p-64 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-41 d-6 v-24 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-12 p-64 d-12 v-20 o-12 t-35 i-0 p-69 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-56 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-8 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 p-60 d-12 v-20 o-12 t-35 i-0 p-64 d-6 v-20 i-128 p-170 t-35 i-0 p-64 d-3 v-20 p-48 i-42 d-3 v-20 o-12 t-35 i-0 p-60 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-12 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 o-12 t-35 i-0 p-65 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 o-0 t-35 i-0 p-67 d-3 v-20 p-64 v-20 p-55 d-6 v-20 p-64 d-3 v-20 o-24 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 o-12 t-35 i-0 p-67 d-3 v-20 p-52 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-1 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-3 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-14 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-42 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 v-20 o-18 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-67 d-3 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-56 p-67 d-3 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-79 d-6 v-20 b-1 s-8 o-0 d-3 v-20 b-1 s-9 o-0 t-35 i-12 p-65 t-35 i-0 o-0 d-6 v-20 b-1 s-8 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 v-20 o-6 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-52 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 i-0 p-64 d-6 v-20 b-1 s-8 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-6 p-57 d-6 v-20 p-48 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 v-20 b-1 v-20 o-12 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 i-56 i-0 p-76 v-20 o-12 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-57 o-0 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-60 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-12 v-20 b-1 s-9 o-0 t-35 i-0 p-67 p-57 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-3 v-20 b-1 s-3 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 v-20 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 v-20 p-52 d-6 v-20 p-55 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-3 v-20 o-12 d-6 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-56 i-0 v-20 o-6 t-35 i-0 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-77 p-65 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 p-57 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 i-1 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 o-0 t-35 i-0 i-0 p-67 d-3 v-20 i-56 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 o-0 t-35 i-0 v-20 o-6 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-3 v-20 b-1 s-8 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 v-20 o-12 t-35 i-0 p-64 t-35 i-0 p-64 d-6 v-20 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 t-35 i-0 p-67 d-3 v-20 b-1 s-8 v-20 p-64 d-3 v-20 p-60 d-3 v-20 b-1 s-8 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-3 v-20 p-48 d-6 v-20 p-53 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-76 t-35 i-0 p-77 v-20 o-12 t-35 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 p-72 v-20 b-1 i-1 p-64 d-3 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-76 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-67 t-35 i-0 p-60 v-20 o-24 t-35 i-0 p-67 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-67 v-20 p-60 d-6 v-20 i-1 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 p-57 d-6 v-20 b-1 v-20 b-1 s-8 o-0 t-35 i-0 p-76 o-0 t-35 i-0 o-0 t-35 i-1 v-20 o-12 v-20 p-57 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 v-20 o-12 t-35 i-0 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-52 d-6 v-20 o-0 t-35 i-0 p-67 d-6 v-20 b-1 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 i-42 o-0 t-35 i-0 p-67 t-35 i-0 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-76 v-20 b-1 s-20 d-6 v-20 o-36 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 p-67 d-3 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-71 o-0 t-35 i-0 p-76 v-20 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-74 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-35 i-0 p-67 o-0 t-35 i-0 p-67 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-64 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 p-57 p-67 t-35 i-0 p-74 v-20 b-1 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 d-3 v-20 b-1 i-0 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 o-0 v-20 b-1 s-8 o-0 t-35 i-56 t-35 i-0 p-71 d-6 v-20 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 i-0 p-76 d-6 v-20 b-1 s-8 o-0 p-48 d-6 v-20 o-6 t-35 i-0 p-67 d-3 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 p-48 i-0 p-67 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 v-20 o-12 t-35 i-0 v-20 o-6 t-35 i-0 o-0 t-35 i-0 o-0 t-35 i-0 v-20 b-1 v-20 p-57 d-6 v-20 p-64 d-3 v-20 p-64 d-3 v-20 p-57 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 t-35 i-0 p-67 d-3 v-20 o-24 t-35 i-0 p-76 i-0 p-64 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 i-1 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 v-20 o-24 t-35 i-0 p-65 v-20 o-24 t-35 i-0 p-67 d-3 v-20 o-12 t-35 i-0 p-48 v-20 p-57 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 t-35 i-0 p-67 d-3 v-20 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 p-72 d-4 d-6 v-23 p-53 d-6 v-20 o-36 t-34 i-0 p-64 d-6 v-20 o-36 t-26 i-0 p-64 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-4 v-20 b-1 s-9 o-0 t-30 d-6 v-20 o-6 t-34 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-39 d-3 v-20 o-30 t-34 i-0 p-64 d-6 v-20 i-52 o-12 t-35 i-0 p-60 d-6 v-23 p-50 d-6 v-20 o-12 t-30 i-0 p-60 d-17 v-20 o-12 t-35 i-0 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 o-12 t-35 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-30 i-0 p-69 i-0 p-48 d-6 v-16 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-42 i-0 p-69 d-6 v-20 o-12 v-20 b-1 s-9 o-0 t-30 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-34 i-0 p-67 i-0 p-60 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 o-6 t-34 i-0 p-52 d-6 v-20 o-18 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-74 d-12 v-20 o-12 v-20 o-36 t-35 i-0 p-76 d-12 v-20 p-69 d-11 v-20 o-18 d-6 v-20 o-12 t-27 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 o-30 t-30 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-72 i-0 p-72 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-34 i-0 v-20 o-12 d-6 v-20 o-6 o-6 t-30 i-0 p-60 d-3 v-20 i-52 p-76 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 o-36 i-0 p-76 d-6 v-20 o-12 o-0 t-35 i-0 p-62 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-30 i-0 p-76 d-6 v-20 o-24 t-34 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-30 t-30 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-77 d-6 v-20 i-52 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-71 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-48 d-6 v-20 p-71 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 i-0 t-35 i-0 p-74 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-39 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 i-0 p-79 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 i-0 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-71 d-6 v-20 b-1 d-6 v-20 o-18 t-30 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 i-0 p-69 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 o-42 t-35 i-0 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-11 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-52 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-34 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 p-57 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 v-20 b-1 s-9 o-0 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-12 i-0 p-71 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-52 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 o-18 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-71 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-74 d-6 v-20 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-77 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-59 d-6 v-20 o-30 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-76 v-20 o-12 t-30 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-64 d-11 v-20 o-12 t-35 i-0 p-59 d-6 v-20 o-6 t-39 o-18 t-35"
# model_result_str = "s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-9 v-20 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-6 v-20 p-52 d-6 v-20 p-48 d-3 v-24 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 p-48 d-3 v-24 p-33 d-6 v-20 o-33 t-33 i-57 p-48 d-3 v-24 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-12 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-12 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-50 d-9 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-24 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-50 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 o-30 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 o-33 t-33 i-57 p-52 d-6 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-48 d-6 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-43 d-3 v-20 o-42 t-33 i-57 p-48 d-3 v-20 p-48 d-3 v-20 p-45 d-3 v-20 o-45 t-33 i-57 p-50 d-13 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-43 d-3 v-20 p-43 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-12 t-33 i-57 p-53 d-9 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-15 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-18 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 o-21 t-33 i-57 p-53 d-3 v-24 o-24 t-33 i-57 p-53 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-24 p-43 d-3 v-20 p-43 d-3 v-20 o-39 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 p-42 d-3 v-20 o-42 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-18 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-30 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 o-42 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-15 v-24 p-48 d-15 v-20 p-48 d-15 v-20 p-38 d-6 v-20 o-12 t-33 i-57 p-43 d-9 v-20 o-21 t-33 i-57 p-52 d-6 v-24 p-52 d-6 v-20 p-52 d-6 v-20 p-36 d-9 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-48 d-3 v-20 p-48 d-3 v-20 o-30 t-33 i-57 p-50 d-6 v-24 p-50 d-6 v-20 p-50 d-6 v-20 p-33 d-6 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-48 d-12 v-20 p-48 d-12 v-20 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-55 d-12 v-24 p-52 d-12 v-20 p-52 d-12 v-20 p-36 d-12 v-20 o-12 t-33 i-57 p-52 d-9 v-20 p-52 d-9 v-20 p-52 d-3 v-24 o-15 t-33 i-57 p-50 d-6 v-24 o-21 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-48 d-6 v-24 o-27 t-33 i-57 p-50 d-6 v-24 o-30 t-33 i-57 p-53 d-6 v-20 p-53 d-6 v-20 o-33 t-33 i-57 p-52 d-12 v-24 p-36 d-3 v-20 o-36 t-33 i-57 p-53 d-12 v-20 p-53 d-12 v-20 p-40 d-3 v-20 o-39 t-33 i-57 p-43 d-3 v-20 o-42 t-33 i-57 p-45 d-3 v-20 o-45 t-33 i-57 p-48 d-3 v-24 p-41 d-13 v-20 b-1 s-9 o-0 t-33 i-57 p-50 d-12 v-20 p-50 d-12 v-20 p-50 d-6 v-24 o-6 t-33 i-57 p-48 d-3 v-24 o-9 t-33 i-57 p-45 d-13 v-24 o-12 t-33 i-57 p-53 d-9 v-20 p-53 d-9 v-20 o-21 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-24 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 o-27 t-33 i-57 p-53 d-3 v-20 p-53 d-3 v-20 p-38 d-3 v-20 o-30 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-40 d-3 v-20 o-33 t-33 i-57 p-52 d-3 v-20 p-52 d-3 v-20 p-41 d-6 v-20 o-36 t-33 i-57 p-50 d-6 v-20 p-50 d-6 v-20 o-39 t-33 i-57 p-42 d-3 v-20 o-42 t-33 i-57 p-48 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 p-43 d-3 v-20 o-45 t-33 i-57 p-44 d-3 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-45 d-12 v-20 p-41 d-3 v-20 p-41 d-3 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-47 d-3 v-20 p-47 d-3 v-20 o-9 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-52 d-12 v-24 p-40 d-12 v-20 o-15 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-21 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-24 t-33 i-57 p-52 d-12 v-24 p-41 d-9 v-20 o-27 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-33 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 p-36 d-13 v-20 o-36 t-33 i-57 p-52 d-12 v-24 p-48 d-6 v-20 p-48 d-6 v-20 o-42 t-33 i-57 p-45 d-6 v-20 p-45 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-41 d-3 v-20 p-41 d-3 v-20 p-38 d-6 v-20 o-3 t-33 i-57 p-45 d-3 v-20 p-45 d-3 v-20 o-6 t-33 i-57 p-47 d-3 v-20 p-47 d-3 v-20 o-9 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-9 v-20 o-15 t-33 i-57 p-50 d-6 v-24 p-48 d-6 v-20 p-48 d-6 v-20 o-21 t-33 i-57 p-48 d-3 v-24 p-45 d-3 v-20 p-45 d-3 v-20 p-36 d-9 v-20 o-24 t-33 i-57 p-48 d-6 v-20 p-48 d-6 v-20 o-27 t-33 i-57 p-48 d-3 v-24 o-30 t-33 i-57 p-48 d-3 v-24 p-33 d-6 v-20 o-33 t-33 i-57 p-48 d-3 v-24 o-36 t-33 i-57 p-48 d-12 v-24 p-36 d-3 v-20 o-39 t-33 i-57 p-33 d-3 v-20 o-42 t-33 i-57 p-36 d-6 v-20 b-1 s-9 o-0 t-33 i-57 p-52 d-12 v-24 p-48 d-12 v-20 p-40 d-12 v-20 o-6 t-33 i-57 p-48 d-6 v-20 o-12 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-15 t-33 i-57 p-50 d-6 v-24 p-45 d-6 v-20 p-45 d-6 v-20 p-38 d-6 v-20 o-21 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-27 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-30 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-33 t-33 i-57 p-48 d-3 v-24 p-43 d-3 v-20 p-43 d-3 v-20 p-36 d-3 v-20 o-36 t-33 i-57 p-48 d-12 v-24 p-43 d-12 v-20 p-43 d-12 v-20 p-36 d-12 v-20 b-1"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 p-72 d-4 d-6 v-23 p-53 d-6 v-20 o-36 t-34 i-0 p-64 d-6 v-20 o-36 t-26 i-0 p-64 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-4 v-20 b-1 s-9 o-0 t-30 d-6 v-20 o-6 t-34 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-39 d-3 v-20 o-30 t-34 i-0 p-64 d-6 v-20 i-52 o-12 t-35 i-0 p-60 d-6 v-23 p-50 d-6 v-20 o-12 t-30 i-0 p-60 d-17 v-20 o-12 t-35 i-0 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 o-12 t-35 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-30 i-0 p-69 i-0 p-48 d-6 v-16 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-42 i-0 p-69 d-6 v-20 o-12 v-20 b-1 s-9 o-0 t-30 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-34 i-0 p-67 i-0 p-60 d-6 v-20 o-12 t-30 i-0 p-74 d-6 v-20 o-6 t-34 i-0 p-52 d-6 v-20 o-18 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-74 d-12 v-20 o-12 v-20 o-36 t-35 i-0 p-76 d-12 v-20 p-69 d-11 v-20 o-18 d-6 v-20 o-12 t-27 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 o-30 t-30 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-72 i-0 p-72 d-6 v-20 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-34 i-0 v-20 o-12 d-6 v-20 o-6 o-6 t-30 i-0 p-60 d-3 v-20 i-52 p-76 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 o-36 i-0 p-76 d-6 v-20 o-12 o-0 t-35 i-0 p-62 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-30 i-0 p-76 d-6 v-20 o-24 t-34 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 o-30 t-30 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-77 d-6 v-20 i-52 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-71 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-48 d-6 v-20 p-71 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 i-0 t-35 i-0 p-74 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-76 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-39 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 i-0 p-79 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 i-0 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-71 d-6 v-20 b-1 d-6 v-20 o-18 t-30 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 i-0 p-69 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 o-42 t-35 i-0 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-11 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-52 p-62 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-6 t-34 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 p-57 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 v-20 b-1 s-9 o-0 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-12 i-0 p-71 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-52 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 p-48 d-6 v-20 o-18 t-35 i-0 p-74 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-71 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-74 d-6 v-20 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-77 d-6 v-20 b-1 s-9 o-0 t-35 i-0 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-59 d-6 v-20 o-30 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-18 t-35 i-0 p-76 v-20 o-12 t-30 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-64 d-11 v-20 o-12 t-35 i-0 p-59 d-6 v-20 o-6 t-39 o-18 t-35"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 d-11 v-20 b-1 s-9 o-0 t-34 i-0 p-69 d-11 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-69 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-11 v-20 o-24 t-34 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-34 i-0 p-72 v-20 o-24 t-34 i-0 p-69 d-11 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-34 t-42 i-0 d-6 v-20 b-1 s-9 o-0 t-34 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-24 t-34 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-44 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-69 v-20 b-1 s-8 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-25 d-11 v-20 b-1 s-9 o-0 t-34 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 v-20 b-1 s-8 o-0 v-20 b-1 s-9 o-0 t-34 i-0 p-76 d-4 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-11 v-20 b-1 s-9 o-0 t-30 v-20 b-1 s-8 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-38 i-0 p-67 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-40 v-20 b-1 s-9 o-0 t-35 i-0 p-74 v-20 b-1 s-9 o-0 t-30 i-0 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-72 v-20 b-1 s-9 o-0 t-34 i-0 p-74 v-20 b-1 s-9 o-0 t-26 i-0 i-1 b-1 s-9 o-0 t-27 i-0 p-76 d-11 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-19 o-0 t-35 i-0 t-35 i-0 p-60 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-30 i-52 v-20 b-1 s-9 o-0 t-34 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-30 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-11 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-35 i-1 v-20 b-1 s-9 o-0 t-34 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-11 v-20 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-8 v-20 b-1 s-9 o-0 t-30 i-0 p-76 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-76 v-20 o-12 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-76 d-11 v-20 b-1 s-8 o-0 t-34 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-12 v-20 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-26 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-8 o-0 t-34 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-30 v-20 b-1 s-8 o-0 t-34 i-0 p-74 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-72 t-25 i-0 p-76 d-6 v-20 b-1 s-9 o-0 v-20 b-1 s-8 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 v-20 b-1 s-9 o-0 t-34 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-30 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-34 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-72 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-74 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-52 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-8 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-34 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-8 o-0 t-34 i-0 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-34 i-0 p-72 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-30"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1"
# model_result_str = "s-9 o-0 t-36 i-27 p-71 d-6 v-20 o-6 t-36 i-27 p-71 d-6 v-20 o-12 t-36 i-27 p-71 d-6 v-20 o-18 t-36 i-27 p-71 d-6 v-20 o-24 t-36 i-27 p-71 d-6 v-20 o-30 t-36 i-27 p-71 d-6 v-20 o-36 t-36 i-27 p-73 d-6 v-20 o-42 t-36 i-27 p-73 d-6 v-20 o-48 t-36 i-27 p-68 d-6 v-20 o-54 t-36 i-27 p-68 d-6 v-20 o-60 t-36 i-27 p-68 d-6 v-20 o-66 t-36 i-27 p-68 d-6 v-20 o-72 t-36 i-27 p-68 d-6 v-20 o-78 t-36 i-27 p-68 d-6 v-20 o-84 t-36 i-27 p-68 d-6 v-20 o-90 t-36 i-27 p-68 d-6 v-20 o-96 t-36 i-27 p-66 d-6 v-20 o-102 t-36 i-27 p-66 d-6 v-20 o-108 t-36 i-27 p-66 d-6 v-20 o-114 t-36 i-27 p-66 d-6 v-20 o-120 t-36 i-27 p-66 d-6 v-20 o-126 t-36 i-27 p-66 d-6 v-20 o-132 t-36 i-27 p-68 d-6 v-20 o-138 t-36 i-27 p-68 d-6 v-20 o-144 t-36 i-27 p-64 d-6 v-20 o-150 t-36 i-27 p-64 d-6 v-20 o-156 t-36 i-27 p-64 d-6 v-20 o-162 t-36 i-27 p-64 d-6 v-20 o-168 t-36 i-27 p-61 d-6 v-20 o-174 t-36 i-27 p-61 d-6 v-20 o-180 t-36 i-27 p-61 d-6 v-20 o-186 t-36 i-27 p-61 d-6 v-20 o-192 t-36 i-27 p-71 d-6 v-20 o-198 t-36 i-27 p-71 d-6 v-20 o-204 t-36 i-27 p-71 d-6 v-20 o-210 t-36 i-27 p-71 d-6 v-20 o-216 t-36 i-27 p-71 d-6 v-20 o-222 t-36 i-27 p-71 d-6 v-20 o-228 t-36 i-27 p-73 d-6 v-20 o-234 t-36 i-27 p-73 d-6 v-20 o-240 t-36 i-27 p-68 d-6 v-20 o-246 t-36 i-27 p-68 d-6 v-20 o-252 t-36 i-27 p-68 d-6 v-20 o-258 t-36 i-27 p-68 d-6 v-20 o-264 t-36 i-27 p-68 d-6 v-20 o-270 t-36 i-27 p-68 d-6 v-20 o-276 t-36 i-27 p-68 d-6 v-20 o-282 t-36 i-27 p-68 d-6 v-20 o-288 t-36 i-27 p-66 d-6 v-20 o-294 t-36 i-27 p-66 d-6 v-20 o-300 t-36 i-27 p-66 d-6 v-20 o-306 t-36 i-27 p-66 d-6 v-20 o-312 t-36 i-27 p-66 d-6 v-20 o-318 t-36 i-27 p-66 d-6 v-20 o-324 t-36 i-27 p-68 d-6 v-20 o-330 t-36 i-27 p-68 d-6 v-20 o-336 t-36 i-27 p-64 d-6 v-20 o-342 t-36 i-27 p-64 d-6 v-20 o-348 t-36 i-27 p-64 d-6 v-20 o-354 t-36 i-27 p-64 d-6 v-20 o-360 t-36 i-27 p-61 d-6 v-20 o-366 t-36 i-27 p-61 d-6 v-20 o-372 t-36 i-27 p-61 d-6 v-20 o-378 t-36 i-27 p-61 d-6 v-20 o-0 t-36 i-8 p-59 d-27 v-17 o-0 t-36 i-8 p-62 d-27 v-17 o-0 t-36 i-8 p-66 d-27 v-17 o-0 t-36 i-8 p-35 d-27 v-17 o-48 t-36 i-8 p-57 d-27 v-17 o-48 t-36 i-8 p-61 d-27 v-17 o-48 t-36 i-8 p-64 d-27 v-17 o-48 t-36 i-8 p-33 d-27 v-17 o-96 t-36 i-8 p-66 d-27 v-17 o-96 t-36 i-8 p-69 d-27 v-17 o-96 t-36 i-8 p-73 d-27 v-17 o-96 t-36 i-8 p-42 d-27 v-17 o-144 t-36 i-8 p-64 d-18 v-17 o-144 t-36 i-8 p-68 d-18 v-17 o-144 t-36 i-8 p-71 d-18 v-17 o-144 t-36 i-8 p-40 d-18 v-17 o-168 t-36 i-8 p-61 d-18 v-17 o-168 t-36 i-8 p-64 d-18 v-17 o-168 t-36 i-8 p-68 d-18 v-17 o-168 t-36 i-8 p-37 d-18 v-17 o-192 t-36 i-8 p-59 d-27 v-17 o-192 t-36 i-8 p-62 d-27 v-17 o-192 t-36 i-8 p-66 d-27 v-17 o-192 t-36 i-8 p-35 d-27 v-17 o-240 t-36 i-8 p-57 d-27 v-17 o-240 t-36 i-8 p-61 d-27 v-17 o-240 t-36 i-8 p-64 d-27 v-17 o-240 t-36 i-8 p-33 d-27 v-17 o-288 t-36 i-8 p-66 d-27 v-17 o-288 t-36 i-8 p-69 d-27 v-17 o-288 t-36 i-8 p-73 d-27 v-17 o-288 t-36 i-8 p-42 d-27 v-17 o-336 t-36 i-8 p-64 d-18 v-17 o-336 t-36 i-8 p-68 d-18 v-17 o-336 t-36 i-8 p-71 d-18 v-17 o-336 t-36 i-8 p-40 d-18 v-17 o-360 t-36 i-8 p-61 d-18 v-17 o-360 t-36 i-8 p-64 d-18 v-17 o-360 t-36 i-8 p-68 d-18 v-17 o-360 t-36 i-8 p-37 d-18 v-17"
# model_result_str = "s-9 o-0 t-31 i-24 p-61 d-27 v-20 o-48 t-31 i-24 p-60 d-12 v-20 o-72 t-31 i-24 p-58 d-9 v-20 o-81 t-31 i-24 p-58 d-9 v-20 o-90 t-31 i-24 p-58 d-12 v-20 o-102 t-31 i-24 p-58 d-12 v-20 o-189 t-31 i-24 p-51 d-3 v-20 o-192 t-31 i-24 p-61 d-27 v-20 o-240 t-31 i-24 p-60 d-12 v-20 o-264 t-31 i-24 p-58 d-4 v-20 o-268 t-31 i-24 p-58 d-4 v-20 o-272 t-31 i-24 p-58 d-8 v-20 o-280 t-31 i-24 p-58 d-8 v-20 o-288 t-31 i-24 p-58 d-3 v-20 o-291 t-31 i-24 p-56 d-3 v-20 o-294 t-31 i-24 p-55 d-12 v-20 o-381 t-31 i-24 p-51 d-3 v-20 o-384 t-31 i-24 p-61 d-27 v-20 o-432 t-31 i-24 p-60 d-18 v-20 o-456 t-31 i-24 p-58 d-9 v-20 o-465 t-31 i-24 p-58 d-9 v-20 o-474 t-31 i-24 p-58 d-12 v-20 o-486 t-31 i-24 p-58 d-15 v-20 o-573 t-31 i-24 p-51 d-3 v-20 o-576 t-31 i-24 p-61 d-27 v-20 o-624 t-31 i-24 p-60 d-18 v-20 o-648 t-31 i-24 p-58 d-4 v-20 o-652 t-31 i-24 p-58 d-4 v-20 o-656 t-31 i-24 p-58 d-8 v-20 o-664 t-31 i-24 p-63 d-13 v-20 o-678 t-31 i-24 p-63 d-15 v-20 o-0 t-31 i-16 p-73 d-27 v-17 o-0 t-31 i-16 p-77 d-27 v-17 o-0 t-31 i-16 p-68 d-27 v-17 o-0 t-31 i-16 p-75 d-27 v-17 o-0 t-31 i-16 p-44 d-27 v-17 o-48 t-31 i-16 p-56 d-27 v-17 o-48 t-31 i-16 p-60 d-27 v-17 o-48 t-31 i-16 p-63 d-27 v-17 o-48 t-31 i-16 p-32 d-27 v-17 o-96 t-31 i-16 p-51 d-33 v-17 o-96 t-31 i-16 p-55 d-33 v-17 o-96 t-31 i-16 p-58 d-33 v-17 o-96 t-31 i-16 p-27 d-33 v-17 o-168 t-31 i-16 p-63 d-18 v-17 o-168 t-31 i-16 p-67 d-18 v-17 o-168 t-31 i-16 p-58 d-18 v-17 o-168 t-31 i-16 p-34 d-18 v-17 o-192 t-31 i-16 p-73 d-27 v-17 o-192 t-31 i-16 p-77 d-27 v-17 o-192 t-31 i-16 p-68 d-27 v-17 o-192 t-31 i-16 p-75 d-27 v-17 o-192 t-31 i-16 p-44 d-27 v-17 o-240 t-31 i-16 p-68 d-27 v-17 o-240 t-31 i-16 p-60 d-27 v-17 o-240 t-31 i-16 p-63 d-27 v-17 o-240 t-31 i-16 p-67 d-27 v-17 o-240 t-31 i-16 p-36 d-27 v-17 o-288 t-31 i-16 p-51 d-33 v-17 o-288 t-31 i-16 p-55 d-33 v-17 o-288 t-31 i-16 p-58 d-33 v-17 o-288 t-31 i-16 p-27 d-33 v-17 o-360 t-31 i-16 p-63 d-18 v-17 o-360 t-31 i-16 p-67 d-18 v-17 o-360 t-31 i-16 p-58 d-18 v-17 o-360 t-31 i-16 p-34 d-18 v-17 o-384 t-31 i-16 p-73 d-27 v-17 o-384 t-31 i-16 p-77 d-27 v-17 o-384 t-31 i-16 p-68 d-27 v-17 o-384 t-31 i-16 p-75 d-27 v-17 o-384 t-31 i-16 p-44 d-27 v-17 o-432 t-31 i-16 p-56 d-27 v-17 o-432 t-31 i-16 p-60 d-27 v-17 o-432 t-31 i-16 p-63 d-27 v-17 o-432 t-31 i-16 p-32 d-27 v-17 o-480 t-31 i-16 p-51 d-33 v-17 o-480 t-31 i-16 p-55 d-33 v-17 o-480 t-31 i-16 p-58 d-33 v-17 o-480 t-31 i-16 p-27 d-33 v-17 o-552 t-31 i-16 p-63 d-18 v-17 o-552 t-31 i-16 p-67 d-18 v-17 o-552 t-31 i-16 p-58 d-18 v-17 o-552 t-31 i-16 p-34 d-18 v-17 o-576 t-31 i-16 p-73 d-27 v-17 o-576 t-31 i-16 p-77 d-27 v-17 o-576 t-31 i-16 p-68 d-27 v-17 o-576 t-31 i-16 p-75 d-27 v-17 o-576 t-31 i-16 p-44 d-27 v-17 o-624 t-31 i-16 p-68 d-27 v-17 o-624 t-31 i-16 p-60 d-27 v-17 o-624 t-31 i-16 p-63 d-27 v-17 o-624 t-31 i-16 p-67 d-27 v-17 o-624 t-31 i-16 p-36 d-27 v-17 o-672 t-31 i-16 p-51 d-27 v-17 o-672 t-31 i-16 p-55 d-27 v-17 o-672 t-31 i-16 p-58 d-27 v-17 o-672 t-31 i-16 p-27 d-27 v-17 o-720 t-31 i-16 p-58 d-27 v-17 o-720 t-31 i-16 p-62 d-27 v-17 o-720 t-31 i-16 p-63 d-27 v-17 o-720 t-31 i-16 p-65 d-27 v-17 o-720 t-31 i-16 p-34 d-27 v-17"
# model_result_str = "s-9 o-0 t-30 i-40 p-77 d-6 v-20 o-6 t-30 i-40 p-76 d-6 v-20 o-12 t-30 i-40 p-77 d-6 v-20 o-18 t-30 i-40 p-76 d-6 v-20 o-24 t-30 i-40 p-77 d-6 v-20 o-30 t-30 i-40 p-73 d-6 v-20 o-36 t-30 i-40 p-70 d-6 v-20 o-42 t-30 i-40 p-68 d-6 v-20 o-48 t-30 i-40 p-76 d-6 v-20 o-54 t-30 i-40 p-76 d-6 v-20 o-60 t-30 i-40 p-78 d-6 v-20 o-66 t-30 i-40 p-71 d-6 v-20 o-78 t-30 i-40 p-71 d-6 v-20 o-84 t-30 i-40 p-71 d-3 v-20 o-87 t-30 i-40 p-71 d-3 v-20 o-90 t-30 i-40 p-68 d-3 v-20 o-93 t-30 i-40 p-75 d-9 v-20 o-102 t-30 i-40 p-78 d-6 v-20 o-108 t-30 i-40 p-75 d-6 v-20 o-114 t-30 i-40 p-71 d-6 v-20 o-126 t-30 i-40 p-71 d-6 v-20 o-132 t-30 i-40 p-71 d-6 v-20 o-138 t-30 i-40 p-66 d-3 v-20 o-141 t-30 i-40 p-75 d-6 v-20 o-147 t-30 i-40 p-73 d-9 v-20 o-156 t-30 i-40 p-66 d-6 v-20 o-162 t-30 i-40 p-64 d-3 v-20 o-165 t-30 i-40 p-64 d-3 v-20 o-171 t-30 i-40 p-65 d-3 v-20 o-174 t-30 i-40 p-65 d-3 v-20 o-180 t-30 i-40 p-66 d-6 v-20 o-186 t-30 i-40 p-67 d-6 v-20 o-192 t-30 i-40 p-78 d-6 v-20 o-198 t-30 i-40 p-78 d-3 v-20 o-201 t-30 i-40 p-78 d-6 v-20 o-213 t-30 i-40 p-75 d-3 v-20 o-216 t-30 i-40 p-81 d-3 v-20 o-219 t-30 i-40 p-81 d-6 v-20 o-225 t-30 i-40 p-81 d-6 v-20 o-231 t-30 i-40 p-80 d-3 v-20 o-234 t-30 i-40 p-78 d-6 v-20 o-240 t-30 i-40 p-78 d-6 v-20 o-246 t-30 i-40 p-78 d-3 v-20 o-249 t-30 i-40 p-78 d-6 v-20 o-261 t-30 i-40 p-73 d-3 v-20 o-264 t-30 i-40 p-81 d-3 v-20 o-267 t-30 i-40 p-81 d-6 v-20 o-273 t-30 i-40 p-83 d-6 v-20 o-279 t-30 i-40 p-81 d-3 v-20 o-282 t-30 i-40 p-78 d-6 v-20 o-288 t-30 i-40 p-78 d-6 v-20 o-294 t-30 i-40 p-82 d-6 v-20 o-300 t-30 i-40 p-85 d-6 v-20 o-306 t-30 i-40 p-87 d-12 v-20 o-318 t-30 i-40 p-82 d-6 v-20 o-324 t-30 i-40 p-82 d-6 v-20 o-330 t-30 i-40 p-82 d-3 v-20 o-333 t-30 i-40 p-80 d-3 v-20 o-336 t-30 i-40 p-80 d-12 v-20 o-0 t-30 i-8 p-61 d-27 v-17 o-0 t-30 i-8 p-65 d-27 v-17 o-0 t-30 i-8 p-68 d-27 v-17 o-0 t-30 i-8 p-37 d-27 v-17 o-48 t-30 i-8 p-64 d-27 v-17 o-48 t-30 i-8 p-68 d-27 v-17 o-48 t-30 i-8 p-71 d-27 v-17 o-48 t-30 i-8 p-40 d-27 v-17 o-96 t-30 i-8 p-59 d-27 v-17 o-96 t-30 i-8 p-63 d-27 v-17 o-96 t-30 i-8 p-66 d-27 v-17 o-96 t-30 i-8 p-35 d-27 v-17 o-144 t-30 i-8 p-69 d-27 v-17 o-144 t-30 i-8 p-73 d-27 v-17 o-144 t-30 i-8 p-76 d-27 v-17 o-144 t-30 i-8 p-79 d-27 v-17 o-144 t-30 i-8 p-45 d-27 v-17 o-192 t-30 i-8 p-59 d-18 v-17 o-192 t-30 i-8 p-63 d-18 v-17 o-192 t-30 i-8 p-66 d-18 v-17 o-192 t-30 i-8 p-35 d-18 v-17 o-216 t-30 i-8 p-62 d-18 v-17 o-216 t-30 i-8 p-66 d-18 v-17 o-216 t-30 i-8 p-69 d-18 v-17 o-216 t-30 i-8 p-38 d-18 v-17 o-240 t-30 i-8 p-66 d-18 v-17 o-240 t-30 i-8 p-70 d-18 v-17 o-240 t-30 i-8 p-73 d-18 v-17 o-240 t-30 i-8 p-42 d-18 v-17 o-264 t-30 i-8 p-62 d-18 v-17 o-264 t-30 i-8 p-66 d-18 v-17 o-264 t-30 i-8 p-69 d-18 v-17 o-264 t-30 i-8 p-38 d-18 v-17 o-288 t-30 i-8 p-66 d-15 v-17 o-288 t-30 i-8 p-70 d-15 v-17 o-288 t-30 i-8 p-73 d-15 v-17 o-288 t-30 i-8 p-42 d-15 v-17 o-306 t-30 i-8 p-63 d-21 v-17 o-306 t-30 i-8 p-66 d-21 v-17 o-306 t-30 i-8 p-70 d-21 v-17 o-306 t-30 i-8 p-39 d-21 v-17 o-336 t-30 i-8 p-61 d-37 v-17 o-336 t-30 i-8 p-65 d-37 v-17 o-336 t-30 i-8 p-68 d-37 v-17 o-336 t-30 i-8 p-71 d-37 v-17 o-336 t-30 i-8 p-37 d-37 v-17 o-432 t-30 i-8 p-66 d-37 v-17 o-432 t-30 i-8 p-70 d-37 v-17 o-432 t-30 i-8 p-73 d-37 v-17 o-432 t-30 i-8 p-76 d-37 v-17 o-432 t-30 i-8 p-42 d-37 v-17"
# model_result_str = "s-9 o-6 t-33 i-29 p-55 d-6 v-20 o-12 t-33 i-29 p-60 d-12 v-20 o-24 t-33 i-29 p-58 d-6 v-20 o-30 t-33 i-29 p-60 d-6 v-20 o-36 t-33 i-29 p-60 d-6 v-20 o-42 t-33 i-29 p-61 d-12 v-20 o-54 t-33 i-29 p-61 d-6 v-20 o-60 t-33 i-29 p-60 d-6 v-20 o-66 t-33 i-29 p-58 d-6 v-20 o-72 t-33 i-29 p-58 d-6 v-20 o-78 t-33 i-29 p-60 d-6 v-20 o-84 t-33 i-29 p-60 d-6 v-20 o-90 t-33 i-29 p-60 d-6 v-20 o-108 t-33 i-29 p-60 d-6 v-20 o-114 t-33 i-29 p-58 d-6 v-20 o-120 t-33 i-29 p-58 d-6 v-20 o-126 t-33 i-29 p-60 d-12 v-20 o-138 t-33 i-29 p-61 d-12 v-20 o-150 t-33 i-29 p-63 d-6 v-20 o-156 t-33 i-29 p-60 d-6 v-20 o-162 t-33 i-29 p-61 d-6 v-20 o-168 t-33 i-29 p-58 d-6 v-20 o-174 t-33 i-29 p-60 d-6 v-20 o-180 t-33 i-29 p-61 d-9 v-20 o-189 t-33 i-29 p-60 d-3 v-20 o-192 t-33 i-29 p-60 d-12 v-20 o-204 t-33 i-29 p-60 d-6 v-20 o-210 t-33 i-29 p-60 d-6 v-20 o-216 t-33 i-29 p-58 d-6 v-20 o-222 t-33 i-29 p-60 d-12 v-20 o-234 t-33 i-29 p-60 d-6 v-20 o-240 t-33 i-29 p-61 d-6 v-20 o-246 t-33 i-29 p-61 d-6 v-20 o-252 t-33 i-29 p-60 d-6 v-20 o-258 t-33 i-29 p-60 d-6 v-20 o-264 t-33 i-29 p-58 d-6 v-20 o-270 t-33 i-29 p-60 d-6 v-20 o-276 t-33 i-29 p-60 d-12 v-20 o-300 t-33 i-29 p-60 d-6 v-20 o-306 t-33 i-29 p-58 d-6 v-20 o-312 t-33 i-29 p-58 d-6 v-20 o-318 t-33 i-29 p-60 d-12 v-20 o-330 t-33 i-29 p-61 d-12 v-20 o-342 t-33 i-29 p-63 d-6 v-20 o-348 t-33 i-29 p-65 d-6 v-20 o-354 t-33 i-29 p-63 d-6 v-20 o-360 t-33 i-29 p-63 d-6 v-20 o-366 t-33 i-29 p-63 d-6 v-20 o-372 t-33 i-29 p-65 d-6 v-20 o-378 t-33 i-29 p-63 d-6 v-20 o-384 t-33 i-29 p-67 d-15 v-20 o-402 t-33 i-29 p-68 d-27 v-20 o-450 t-33 i-29 p-67 d-12 v-20 o-462 t-33 i-29 p-68 d-6 v-20 o-468 t-33 i-29 p-70 d-6 v-20 o-474 t-33 i-29 p-68 d-6 v-20 o-480 t-33 i-29 p-67 d-15 v-20 o-498 t-33 i-29 p-68 d-28 v-20 o-564 t-33 i-29 p-63 d-6 v-20 o-570 t-33 i-29 p-63 d-6 v-20 o-576 t-33 i-29 p-67 d-15 v-20 o-594 t-33 i-29 p-68 d-27 v-20 o-642 t-33 i-29 p-68 d-6 v-20 o-648 t-33 i-29 p-67 d-6 v-20 o-654 t-33 i-29 p-68 d-6 v-20 o-660 t-33 i-29 p-70 d-6 v-20 o-666 t-33 i-29 p-68 d-6 v-20 o-672 t-33 i-29 p-67 d-15 v-20 o-690 t-33 i-29 p-68 d-12 v-20 o-702 t-33 i-29 p-63 d-6 v-20 o-708 t-33 i-29 p-63 d-6 v-20 o-714 t-33 i-29 p-61 d-6 v-20 o-720 t-33 i-29 p-61 d-6 v-20 o-726 t-33 i-29 p-60 d-6 v-20 o-732 t-33 i-29 p-61 d-6 v-20 o-738 t-33 i-29 p-68 d-12 v-20 o-750 t-33 i-29 p-61 d-6 v-20 o-756 t-33 i-29 p-61 d-6 v-20 o-762 t-33 i-29 p-60 d-6 v-20 o-768 t-33 i-29 p-61 d-6 v-20 o-774 t-33 i-29 p-68 d-12 v-20 o-786 t-33 i-29 p-61 d-6 v-20 o-792 t-33 i-29 p-61 d-6 v-20 o-798 t-33 i-29 p-60 d-6 v-20 o-804 t-33 i-29 p-56 d-6 v-20 o-810 t-33 i-29 p-58 d-12 v-20 o-822 t-33 i-29 p-60 d-12 v-20 o-834 t-33 i-29 p-60 d-21 v-20 o-0 t-33 i-8 p-56 d-60 v-17 o-0 t-33 i-8 p-60 d-60 v-17 o-0 t-33 i-8 p-63 d-60 v-17 o-0 t-33 i-8 p-32 d-60 v-17 o-384 t-33 i-8 p-61 d-37 v-17 o-384 t-33 i-8 p-65 d-37 v-17 o-384 t-33 i-8 p-68 d-37 v-17 o-384 t-33 i-8 p-75 d-37 v-17 o-384 t-33 i-8 p-37 d-37 v-17 o-480 t-33 i-8 p-56 d-37 v-17 o-480 t-33 i-8 p-60 d-37 v-17 o-480 t-33 i-8 p-63 d-37 v-17 o-480 t-33 i-8 p-32 d-37 v-17 o-576 t-33 i-8 p-61 d-37 v-17 o-576 t-33 i-8 p-65 d-37 v-17 o-576 t-33 i-8 p-68 d-37 v-17 o-576 t-33 i-8 p-75 d-37 v-17 o-576 t-33 i-8 p-37 d-37 v-17 o-672 t-33 i-8 p-56 d-27 v-17 o-672 t-33 i-8 p-60 d-27 v-17 o-672 t-33 i-8 p-63 d-27 v-17 o-672 t-33 i-8 p-32 d-27 v-17 o-720 t-33 i-8 p-61 d-37 v-17 o-720 t-33 i-8 p-65 d-37 v-17 o-720 t-33 i-8 p-68 d-37 v-17 o-720 t-33 i-8 p-75 d-37 v-17 o-720 t-33 i-8 p-37 d-37 v-17 o-816 t-33 i-8 p-56 d-37 v-17 o-816 t-33 i-8 p-60 d-37 v-17 o-816 t-33 i-8 p-63 d-37 v-17 o-816 t-33 i-8 p-32 d-37 v-17"
# model_result_str = "s-9 o-0 t-36 i-40 p-62 d-6 v-20 o-6 t-36 i-40 p-64 d-6 v-20 o-18 t-36 i-40 p-64 d-15 v-20 o-36 t-36 i-40 p-66 d-12 v-20 o-48 t-36 i-40 p-66 d-6 v-20 o-54 t-36 i-40 p-67 d-15 v-20 o-72 t-36 i-40 p-67 d-12 v-20 o-84 t-36 i-40 p-69 d-6 v-20 o-90 t-36 i-40 p-71 d-12 v-20 o-108 t-36 i-40 p-62 d-12 v-20 o-192 t-36 i-40 p-64 d-6 v-20 o-198 t-36 i-40 p-66 d-12 v-20 o-210 t-36 i-40 p-66 d-15 v-20 o-228 t-36 i-40 p-67 d-12 v-20 o-240 t-36 i-40 p-67 d-6 v-20 o-246 t-36 i-40 p-69 d-12 v-20 o-258 t-36 i-40 p-69 d-15 v-20 o-276 t-36 i-40 p-71 d-12 v-20 o-288 t-36 i-40 p-69 d-12 v-20 o-300 t-36 i-40 p-69 d-12 v-20 o-384 t-36 i-40 p-67 d-6 v-20 o-390 t-36 i-40 p-69 d-12 v-20 o-402 t-36 i-40 p-69 d-15 v-20 o-420 t-36 i-40 p-71 d-12 v-20 o-432 t-36 i-40 p-71 d-12 v-20 o-444 t-36 i-40 p-72 d-6 v-20 o-450 t-36 i-40 p-72 d-12 v-20 o-474 t-36 i-40 p-67 d-6 v-20 o-480 t-36 i-40 p-67 d-6 v-20 o-486 t-36 i-40 p-69 d-12 v-20 o-498 t-36 i-40 p-69 d-15 v-20 o-516 t-36 i-40 p-70 d-12 v-20 o-528 t-36 i-40 p-70 d-12 v-20 o-540 t-36 i-40 p-72 d-12 v-20 o-552 t-36 i-40 p-72 d-12 v-20 o-564 t-36 i-40 p-71 d-6 v-20 o-570 t-36 i-40 p-72 d-6 v-20 o-576 t-36 i-40 p-71 d-18 v-20 o-600 t-36 i-40 p-74 d-18 v-20 o-624 t-36 i-40 p-69 d-18 v-20 o-648 t-36 i-40 p-74 d-18 v-20 o-672 t-36 i-40 p-67 d-18 v-20 o-744 t-36 i-40 p-74 d-12 v-20 o-756 t-36 i-40 p-72 d-12 v-20 o-0 t-36 i-16 p-55 d-27 v-17 o-0 t-36 i-16 p-59 d-27 v-17 o-0 t-36 i-16 p-62 d-27 v-17 o-0 t-36 i-16 p-31 d-27 v-17 o-48 t-36 i-16 p-72 d-27 v-17 o-48 t-36 i-16 p-76 d-27 v-17 o-48 t-36 i-16 p-67 d-27 v-17 o-48 t-36 i-16 p-43 d-27 v-17 o-96 t-36 i-16 p-55 d-33 v-17 o-96 t-36 i-16 p-59 d-33 v-17 o-96 t-36 i-16 p-62 d-33 v-17 o-96 t-36 i-16 p-31 d-33 v-17 o-168 t-36 i-16 p-67 d-18 v-17 o-168 t-36 i-16 p-71 d-18 v-17 o-168 t-36 i-16 p-74 d-18 v-17 o-168 t-36 i-16 p-66 d-18 v-17 o-168 t-36 i-16 p-42 d-18 v-17 o-192 t-36 i-16 p-64 d-37 v-17 o-192 t-36 i-16 p-67 d-37 v-17 o-192 t-36 i-16 p-71 d-37 v-17 o-192 t-36 i-16 p-74 d-37 v-17 o-192 t-36 i-16 p-40 d-37 v-17 o-288 t-36 i-16 p-57 d-27 v-17 o-288 t-36 i-16 p-62 d-27 v-17 o-288 t-36 i-16 p-64 d-27 v-17 o-288 t-36 i-16 p-67 d-27 v-17 o-288 t-36 i-16 p-33 d-27 v-17 o-336 t-36 i-16 p-57 d-27 v-17 o-336 t-36 i-16 p-61 d-27 v-17 o-336 t-36 i-16 p-64 d-27 v-17 o-336 t-36 i-16 p-67 d-27 v-17 o-336 t-36 i-16 p-33 d-27 v-17 o-384 t-36 i-16 p-57 d-37 v-17 o-384 t-36 i-16 p-60 d-37 v-17 o-384 t-36 i-16 p-64 d-37 v-17 o-384 t-36 i-16 p-67 d-37 v-17 o-384 t-36 i-16 p-33 d-37 v-17 o-480 t-36 i-16 p-60 d-37 v-17 o-480 t-36 i-16 p-63 d-37 v-17 o-480 t-36 i-16 p-67 d-37 v-17 o-480 t-36 i-16 p-36 d-37 v-17 o-576 t-36 i-16 p-55 d-27 v-17 o-576 t-36 i-16 p-59 d-27 v-17 o-576 t-36 i-16 p-62 d-27 v-17 o-576 t-36 i-16 p-31 d-27 v-17 o-624 t-36 i-16 p-72 d-27 v-17 o-624 t-36 i-16 p-75 d-27 v-17 o-624 t-36 i-16 p-67 d-27 v-17 o-624 t-36 i-16 p-43 d-27 v-17 o-672 t-36 i-16 p-55 d-30 v-17 o-672 t-36 i-16 p-59 d-30 v-17 o-672 t-36 i-16 p-62 d-30 v-17 o-672 t-36 i-16 p-31 d-30 v-17 o-732 t-36 i-16 p-60 d-24 v-17 o-732 t-36 i-16 p-64 d-24 v-17 o-732 t-36 i-16 p-67 d-24 v-17 o-732 t-36 i-16 p-36 d-24 v-17"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1"
# model_result_str = "s-9 o-78 t-39 i-24 p-60 d-6 v-20 o-84 t-39 i-24 p-60 d-6 v-20 o-90 t-39 i-24 p-59 d-6 v-20 o-0 t-39 i-2 p-62 d-37 v-17 o-0 t-39 i-2 p-65 d-37 v-17 o-0 t-39 i-2 p-67 d-37 v-17 o-0 t-39 i-2 p-69 d-37 v-17 o-0 t-39 i-2 p-72 d-37 v-17 o-0 t-39 i-2 p-38 d-37 v-17"
# model_result_str = "s-9 o-0 t-35 i-52 p-68 d-6 v-20 o-6 t-35 i-52 p-64 d-6 v-20 o-12 t-35 i-52 p-66 d-6 v-20 o-18 t-35 i-52 p-68 d-6 v-20 o-24 t-35 i-52 p-71 d-12 v-20 o-48 t-35 i-52 p-70 d-6 v-20 o-54 t-35 i-52 p-66 d-6 v-20 o-60 t-35 i-52 p-68 d-6 v-20 o-66 t-35 i-52 p-70 d-6 v-20 o-72 t-35 i-52 p-73 d-12 v-20 o-96 t-35 i-52 p-75 d-6 v-20 o-102 t-35 i-52 p-73 d-6 v-20 o-108 t-35 i-52 p-71 d-6 v-20 o-114 t-35 i-52 p-70 d-6 v-20 o-120 t-35 i-52 p-71 d-12 v-20 o-132 t-35 i-52 p-66 d-6 v-20 o-144 t-35 i-52 p-71 d-6 v-20 o-150 t-35 i-52 p-70 d-6 v-20 o-156 t-35 i-52 p-68 d-6 v-20 o-162 t-35 i-52 p-66 d-6 v-20 o-168 t-35 i-52 p-68 d-12 v-20 o-180 t-35 i-52 p-63 d-6 v-20 o-192 t-35 i-52 p-64 d-6 v-20 o-198 t-35 i-52 p-66 d-6 v-20 o-204 t-35 i-52 p-68 d-6 v-20 o-216 t-35 i-52 p-68 d-6 v-20 o-222 t-35 i-52 p-70 d-6 v-20 o-228 t-35 i-52 p-71 d-6 v-20 o-240 t-35 i-52 p-73 d-6 v-20 o-246 t-35 i-52 p-71 d-6 v-20 o-252 t-35 i-52 p-70 d-6 v-20 o-258 t-35 i-52 p-71 d-6 v-20 o-264 t-35 i-52 p-73 d-12 v-20 o-276 t-35 i-52 p-66 d-6 v-20 o-288 t-35 i-52 p-71 d-6 v-20 o-294 t-35 i-52 p-73 d-6 v-20 o-300 t-35 i-52 p-75 d-6 v-20 o-306 t-35 i-52 p-76 d-6 v-20 o-312 t-35 i-52 p-78 d-6 v-20 o-318 t-35 i-52 p-76 d-6 v-20 o-324 t-35 i-52 p-75 d-6 v-20 o-330 t-35 i-52 p-73 d-6 v-20 o-336 t-35 i-52 p-68 d-6 v-20 o-342 t-35 i-52 p-70 d-6 v-20 o-348 t-35 i-52 p-71 d-6 v-20 o-354 t-35 i-52 p-73 d-6 v-20 o-360 t-35 i-52 p-75 d-6 v-20 o-366 t-35 i-52 p-68 d-6 v-20 o-372 t-35 i-52 p-75 d-12 v-20 o-384 t-35 i-52 p-68 d-6 v-20 o-390 t-35 i-52 p-64 d-6 v-20 o-396 t-35 i-52 p-66 d-6 v-20 o-402 t-35 i-52 p-68 d-6 v-20 o-408 t-35 i-52 p-71 d-12 v-20 o-432 t-35 i-52 p-70 d-6 v-20 o-438 t-35 i-52 p-66 d-6 v-20 o-444 t-35 i-52 p-68 d-6 v-20 o-450 t-35 i-52 p-70 d-6 v-20 o-456 t-35 i-52 p-73 d-12 v-20 o-480 t-35 i-52 p-75 d-6 v-20 o-486 t-35 i-52 p-73 d-6 v-20 o-492 t-35 i-52 p-71 d-6 v-20 o-498 t-35 i-52 p-70 d-6 v-20 o-504 t-35 i-52 p-71 d-12 v-20 o-516 t-35 i-52 p-66 d-6 v-20 o-528 t-35 i-52 p-71 d-6 v-20 o-534 t-35 i-52 p-70 d-6 v-20 o-540 t-35 i-52 p-68 d-6 v-20 o-546 t-35 i-52 p-66 d-6 v-20 o-552 t-35 i-52 p-68 d-12 v-20 o-564 t-35 i-52 p-63 d-6 v-20 o-576 t-35 i-52 p-64 d-6 v-20 o-582 t-35 i-52 p-66 d-6 v-20 o-588 t-35 i-52 p-68 d-6 v-20 o-600 t-35 i-52 p-68 d-6 v-20 o-606 t-35 i-52 p-70 d-6 v-20 o-612 t-35 i-52 p-71 d-6 v-20 o-624 t-35 i-52 p-73 d-6 v-20 o-630 t-35 i-52 p-71 d-6 v-20 o-636 t-35 i-52 p-70 d-6 v-20 o-642 t-35 i-52 p-71 d-6 v-20 o-648 t-35 i-52 p-73 d-6 v-20 o-654 t-35 i-52 p-66 d-6 v-20 o-660 t-35 i-52 p-70 d-6 v-20 o-666 t-35 i-52 p-73 d-6 v-20 o-672 t-35 i-52 p-71 d-6 v-20 o-684 t-35 i-52 p-66 d-6 v-20 o-696 t-35 i-52 p-71 d-12 v-20 o-720 t-35 i-52 p-71 d-6 v-20 o-732 t-35 i-52 p-75 d-6 v-20 o-744 t-35 i-52 p-71 d-12 v-20 o-0 t-35 i-90 p-61 d-27 v-17 o-0 t-35 i-90 p-64 d-27 v-17 o-0 t-35 i-90 p-68 d-27 v-17 o-0 t-35 i-90 p-37 d-27 v-17 o-48 t-35 i-90 p-78 d-27 v-17 o-48 t-35 i-90 p-70 d-27 v-17 o-48 t-35 i-90 p-73 d-27 v-17 o-48 t-35 i-90 p-46 d-27 v-17 o-96 t-35 i-90 p-59 d-27 v-17 o-96 t-35 i-90 p-63 d-27 v-17 o-96 t-35 i-90 p-66 d-27 v-17 o-96 t-35 i-90 p-35 d-27 v-17 o-144 t-35 i-90 p-68 d-27 v-17 o-144 t-35 i-90 p-71 d-27 v-17 o-144 t-35 i-90 p-75 d-27 v-17 o-144 t-35 i-90 p-78 d-27 v-17 o-144 t-35 i-90 p-44 d-27 v-17 o-192 t-35 i-90 p-64 d-27 v-17 o-192 t-35 i-90 p-68 d-27 v-17 o-192 t-35 i-90 p-71 d-27 v-17 o-192 t-35 i-90 p-40 d-27 v-17 o-240 t-35 i-90 p-78 d-27 v-17 o-240 t-35 i-90 p-70 d-27 v-17 o-240 t-35 i-90 p-73 d-27 v-17 o-240 t-35 i-90 p-46 d-27 v-17 o-288 t-35 i-90 p-59 d-27 v-17 o-288 t-35 i-90 p-63 d-27 v-17 o-288 t-35 i-90 p-66 d-27 v-17 o-288 t-35 i-90 p-35 d-27 v-17 o-336 t-35 i-90 p-68 d-27 v-17 o-336 t-35 i-90 p-71 d-27 v-17 o-336 t-35 i-90 p-75 d-27 v-17 o-336 t-35 i-90 p-78 d-27 v-17 o-336 t-35 i-90 p-44 d-27 v-17 o-384 t-35 i-90 p-61 d-27 v-17 o-384 t-35 i-90 p-64 d-27 v-17 o-384 t-35 i-90 p-68 d-27 v-17 o-384 t-35 i-90 p-37 d-27 v-17 o-432 t-35 i-90 p-78 d-27 v-17 o-432 t-35 i-90 p-70 d-27 v-17 o-432 t-35 i-90 p-73 d-27 v-17 o-432 t-35 i-90 p-46 d-27 v-17 o-480 t-35 i-90 p-59 d-18 v-17 o-480 t-35 i-90 p-63 d-18 v-17 o-480 t-35 i-90 p-66 d-18 v-17 o-480 t-35 i-90 p-35 d-18 v-17 o-504 t-35 i-90 p-80 d-18 v-17 o-504 t-35 i-90 p-71 d-18 v-17 o-504 t-35 i-90 p-75 d-18 v-17 o-504 t-35 i-90 p-78 d-18 v-17 o-504 t-35 i-90 p-47 d-18 v-17 o-528 t-35 i-90 p-68 d-27 v-17 o-528 t-35 i-90 p-71 d-27 v-17 o-528 t-35 i-90 p-75 d-27 v-17 o-528 t-35 i-90 p-78 d-27 v-17 o-528 t-35 i-90 p-44 d-27 v-17 o-576 t-35 i-90 p-61 d-27 v-17 o-576 t-35 i-90 p-64 d-27 v-17 o-576 t-35 i-90 p-68 d-27 v-17 o-576 t-35 i-90 p-71 d-27 v-17 o-576 t-35 i-90 p-37 d-27 v-17 o-624 t-35 i-90 p-78 d-27 v-17 o-624 t-35 i-90 p-70 d-27 v-17 o-624 t-35 i-90 p-73 d-27 v-17 o-624 t-35 i-90 p-46 d-27 v-17 o-672 t-35 i-90 p-59 d-18 v-17 o-672 t-35 i-90 p-63 d-18 v-17 o-672 t-35 i-90 p-66 d-18 v-17 o-672 t-35 i-90 p-35 d-18 v-17 o-696 t-35 i-90 p-59 d-18 v-17 o-696 t-35 i-90 p-63 d-18 v-17 o-696 t-35 i-90 p-66 d-18 v-17 o-696 t-35 i-90 p-35 d-18 v-17 o-720 t-35 i-90 p-59 d-18 v-17 o-720 t-35 i-90 p-63 d-18 v-17 o-720 t-35 i-90 p-66 d-18 v-17 o-720 t-35 i-90 p-35 d-18 v-17 o-744 t-35 i-90 p-59 d-18 v-17 o-744 t-35 i-90 p-63 d-18 v-17 o-744 t-35 i-90 p-66 d-18 v-17 o-744 t-35 i-90 p-35 d-18 v-17"
# model_result_str = "s-9 o-0 t-31 i-35 p-43 d-9 v-25 i-78 p-79 d-6 v-20 o-12 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-33 i-0 p-71 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 o-12 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 o-12 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-52 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 o-12 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-9 o-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-64 d-3 v-20 p-60 I1s2_7_1 v-20 b-1 s-9 o-0 t-35 i-40 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-16 p-60 d-11 v-20 i-42 p-48 p-48 d-12 v-20 o-24 t-35 i-0 p-64 t-35 i-40 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 v-20 b-1 s-9 o-0 t-35 i-0 p-64 p-64 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-41 d-6 v-24 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-12 p-64 d-12 v-20 o-12 t-35 i-0 p-69 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-56 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-8 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 p-60 d-12 v-20 o-12 t-35 i-0 p-64 d-6 v-20 i-128 p-170 t-35 i-0 p-64 d-3 v-20 p-48 i-42 d-3 v-20 o-12 t-35 i-0 p-60 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-12 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 o-12 t-35 i-0 p-65 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 o-0 t-35 i-0 p-67 d-3 v-20 p-64 v-20 p-55 d-6 v-20 p-64 d-3 v-20 o-24 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 o-12 t-35 i-0 p-67 d-3 v-20 p-52 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-1 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-3 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-3 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-14 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-42 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 v-20 o-18 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-24 t-35 i-0 p-67 d-3 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-56 p-67 d-3 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-79 d-6 v-20 b-1 s-8 o-0 d-3 v-20 b-1 s-9 o-0 t-35 i-12 p-65 t-35 i-0 o-0 d-6 v-20 b-1 s-8 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 v-20 o-6 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-52 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 i-0 p-64 d-6 v-20 b-1 s-8 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-6 p-57 d-6 v-20 p-48 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 v-20 b-1 v-20 o-12 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 i-56 i-0 p-76 v-20 o-12 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-57 o-0 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-60 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-12 v-20 b-1 s-9 o-0 t-35 i-0 p-67 p-57 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-3 v-20 b-1 s-3 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 v-20 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 v-20 p-52 d-6 v-20 p-55 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 d-3 v-20 o-12 d-6 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-56 i-0 v-20 o-6 t-35 i-0 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-77 p-65 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 p-57 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 i-1 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 o-6 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 o-0 t-35 i-0 i-0 p-67 d-3 v-20 i-56 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 o-0 t-35 i-0 v-20 o-6 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-72 d-6 v-20 o-24 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-3 v-20 b-1 s-8 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 v-20 o-12 t-35 i-0 p-64 t-35 i-0 p-64 d-6 v-20 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 t-35 i-0 p-67 d-3 v-20 b-1 s-8 v-20 p-64 d-3 v-20 p-60 d-3 v-20 b-1 s-8 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-3 v-20 p-48 d-6 v-20 p-53 o-0 t-35 i-0 p-67 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-76 t-35 i-0 p-77 v-20 o-12 t-35 t-35 i-0 p-67 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 p-72 v-20 b-1 i-1 p-64 d-3 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-76 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-67 t-35 i-0 p-60 v-20 o-24 t-35 i-0 p-67 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-24 t-35 i-0 p-67 v-20 p-60 d-6 v-20 i-1 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 p-57 d-6 v-20 b-1 v-20 b-1 s-8 o-0 t-35 i-0 p-76 o-0 t-35 i-0 o-0 t-35 i-1 v-20 o-12 v-20 p-57 d-6 v-20 p-57 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 v-20 o-12 t-35 i-0 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 p-52 d-6 v-20 o-0 t-35 i-0 p-67 d-6 v-20 b-1 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 i-42 o-0 t-35 i-0 p-67 t-35 i-0 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-76 v-20 b-1 s-20 d-6 v-20 o-36 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 p-67 d-3 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 p-71 o-0 t-35 i-0 p-76 v-20 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-30 t-35 i-0 p-74 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-35 i-0 p-67 o-0 t-35 i-0 p-67 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-64 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 t-35 i-0 o-0 t-35 i-0 o-0 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 p-57 p-67 t-35 i-0 p-74 v-20 b-1 v-20 p-50 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 p-64 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 v-20 b-1 s-8 o-0 t-35 i-0 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 o-6 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 d-3 v-20 b-1 i-0 t-35 i-0 p-72 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-9 o-0 t-35 i-0 o-0 v-20 b-1 s-8 o-0 t-35 i-56 t-35 i-0 p-71 d-6 v-20 t-35 i-0 p-76 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 i-0 p-76 d-6 v-20 b-1 s-8 o-0 p-48 d-6 v-20 o-6 t-35 i-0 p-67 d-3 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 p-48 i-0 p-67 v-20 o-6 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 v-20 o-12 t-35 i-0 v-20 o-6 t-35 i-0 o-0 t-35 i-0 o-0 t-35 i-0 v-20 b-1 v-20 p-57 d-6 v-20 p-64 d-3 v-20 p-64 d-3 v-20 p-57 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 t-35 i-0 p-67 d-3 v-20 o-24 t-35 i-0 p-76 i-0 p-64 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-3 v-20 b-1 i-1 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 v-20 o-24 t-35 i-0 p-65 v-20 o-24 t-35 i-0 p-67 d-3 v-20 o-12 t-35 i-0 p-48 v-20 p-57 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 t-35 i-0 p-67 d-3 v-20 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-24 t-42 d-6 v-24 p-55 d-6 v-20 i-60 p-57 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-60 d-12 d-12 v-20 o-6 p-55 d-12 v-20 b-1 s-9 o-0 t-35 i-24 d-6 v-20 o-24 t-35 i-0 p-36 d-6 v-20 b-1 s-9 o-0 t-35 i-9 d-18 v-20 b-1 s-8 o-0 t-35 i-40 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-9 d-6 v-20 o-24 t-35 i-24 d-12 v-24 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 o-12 t-35 d-6 v-24 d-6 v-20 b-1 s-9 o-0 t-35 i-58 d-12 v-20 b-1 s-9 o-0 t-35 d-6 v-24 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-60 p-59 o-0 t-35 i-12 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-9 d-6 v-20 p-60 d-12 t-34 v-20 o-12 t-35 i-48 p-69 d-6 v-20 o-18 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-14 d-12 v-20 b-1 s-9 o-0 t-35 i-48 o-0 t-35 i-57 p-55 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 i-58 t-35 i-0 p-62 d-6 v-20 o-24 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 p-57 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 i-128 t-35 i-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 v-20 b-1 s-9 o-0 t-35 i-0 p-71 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 o-24 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-12 d-6 v-20 i-57 d-6 v-20 b-1 s-9 o-0 t-35 i-58 p-52 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 o-12 t-35 i-57 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 p-69 p-62 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 p-57 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-57 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-14 t-35 i-57 p-59 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 o-6 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-48 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-24 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-24 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-12 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-48 d-6 v-20 o-24 t-35 i-56 i-52 p-72 t-35 i-58 t-35 v-20 o-6 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 o-12 t-35 i-12 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-48 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-65 d-6 v-20 o-24 t-35 i-0 p-67 d-12 v-20 o-12 t-35 i-0 p-76 d-6 v-20 o-30 t-35 i-58 p-57 d-6 v-20 o-24 t-35 i-56 p-60 d-6 v-20 b-1 s-7 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-72 d-6 d-6 v-20 p-60 d-6 v-20 o-24 t-35 i-128 p-187 d-6 v-20 o-12 v-20 b-1 s-8 o-0 t-35 v-20 o-12 t-35 i-0 p-65 d-6 v-20 o-12 t-35 i-0 p-62 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-164 d-6 v-20 o-24 d-12 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 t-35 i-0 o-0 t-35 i-57 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-72 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 o-6 t-35 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 b-1 s-8 o-0 t-35 i-48 p-48 d-6 v-20 p-57 d-6 v-20 o-42 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-77 d-6 v-20 o-18 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-43 d-12 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-56 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-14 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 v-20 p-55 d-6 v-20 b-1 s-3 t-35 i-0 p-57 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-12 v-20 o-24 t-35 i-0 p-76 t-35 i-58 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 o-0 t-35 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-164 d-6 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 d-6 v-20 b-1 s-8 o-0 t-34 i-0 t-35 i-0 p-72 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 i-0 p-65 d-6 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 p-64 d-6 v-20 p-57 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 d-6 v-20 d-12 v-20 o-12 t-35 i-58 d-6 v-20 o-24 t-35 i-0 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-76 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-71 d-12 v-20 o-12 t-35 i-0 p-67 d-12 v-20 b-1 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 d-6 v-20 b-1 v-20 b-1 s-9 o-0 t-35 i-0 p-77 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 p-48 v-20 p-60 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 t-35 i-0 p-76 v-20 p-53 d-6 v-20 b-1 s-9 o-0 t-35 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 v-20 o-42 t-35 i-128 p-170 d-6 v-20 p-64 d-6 v-20 b-1 s-8 v-20 p-57 d-6 v-20 p-55 d-6 v-20 o-12 t-35 i-24 t-35 i-0 p-65 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-3 v-20 b-1 s-8 o-0 t-35 i-0 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-57 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 i-48 v-20 p-57 o-0 t-35 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-6 t-35 i-57 o-0 t-35 i-0 p-64 d-6 v-20 p-60 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-48 d-6 v-20 p-57 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-48 o-0 t-35 i-58 d-6 v-20 d-6 v-20 i-65 d-6 v-20 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-6 t-35 i-0 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-18 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 p-57 d-6 v-20 p-48 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 b-1 v-20 o-12 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 d-6 v-20 b-1 s-8 o-0 t-35 i-58 d-6 v-20 o-24 t-35 i-56 v-20 o-6 t-35 i-57 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 p-43 d-6 v-20 o-24 t-35 v-20 o-24 t-35 v-20 p-48 d-6 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 p-48 d-6 v-20 i-65 p-74 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 v-20 o-12 t-35 i-0 p-64 d-6 v-20 o-42 t-35 i-57 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-8 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-24 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-24 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 b-1 v-20 p-60 t-35 i-0 p-67 d-6 v-20 i-56 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-12 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 v-20 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 v-20 o-24 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-60 d-6 v-20 p-48 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-57 d-6 v-20 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-65 t-35 i-40 p-77 t-35 i-0 p-65 v-20 b-1 v-20 o-24 t-35 i-57 o-0 t-35 t-35 i-0 p-67 d-6 v-20 b-1 s-9 d-6 v-20 p-48 t-35 i-0 p-67 d-6 v-20 o-18 t-35 d-6 v-20 o-12 t-35 i-0 o-0 t-35 i-0 p-72 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 t-35 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-6 v-20 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-53 d-6 v-20 b-1 s-8 o-0 t-35 i-24 v-20 o-24 t-35 i-0 p-67 d-6 v-20 o-12 t-35 i-24 p-48 d-6 v-20 o-24 t-35 i-48 t-35 i-0 p-48 d-6 v-20 o-12 t-35 v-20 p-45 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-65 t-35 i-0 p-59 d-6 v-20 o-6 t-35 i-0 p-64 d-6 v-20 o-36 t-35 i-0 v-20 p-64 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-48 d-6 v-20 d-6 v-20 p-60 t-35 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-8 o-0 v-20 b-1 s-8 v-20 b-1 v-20 b-1 s-9 o-0 t-35 i-56 d-6 v-20 b-1 s-9 o-0 t-35 i-24 p-41 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-8 o-0 t-35 v-20 p-57 t-35 i-56 p-67 d-6 v-20 i-42 p-43 d-6 v-20 p-57 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-24 p-57 d-6 v-20 b-1 s-9 o-0 d-6 v-20 d-6 v-20 p-60 v-20 o-24 t-35 i-0"
# model_result_str = "s-8 o-0 t-34 i-0 p-62 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-6 v-20 o-24 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-48 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-48 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 i-42 p-48 d-6 v-20 i-42 p-48 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-50 d-6 v-20 i-42 p-50 d-6 v-20 i-42 p-50 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-50 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-50 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 I1s2_2_1 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-35 I1s2_2_1 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 I1s2_2_1 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 i-42 p-50 d-6 v-20 i-42 p-50 d-6 v-20 b-1 s-8 o-0 t-35 I1s2_2_1 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 i-40 p-65 d-6 v-20 i-42"
# model_result_str = "s-8 o-0 t-34 i-0 p-62 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-72 d-11 v-20 b-1 s-9 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-18 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-67 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 i-42 p-48 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 i-42 p-48 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 i-42 S4_6_1 TS1s1_7 K1_2 T1s1_3 P4_12 ST1_14 EM1_4 TM1_5 <sep> s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-12 v-20 i-42 S4_6_1 TS1s1_7 K1_2 T1s1_3 P4_12 ST1_14 EM1_4 TM1_5 <sep> s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-62 d-11 v-20 i-42 p-50 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 I1s2_2_2 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-12 v-20 i-42 p-50 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 p-55 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 I1s2_2_2 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_2 S4_7_1 S4_8_1 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 S4_9_1 C1_4 R1_2 R3_3 S2s1_17 ST1_14 EM1_4 TM1_5 <sep> s-8 o-0 t-41 i-40 p-65 d-12 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 i-42 S4_6_1 TS1s1_7 K1_2 T1s1_3 P4_12 ST1_14 EM1_4 TM1_5 <sep> s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 i-24 p-62 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-6 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-64 d-6 v-20 i-42 p-55 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-35 I1s2_2_1 i-40 p-65 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-12 v-20 i-42 S4_6_1 S4_7_1 S4_8_1 t-41 i-40 p-65 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 b-1 s-8"
# model_result_str = "o-30 t-35 i-60 p-62 o-30 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-56 p-72 v-24 b-1 s-9 o-0 t-35 i-40 p-81 d-11 v-20 i-60 p-64 d-6 v-20 o-6 t-35 i-58 p-64 d-6 v-20 p-48 d-6 v-20 o-6 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 p-48 d-6 v-20 b-1 s-9 o-0 t-35 i-57 p-55 d-6 v-20 i-128 p-170 v-24 p-59 d-6 v-20 o-24 t-35 i-0 p-74 d-6 v-20 o-18 v-16 i-57 v-20 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-58 p-64 d-6 v-20 o-24 t-35 i-71 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-43 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 v-20 i-57 p-48 v-20 o-24 d-6 v-20 i-43 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-36 t-35 i-0 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 i-128 p-170 d-6 v-20 p-52 d-6 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 v-20 b-1 s-9 o-0 t-35 i-0 p-67 v-20 o-18 t-35 i-48 d-12 v-20 b-1 s-9 o-0 t-35 i-52 o-0 t-35 i-0 p-67 d-12 v-20 b-1 s-9 o-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 i-58 p-48 t-35 i-0 p-69 d-6 v-24 i-67 t-35 i-0 p-67 d-6 v-20 p-64 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-38 v-20 o-24 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 o-18 d-6 v-20 b-1 s-8 o-0 t-35 i-0 d-6 v-20 p-64 d-6 v-20 o-18 t-35 i-0 p-69 d-6 v-20 p-57 d-6 v-20 d-6 v-20 p-64 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 b-1 s-9 o-0 t-35 i-52 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-34 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-48 v-20 o-24 t-35 i-0 p-72 d-6 v-20 o-36 t-30 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-60 d-12 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 t-35 i-0 p-69 d-12 v-20 b-1 v-20 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-60 d-6 v-20 i-128 p-170 d-6 v-20 p-60 d-6 v-20 p-62 d-6 v-20 p-48 d-6 v-20 o-6 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-67 d-6 v-20 o-6 t-35 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-6 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 p-74 i-0 p-67 d-6 v-20 p-164 d-6 v-20 o-30 t-35 i-0 p-62 v-20 p-60 d-6 v-20 b-1 t-33 i-128 p-170 d-6 v-20 i-128 p-173 t-34 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 t-35 i-0 p-76 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-64 p-64 d-6 v-20 o-6 t-35 i-0 p-77 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-48 t-35 i-0 p-64 d-6 v-20 b-1 d-6 d-6 v-20 o-6 t-35 i-0 d-6 v-20 o-18 t-35 i-0 p-64 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-52 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-76 d-6 v-24 i-42 t-35 i-0 p-76 d-6 v-20 p-64 d-6 v-20 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-30 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-60 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-0 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-0 d-6 v-20 o-18 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-173 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-52 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 p-57 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-6 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-6 t-35 i-0 d-6 v-20 p-164 d-6 v-20 d-6 v-20 o-18 t-35 i-0 p-60 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-24 t-35 i-128 p-170 d-6 v-20 o-36 v-20 b-1 s-9 o-0 t-35 i-0 p-76 v-20 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-64 d-6 v-20 o-6 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-64 d-6 v-20 i-128 p-170 v-20 o-30 t-35 i-0 v-20 b-1 v-20 o-12 t-35 i-0 p-67 d-6 v-20 i-0 p-60 d-6 v-20 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 d-6 v-20 o-12 t-35 i-0 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 o-42 t-35 i-0 p-64 d-6 v-20 p-55 d-6 v-20 o-12 t-30 i-0 p-71 v-20 b-1 s-9 o-0 t-35 i-40 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 d-6 v-20 o-36 t-35 i-0 p-76 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 i-128 p-170 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-38 i-0 p-69 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-12 t-35 i-0 p-67 d-6 v-20 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 o-18 t-35 i-0 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 d-6 v-20 p-60 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 i-48 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-30 i-0 p-64 d-6 v-20 i-128 t-35 i-0 p-67 d-6 v-20 b-1 s-9 v-20 b-1 s-9 o-0 t-35 i-52 d-6 v-20 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 i-128 p-170 d-6 v-20 o-24 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 i-128 p-170 d-6 v-20 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 o-36 d-6 v-20 b-1 s-9 o-0 t-35 d-6 v-20 o-12 t-35 i-0 p-74 d-6 v-20 p-55 d-6 v-20 o-24 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 v-20 i-128 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 p-64 d-3 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 t-35 i-0 p-59 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-12 v-20 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 d-6 v-20 i-128 p-170 d-6 v-20 i-128 p-170 d-6 v-20 o-36 t-35 i-0 p-74 t-35 i-0 d-6 v-20 p-57 d-6 v-20 b-1 s-9 o-0 t-35 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 t-35 i-0 p-79 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 t-35 i-0 p-64 i-0 p-67 d-6 v-20 p-55 d-6 v-20 p-64 d-6 v-20 i-0 p-67 d-6 v-20 o-18 t-35 i-0 t-35 i-0 d-6 v-20 o-24 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-69 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 o-0 t-35 i-0 p-72 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 v-20 o-12 t-35 i-0 p-76 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 d-6 v-20 p-62 d-6 v-20 i-48 p-55 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 i-128 p-170 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-74 d-6 v-20 b-1 s-9 o-0 t-35 i-24 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 b-1 s-9 o-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-48 d-6 v-20 i-128 p-173 t-35 i-0 d-6 v-20 i-128 p-170 t-30 i-0 p-60 d-6 v-20 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 d-6 v-20 p-55 d-6 v-20 p-57 d-6 v-20 d-6 v-20 b-1 s-9 o-0 o-0 t-35 i-0 p-57 d-6 v-20 v-20 i-48 t-35 i-0 t-30 i-0 d-6 v-20 o-18 t-35 t-35 i-0 d-6 v-20 t-35 i-0 d-6 v-20 p-43 d-6 v-20 b-1 s-7 o-0 t-35 i-0 p-55 d-6 v-20 p-55 d-11 v-20 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-74 d-6 v-20 o-18 t-35 i-0 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 i-128 p-170 d-6 v-20 o-18 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-30 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-18"
# model_result_str = "s-8 o-0 t-41 i-40 p-65 d-2 v-24 i-42 p-46 d-6 v-20 o-12 t-41 i-42 p-52 d-11 v-20 o-6 t-41 i-40 p-74 d-1 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-11 v-20 p-55 d-17 v-20 b-1 s-8 o-0 t-41 i-42 p-41 d-12 v-20 i-42 p-46 d-6 v-20 o-30 t-41 i-40 p-72 d-3 v-20 o-24 t-41 i-40 p-65 d-3 v-20 o-30 t-41 i-40 p-69 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-72 d-1 v-20 o-24 t-41 i-40 p-69 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-65 d-6 v-20 i-42 p-67 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-67 d-4 v-20 o-36 t-41 i-42 p-65 d-6 v-20 i-42 p-60 d-6 v-20 i-42 p-40 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-6 v-20 p-62 d-12 v-20 i-42 p-45 d-12 v-20 i-42 p-48 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 o-42 t-41 i-42 p-52 d-17 v-20 b-1 s-9 o-0 t-41 i-40 p-64 d-6 v-20 i-42 p-52 d-1 v-20 i-41 p-62 d-11 v-20 i-42 p-60 d-2 v-20 o-24 t-41 i-40 p-69 d-3 v-20 i-42 p-48 d-15 v-20 b-1 s-8 o-0 t-41 i-40 p-62 d-6 v-20 o-24 t-41 i-40 p-69 d-1 v-20 i-42 p-52 d-32 v-20 o-12 t-41 i-40 p-62 d-12 v-20 i-42 p-55 d-17 v-20 o-24 t-41 i-40 p-71 d-11 v-20 o-42 t-41 i-40 p-69 d-4 v-20 i-41 p-64 d-15 v-20 i-42 p-47 d-4 v-20 i-42 p-52 d-2 v-20 o-12 t-41 i-40 p-69 d-4 v-20 i-40 p-69 d-12 v-20 i-42 p-50 d-6 v-20 i-42 p-52 d-12 v-20 i-42 p-53 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-64 d-6 v-20 i-42 p-47 d-2 v-20 i-42 p-52 d-2 v-20 i-42 p-52 d-6 v-20 o-24 t-41 i-40 p-67 d-18 v-20 i-42 p-52 d-1 v-20 i-42 p-47 d-11 v-20 i-42 p-43 d-12 v-20 i-42 p-38 d-12 v-20 i-40 p-69 d-18 v-20 i-42 p-38 d-6 v-20 o-8 t-41 i-42 p-48 d-6 v-20 i-42 p-62 d-6 v-20 i-42 p-48 d-18 v-20 i-40 p-62 d-6 v-20 i-42 p-50 d-6 v-20 o-24 t-41 i-42 p-48 d-6 v-20 i-40 p-64 d-1 v-20 i-42 p-47 d-3 v-20 i-42 p-53 d-6 v-20 o-12 t-41 i-40 p-72 d-18 v-20 i-42 p-48 d-6 v-20 o-24 t-41 i-40 p-64 d-6 v-20 o-24 t-41 i-40 p-69 d-15 v-20 i-42 p-55 d-2 v-20 i-42 p-48 d-3 v-20 i-41 p-57 d-1 v-20 o-12 t-41 i-40 p-76 d-6 v-20 i-40 p-69 d-3 v-20 i-42 p-67 d-3 v-20 i-41 p-57 d-6 v-20 i-41 p-50 d-6 v-20 i-42 p-48 d-18 v-20 i-42 p-44 d-6 v-20 i-42 p-53 d-6 v-20 i-42 p-38 d-6 v-20 i-42 p-45 d-3 v-20 i-41 p-55 d-12 v-20 o-24 t-41 i-40 p-69 d-6 v-20 i-42 p-37 d-4 v-20 i-42 p-48 d-6 v-20 i-42 p-50 d-1 v-20 i-40 p-67 d-4 v-20 i-42 p-47 d-4 v-20 i-42 p-43 d-6 v-20 i-42 p-52 d-6 v-20 i-42 p-62 d-3 v-20 i-42 p-53 d-18 v-20 i-42 p-50 d-6 v-20 i-42 p-65 d-6 v-20 i-42 p-62 d-1 v-20 i-42 p-48 d-6 v-20 i-42 p-48 d-12 v-20 i-42 p-57 d-6 v-20 o-12 t-41 i-40 p-64 d-6 v-20 i-42 p-40 d-18 v-20 i-40 p-62 d-6 v-20 i-42 p-53 d-6 v-20 i-42 p-43 d-17 v-20 i-42 p-48 d-12 v-20 i-40 p-65 d-6 v-20 i-41 p-55 d-6 v-20 i-42 p-64 d-1 v-20 i-40 p-74 d-1 v-20 i-42 p-47 d-15 v-20 i-42 p-47 d-15 v-20 i-42 p-40 d-6 v-20 i-42 p-38 d-18 v-20 i-40 p-69 d-6 v-20 i-42 p-60 d-18 v-20 i-42 p-45 d-6 v-20 i-42 p-43 d-18 v-20 i-42 p-62 d-6 v-20 i-42 p-69 d-6 v-20 i-42 p-64 d-6 v-20 i-42 p-62 d-15 v-20 i-42 p-43 d-6 v-20 i-42 p-48 d-6 v-20 i-42 p-48 d-6 v-20 i-40 p-69 d-6 v-20 i-42 p-40 d-6 v-20 i-41 p-62 d-2 v-20 i-41 p-64 d-12 v-20 i-41 p-62 d-6 v-20 i-42 p-47 d-15 v-20 i-42 p-47 d-6 v-20 i-41 p-50 d-6 v-20 i-42 p-52 d-18 v-20 i-42 p-43 d-18 v-20 i-42 p-50 d-11 v-20 i-42 p-65 d-12 v-20 i-42 p-48 d-6 v-20 i-40 p-65 d-3 v-20 i-41 p-55 d-3 v-20 i-42 p-57 d-2 v-20 i-41 p-64 d-6 v-20 i-42 p-44 d-11 v-20 i-42 p-48 d-18 v-20 i-42 p-44 d-3 v-20 i-42 p-48 d-6 v-20 i-40 p-69 d-24 v-20 i-42 p-40 d-18 v-20 i-42 p-52 d-2 v-20 i-42 p-64 d-12 v-20 i-41 p-52 d-6 v-20 i-42 p-38 d-6 v-20 i-42 p-65 d-2 v-20 i-42 p-43 d-6 v-20 i-42 p-55 d-6 v-20 i-42 p-46 d-12 v-20 i-40 p-67 d-6 v-20 i-42 p-38 d-6 v-20 i-42 p-47 d-6 v-20 i-42 p-48 d-27 v-20 i-42 p-41 d-6 v-20 i-42 p-43 d-3 v-20 i-42 p-45 d-3 v-20 i-42 p-53 d-6 v-20 i-42 p-59 d-2 v-20 i-42 p-64 d-6 v-20 i-42 p-50 d-18 v-20 i-42 p-50 d-6 v-20 i-42 p-38 d-11 v-20 i-42 p-64 d-3 v-20 i-42 p-50 d-6 v-20 i-42 p-47 d-6 v-20 i-42 p-50 d-6 v-20 i-42 p-48 d-6 v-20 i-42 p-67 d-4 v-20 i-40 p-65 d-15 v-20 i-42 p-38 d-6 v-20 i-41 p-62 d-6 v-20 i-42 p-47 d-12 v-20 i-42 p-38 d-6 v-20 i-40 p-74 d-18 v-20 i-42 p-50 d-6 v-20 i-42 p-53 d-2 v-20 i-41 p-69 d-18 v-20 i-42 p-47 d-6 v-20 i-40 p-72 d-12 v-20 i-42 p-36 d-18 v-20 i-42 p-55 d-6 v-20 i-40 p-69 d-12 v-20 i-41 p-67 d-6 v-20 i-42 p-44 d-6 v-20 i-40 p-69 d-12 v-20 i-42 p-47 d-4 v-20 i-42 p-45 d-6 v-20 i-42 p-48 d-11 v-20 i-42 p-47 d-6 v-20 i-42 p-46 d-6 v-20 i-42 p-38 d-12 v-20 i-40 p-69 d-18 v-20 i-42 p-47 d-6 v-20 i-42 p-69 d-3 v-20 i-40 p-69 d-18 v-20 i-42 p-50 d-6 v-20 i-42 p-48 d-6 v-20 i-42 p-69 d-2 v-20 i-42 p-47 d-6 v-20 i-40 p-69 d-18 v-20 i-42 p-50 d-6 v-20 i-42 p-62 d-6 v-20 i-42 p-45 d-6 v-20 i-40 p-74 d-18 v-20 i-42 p-48 d-6 v-20 i-42 p-41 d-6 v-20 i-40 p-67 d-6 v-20 i-42 p-57 d-3 v-20 i-42 p-48 d-6 v-20 i-41 p-64 d-3 v-20 i-41 p-55 d-18 v-20 i-42"
# model_result_str = "s-8 o-0 t-41 i-42 p-43 d-6 v-20 o-24 t-41 i-42 p-50 d-12 v-20 o-12 t-41 i-0 p-65 d-11 v-20 o-12 t-41 i-40 p-69 d-11 v-20 o-30 t-41 i-0 p-65 d-12 v-20 p-52 d-6 v-20 o-24 t-41 i-40 p-65 d-6 v-20 o-24 t-41 i-42 p-43 d-23 v-20 b-1 s-8 o-0 t-41 i-40 p-64 d-12 v-20 b-1 s-8 o-0 t-41 i-40 p-65 d-6 v-20 p-50 d-11 v-20 o-12 t-41 i-42 p-50 d-6 v-20 o-6 t-41 i-42 p-50 d-11 v-20 i-42 p-55 d-18 v-20 o-42 t-41 i-40 p-69 d-6 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-6 v-20 o-24 t-41 i-40 p-62 d-18 v-20 p-43 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-33 d-12 v-20 o-8 t-41 i-42 p-52 d-6 v-20 o-12 t-41 i-42 p-38 d-12 v-20 p-52 d-23 v-20 i-41 p-55 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-50 d-11 v-20 o-24 t-41 i-42 p-38 d-11 v-20 o-24 t-41 i-42 p-48 d-17 v-20 p-48 d-17 v-20 b-1 s-8 o-0 t-41 i-42 p-52 d-6 v-20 o-12 t-41 i-42 p-38 d-11 v-20 p-52 d-17 v-20 o-24 t-41 i-42 p-38 d-11 v-20 o-24 t-41 i-42 p-50 d-11 v-20 o-24 t-41 i-40 p-60 d-11 v-20 i-42 p-55 d-17 v-20 o-12 t-41 i-42 p-57 d-11 v-20 b-1 s-8 o-0 t-41 i-40 p-67 d-11 v-20 o-12 t-41 i-42 p-55 d-11 v-20 o-12 t-41 i-42 p-53 d-17 v-20 o-12 t-41 i-40 p-62 d-11 v-20 i-42 p-64 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-65 d-15 v-20 o-6 t-41 i-42 p-46 d-11 v-20 o-30 t-41 i-42 p-48 d-11 v-20 i-42 p-67 d-11 v-20 o-24 t-41 i-42 p-55 d-17 v-20 o-12 t-41 i-40 p-65 d-11 v-20 o-12 t-41 i-42 p-41 d-11 v-20 i-42 p-47 d-11 v-20 i-42 p-42 d-11 v-20 o-24 t-41 i-42 p-52 d-11 v-20 i-42 p-48 d-11 v-20 o-24 t-41 i-42 p-47 d-11 v-20 i-41 p-55 d-11 v-20 o-12 t-41 i-42 p-55 d-6 v-20 i-42 p-48 d-11 v-20 o-24 t-41 i-42 p-50 d-11 v-20 i-42 p-48 d-11 v-20 o-24 t-41 i-42 p-48 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-52 d-11 v-20 i-42 p-53 d-11 v-20 o-24 t-41 i-42 p-52 d-17 v-20 b-1 s-8 o-0 t-41 i-40 p-69 d-4 v-20 b-1 s-8 o-0 t-41 i-42 p-55 d-11 v-20 i-42 p-38 d-11 v-20 o-12 t-41 i-42 p-50 d-17 v-20 i-42 p-46 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-55 d-17 v-20 o-12 t-41 i-42 p-36 d-11 v-20 o-12 t-41 i-42 p-48 d-11 v-20 o-8 t-41 i-40 p-69 d-4 v-20 i-42 p-65 d-4 v-20 b-1 s-8 o-0 t-41 i-42 p-60 d-17 v-20 o-24 t-41 i-40 p-65 d-4 v-20 b-1 s-8 o-0 t-41 i-42 p-43 d-11 v-20 i-42 p-55 d-17 v-20 i-42 p-55 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-52 S2s1_17 S4_12_2 S4_13_2 S4_14_2 S4_15_2 S4_16_2 o-12 t-41 i-42 p-48 d-11 v-20 i-42 p-52 d-11 v-20 i-42 p-47 d-11 v-20 i-42 p-52 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-48 d-17 v-20 i-42 p-36 d-11 v-20 i-42 p-38 d-11 v-20 i-42 p-45 d-11 v-20 o-12 t-41 i-42 p-43 d-11 v-20 o-24 t-41 i-40 p-65 d-4 v-20 i-42 p-41 d-11 v-20 i-42 p-44 d-11 v-20 o-12 t-41 i-42 p-45 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-48 d-17 v-20 i-42 p-52 S2s1_17 S4_0_2 i-11 p-60 d-11 v-20 i-42 p-52 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-52 S2s1_17 d-17 v-20 i-42 p-55 d-11 v-20 i-42 p-37 d-4 v-20 i-42 p-52 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-48 d-11 v-20 i-42 p-50 d-17 v-20 i-42 p-55 d-17 v-20 i-42 p-52 d-17 v-20 i-41 p-55 d-11 v-20 i-42 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-52 d-17 v-20 i-42 p-36 d-11 v-20 i-42 p-53 d-6 v-20 i-42 p-52 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-48 d-11 v-20 i-42 p-57 d-17 v-20 b-1 s-8 o-0 t-41 i-42 p-43 d-17 v-20 i-42 p-50 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-41 d-11 v-20 i-42 p-60 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-48 d-11 v-20 o-32 t-41 i-42 p-50 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-41 d-11 v-20 i-42 p-55 d-11 v-20 i-42 p-57 d-11 v-20 i-42 p-42 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-50 d-11 v-20 i-42 p-38 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-42 d-32 v-20 i-42 p-52 d-11 v-20 i-42 p-55 d-17 v-20 i-42 p-43 d-11 v-20 o-24 t-41 i-42 p-43 d-11 v-20 i-42 p-62 d-11 v-20 i-42 p-60 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-47 d-11 v-20 b-1 s-8 o-0 t-41 i-42 p-50 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-53 d-11 v-20 i-42 p-64 d-17 v-20 i-42 p-38 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-50 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-48 d-17 v-20 i-42 p-48 d-11 v-20 i-42 p-52 d-11 v-20 i-42 p-38 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-48 d-11 v-20 i-42 p-38 d-11 v-20 i-42 p-36 d-11 v-20 i-42 p-45 d-11 v-20 i-42 p-42 d-17 v-20 i-42 p-65 d-11 v-20 i-42 p-62 d-11 v-20 i-42 p-52 d-11 v-20 i-42 p-52 d-17 v-20 i-42 p-48 d-11 v-20 i-42 p-60 d-11 v-20 i-42 p-52 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-48 d-17 v-20 i-42 p-62 d-4 v-20 i-42 p-38 d-11 v-20 i-42 p-37 d-15 v-20 i-42 p-50 d-11 v-20 i-42 p-36 d-17 v-20 i-42 p-57 d-17 v-20 i-42 p-52 d-17 v-20 i-42 p-50 d-11 v-20 i-42 p-37 d-11 v-20 i-42 p-47 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-45 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-37 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-45 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-62 d-11 v-20 i-42 p-48 d-17 v-20 i-42 p-55 d-17 v-20 i-42 p-48 d-11 v-20 i-42 p-43 d-11 v-20 i-42 p-48 d-11 v-20 i-42 p-50 d-11 v-20 i-42 p-62 d-11 v-20 i-42 p-36 d-11 v-20 i-42 p-55"
# model_result_str = "s-8 o-0 t-41 i-0 p-59 d-17 v-20 b-1 s-9 o-0 t-41 i-42 p-38 d-11 v-20 o-36 t-41 i-0 p-69 d-17 v-20 b-1 s-8 o-0 t-41 i-42 p-33 d-12 v-20 o-30 t-41 i-0 p-60 d-14 v-20 o-12 t-41 i-40 p-76 d-4 v-20 o-12 t-41 i-42 p-48 d-11 v-20 o-24 t-41 i-0 p-72 d-12 v-20 o-12 t-41 i-40 p-69 d-6 v-20 o-12 t-41 i-0 p-57 d-17 v-20 o-24 t-41 i-0 p-67 d-11 v-20 o-30 t-41 i-0 p-72 d-4 v-20 b-1 s-8 o-0 t-41 i-0 p-79 d-4 v-20 o-24 t-41 i-42 p-36 d-11 v-20 o-24 t-41 i-0 p-71 d-4 v-20 o-24 t-41 i-0 p-67 d-11 v-20 o-24 t-41 i-0 p-74 d-4 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-65 d-11 v-20 o-6 t-41 i-0 p-64 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-71 d-17 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 i-42 p-55 d-17 v-20 i-42 p-48 d-11 v-20 o-12 t-41 i-0 p-74 d-11 v-20 o-12 t-41 i-0 p-60 d-11 v-20 o-12 t-41 i-0 p-60 d-17 v-20 o-8 t-41 i-0 p-69 d-11 v-20 o-12 t-41 i-0 p-59 d-17 v-20 o-12 t-41 i-0 p-69 d-11 v-20 o-42 t-41 i-0 p-67 d-11 v-20 o-24 t-41 i-0 p-64 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-17 v-20 b-1 s-8 o-0 t-41 i-0 p-59 d-11 v-20 o-36 t-41 i-0 p-76 d-11 v-20 p-50 d-11 v-20 o-12 t-41 i-0 p-60 d-11 v-20 o-18 t-41 i-0 p-65 d-11 v-20 i-42 p-43 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-69 d-11 v-20 i-42 p-65 d-11 v-20 o-30 t-41 i-0 p-74 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-74 d-17 v-20 o-24 t-41 i-0 p-62 d-17 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 o-12 t-41 i-0 p-79 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-74 d-17 v-20 o-12 t-41 i-0 p-74 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-74 d-17 v-20 o-24 t-41 i-0 p-64 d-17 v-20 o-32 t-41 i-0 p-72 d-11 v-20 o-36 t-41 i-0 p-64 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-76 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 i-42 p-52 d-11 v-20 p-57 d-11 v-20 i-42 p-45 I4_28 C1_4 R1_2 R3_3 d-45 v-20 o-12 t-41 i-0 p-60 d-17 v-20 i-42 p-62 d-11 v-20 o-12 t-41 i-0 p-67 d-11 v-20 p-48 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-59 d-17 v-20 o-6 t-41 i-0 p-74 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-72 d-11 v-20 o-24 t-41 i-0 p-69 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-17 v-20 o-24 t-41"
# model_result_str = "s-9 o-0 t-31 i-24 p-61 d-27 v-20 o-48 t-31 i-24 p-60 d-12 v-20 o-72 t-31 i-24 p-58 d-9 v-20 o-81 t-31 i-24 p-58 d-9 v-20 o-90 t-31 i-24 p-58 d-12 v-20 o-102 t-31 i-24 p-58 d-12 v-20 o-189 t-31 i-24 p-51 d-3 v-20 o-192 t-31 i-24 p-61 d-27 v-20 o-240 t-31 i-24 p-60 d-12 v-20 o-264 t-31 i-24 p-58 d-4 v-20 o-268 t-31 i-24 p-58 d-4 v-20 o-272 t-31 i-24 p-58 d-8 v-20 o-280 t-31 i-24 p-58 d-8 v-20 o-288 t-31 i-24 p-58 d-3 v-20 o-291 t-31 i-24 p-56 d-3 v-20 o-294 t-31 i-24 p-55 d-12 v-20 o-381 t-31 i-24 p-51 d-3 v-20 o-384 t-31 i-24 p-61 d-27 v-20 o-432 t-31 i-24 p-60 d-18 v-20 o-456 t-31 i-24 p-58 d-9 v-20 o-465 t-31 i-24 p-58 d-9 v-20 o-474 t-31 i-24 p-58 d-12 v-20 o-486 t-31 i-24 p-58 d-15 v-20 o-573 t-31 i-24 p-51 d-3 v-20 o-576 t-31 i-24 p-61 d-27 v-20 o-624 t-31 i-24 p-60 d-18 v-20 o-648 t-31 i-24 p-58 d-4 v-20 o-652 t-31 i-24 p-58 d-4 v-20 o-656 t-31 i-24 p-58 d-8 v-20 o-664 t-31 i-24 p-63 d-13 v-20 o-678 t-31 i-24 p-63 d-15 v-20 o-0 t-31 i-16 p-73 d-27 v-17 o-0 t-31 i-16 p-77 d-27 v-17 o-0 t-31 i-16 p-68 d-27 v-17 o-0 t-31 i-16 p-75 d-27 v-17 o-0 t-31 i-16 p-44 d-27 v-17 o-48 t-31 i-16 p-56 d-27 v-17 o-48 t-31 i-16 p-60 d-27 v-17 o-48 t-31 i-16 p-63 d-27 v-17 o-48 t-31 i-16 p-32 d-27 v-17 o-96 t-31 i-16 p-51 d-33 v-17 o-96 t-31 i-16 p-55 d-33 v-17 o-96 t-31 i-16 p-58 d-33 v-17 o-96 t-31 i-16 p-27 d-33 v-17 o-168 t-31 i-16 p-63 d-18 v-17 o-168 t-31 i-16 p-67 d-18 v-17 o-168 t-31 i-16 p-58 d-18 v-17 o-168 t-31 i-16 p-34 d-18 v-17 o-192 t-31 i-16 p-73 d-27 v-17 o-192 t-31 i-16 p-77 d-27 v-17 o-192 t-31 i-16 p-68 d-27 v-17 o-192 t-31 i-16 p-75 d-27 v-17 o-192 t-31 i-16 p-44 d-27 v-17 o-240 t-31 i-16 p-68 d-27 v-17 o-240 t-31 i-16 p-60 d-27 v-17 o-240 t-31 i-16 p-63 d-27 v-17 o-240 t-31 i-16 p-67 d-27 v-17 o-240 t-31 i-16 p-36 d-27 v-17 o-288 t-31 i-16 p-51 d-33 v-17 o-288 t-31 i-16 p-55 d-33 v-17 o-288 t-31 i-16 p-58 d-33 v-17 o-288 t-31 i-16 p-27 d-33 v-17 o-360 t-31 i-16 p-63 d-18 v-17 o-360 t-31 i-16 p-67 d-18 v-17 o-360 t-31 i-16 p-58 d-18 v-17 o-360 t-31 i-16 p-34 d-18 v-17 o-384 t-31 i-16 p-73 d-27 v-17 o-384 t-31 i-16 p-77 d-27 v-17 o-384 t-31 i-16 p-68 d-27 v-17 o-384 t-31 i-16 p-75 d-27 v-17 o-384 t-31 i-16 p-44 d-27 v-17 o-432 t-31 i-16 p-56 d-27 v-17 o-432 t-31 i-16 p-60 d-27 v-17 o-432 t-31 i-16 p-63 d-27 v-17 o-432 t-31 i-16 p-32 d-27 v-17 o-480 t-31 i-16 p-51 d-33 v-17 o-480 t-31 i-16 p-55 d-33 v-17 o-480 t-31 i-16 p-58 d-33 v-17 o-480 t-31 i-16 p-27 d-33 v-17 o-552 t-31 i-16 p-63 d-18 v-17 o-552 t-31 i-16 p-67 d-18 v-17 o-552 t-31 i-16 p-58 d-18 v-17 o-552 t-31 i-16 p-34 d-18 v-17 o-576 t-31 i-16 p-73 d-27 v-17 o-576 t-31 i-16 p-77 d-27 v-17 o-576 t-31 i-16 p-68 d-27 v-17 o-576 t-31 i-16 p-75 d-27 v-17 o-576 t-31 i-16 p-44 d-27 v-17 o-624 t-31 i-16 p-68 d-27 v-17 o-624 t-31 i-16 p-60 d-27 v-17 o-624 t-31 i-16 p-63 d-27 v-17 o-624 t-31 i-16 p-67 d-27 v-17 o-624 t-31 i-16 p-36 d-27 v-17 o-672 t-31 i-16 p-51 d-27 v-17 o-672 t-31 i-16 p-55 d-27 v-17 o-672 t-31 i-16 p-58 d-27 v-17 o-672 t-31 i-16 p-27 d-27 v-17 o-720 t-31 i-16 p-58 d-27 v-17 o-720 t-31 i-16 p-62 d-27 v-17 o-720 t-31 i-16 p-63 d-27 v-17 o-720 t-31 i-16 p-65 d-27 v-17 o-720 t-31 i-16 p-34 d-27 v-17"
# model_result_str = "s-9 o-0 t-28 i-40 p-69 d-6 v-20 o-6 t-28 i-40 p-69 d-6 v-20 o-12 t-28 i-40 p-67 d-6 v-20 o-21 t-28 i-40 p-62 d-3 v-20 o-24 t-28 i-40 p-69 d-6 v-20 o-30 t-28 i-40 p-69 d-3 v-20 o-33 t-28 i-40 p-67 d-3 v-20 o-36 t-28 i-40 p-67 d-3 v-20 o-39 t-28 i-40 p-62 d-3 v-20 o-42 t-28 i-40 p-62 d-3 v-20 o-45 t-28 i-40 p-62 d-3 v-20 o-48 t-28 i-40 p-69 d-6 v-20 o-54 t-28 i-40 p-69 d-6 v-20 o-60 t-28 i-40 p-67 d-6 v-20 o-72 t-28 i-40 p-69 d-6 v-20 o-78 t-28 i-40 p-69 d-6 v-20 o-84 t-28 i-40 p-67 d-3 v-20 o-87 t-28 i-40 p-62 d-3 v-20 o-90 t-28 i-40 p-62 d-6 v-20 o-96 t-28 i-40 p-60 d-3 v-20 o-99 t-28 i-40 p-60 d-3 v-20 o-102 t-28 i-40 p-59 d-15 v-20 o-126 t-28 i-40 p-67 d-3 v-20 o-129 t-28 i-40 p-67 d-3 v-20 o-132 t-28 i-40 p-67 d-3 v-20 o-135 t-28 i-40 p-62 d-3 v-20 o-138 t-28 i-40 p-62 d-9 v-20 o-147 t-28 i-40 p-60 d-3 v-20 o-150 t-28 i-40 p-60 d-3 v-20 o-153 t-28 i-40 p-59 d-16 v-20 o-183 t-28 i-40 p-62 d-3 v-20 o-186 t-28 i-40 p-62 d-6 v-20 o-192 t-28 i-40 p-69 d-6 v-20 o-198 t-28 i-40 p-69 d-6 v-20 o-204 t-28 i-40 p-67 d-6 v-20 o-213 t-28 i-40 p-62 d-3 v-20 o-216 t-28 i-40 p-69 d-6 v-20 o-222 t-28 i-40 p-69 d-3 v-20 o-225 t-28 i-40 p-67 d-3 v-20 o-228 t-28 i-40 p-67 d-3 v-20 o-231 t-28 i-40 p-62 d-3 v-20 o-234 t-28 i-40 p-62 d-3 v-20 o-237 t-28 i-40 p-62 d-3 v-20 o-240 t-28 i-40 p-69 d-6 v-20 o-246 t-28 i-40 p-69 d-6 v-20 o-252 t-28 i-40 p-67 d-6 v-20 o-261 t-28 i-40 p-62 d-3 v-20 o-264 t-28 i-40 p-69 d-6 v-20 o-270 t-28 i-40 p-69 d-6 v-20 o-276 t-28 i-40 p-67 d-3 v-20 o-279 t-28 i-40 p-62 d-3 v-20 o-282 t-28 i-40 p-62 d-9 v-20 o-291 t-28 i-40 p-60 d-3 v-20 o-294 t-28 i-40 p-60 d-3 v-20 o-297 t-28 i-40 p-59 d-12 v-20 o-318 t-28 i-40 p-67 d-3 v-20 o-321 t-28 i-40 p-67 d-3 v-20 o-324 t-28 i-40 p-67 d-3 v-20 o-327 t-28 i-40 p-62 d-3 v-20 o-330 t-28 i-40 p-62 d-9 v-20 o-339 t-28 i-40 p-60 d-3 v-20 o-342 t-28 i-40 p-60 d-3 v-20 o-345 t-28 i-40 p-59 d-15 v-20 o-375 t-28 i-40 p-67 d-3 v-20 o-378 t-28 i-40 p-69 d-3 v-20 o-381 t-28 i-40 p-67 d-3 v-20 o-384 t-28 i-40 p-72 d-6 v-20 o-390 t-28 i-40 p-72 d-6 v-20 o-396 t-28 i-40 p-71 d-6 v-20 o-405 t-28 i-40 p-67 d-3 v-20 o-408 t-28 i-40 p-72 d-6 v-20 o-414 t-28 i-40 p-72 d-3 v-20 o-417 t-28 i-40 p-71 d-3 v-20 o-420 t-28 i-40 p-71 d-3 v-20 o-423 t-28 i-40 p-67 d-3 v-20 o-426 t-28 i-40 p-67 d-3 v-20 o-429 t-28 i-40 p-67 d-3 v-20 o-432 t-28 i-40 p-72 d-6 v-20 o-438 t-28 i-40 p-72 d-6 v-20 o-444 t-28 i-40 p-71 d-6 v-20 o-456 t-28 i-40 p-72 d-6 v-20 o-462 t-28 i-40 p-72 d-6 v-20 o-468 t-28 i-40 p-71 d-3 v-20 o-471 t-28 i-40 p-67 d-3 v-20 o-474 t-28 i-40 p-67 d-6 v-20 o-480 t-28 i-40 p-64 d-3 v-20 o-483 t-28 i-40 p-64 d-3 v-20 o-486 t-28 i-40 p-62 d-15 v-20 o-510 t-28 i-40 p-72 d-3 v-20 o-513 t-28 i-40 p-72 d-3 v-20 o-516 t-28 i-40 p-72 d-3 v-20 o-519 t-28 i-40 p-71 d-3 v-20 o-522 t-28 i-40 p-71 d-9 v-20 o-531 t-28 i-40 p-67 d-3 v-20 o-534 t-28 i-40 p-64 d-3 v-20 o-537 t-28 i-40 p-62 d-18 v-20 o-567 t-28 i-40 p-67 d-3 v-20 o-570 t-28 i-40 p-67 d-6 v-20 o-576 t-28 i-40 p-72 d-6 v-20 o-582 t-28 i-40 p-72 d-6 v-20 o-588 t-28 i-40 p-71 d-6 v-20 o-597 t-28 i-40 p-67 d-3 v-20 o-600 t-28 i-40 p-72 d-6 v-20 o-606 t-28 i-40 p-72 d-3 v-20 o-609 t-28 i-40 p-71 d-3 v-20 o-612 t-28 i-40 p-71 d-3 v-20 o-615 t-28 i-40 p-67 d-3 v-20 o-618 t-28 i-40 p-67 d-3 v-20 o-621 t-28 i-40 p-67 d-3 v-20 o-624 t-28 i-40 p-72 d-6 v-20 o-630 t-28 i-40 p-72 d-6 v-20 o-636 t-28 i-40 p-71 d-6 v-20 o-645 t-28 i-40 p-67 d-3 v-20 o-648 t-28 i-40 p-72 d-6 v-20 o-654 t-28 i-40 p-72 d-6 v-20 o-660 t-28 i-40 p-71 d-3 v-20 o-663 t-28 i-40 p-67 d-3 v-20 o-666 t-28 i-40 p-67 d-9 v-20 o-675 t-28 i-40 p-64 d-3 v-20 o-678 t-28 i-40 p-64 d-3 v-20 o-681 t-28 i-40 p-62 d-13 v-20 o-702 t-28 i-40 p-72 d-3 v-20 o-705 t-28 i-40 p-72 d-3 v-20 o-708 t-28 i-40 p-72 d-3 v-20 o-711 t-28 i-40 p-71 d-3 v-20 o-714 t-28 i-40 p-71 d-9 v-20 o-723 t-28 i-40 p-67 d-3 v-20 o-726 t-28 i-40 p-64 d-3 v-20 o-729 t-28 i-40 p-62 d-18 v-20 o-0 t-28 i-4 p-55 d-18 v-17 o-0 t-28 i-4 p-59 d-18 v-17 o-0 t-28 i-4 p-62 d-18 v-17 o-0 t-28 i-4 p-69 d-18 v-17 o-0 t-28 i-4 p-31 d-18 v-17 o-24 t-28 i-4 p-62 d-18 v-17 o-24 t-28 i-4 p-66 d-18 v-17 o-24 t-28 i-4 p-69 d-18 v-17 o-24 t-28 i-4 p-71 d-18 v-17 o-24 t-28 i-4 p-38 d-18 v-17 o-48 t-28 i-4 p-60 d-27 v-17 o-48 t-28 i-4 p-64 d-27 v-17 o-48 t-28 i-4 p-67 d-27 v-17 o-48 t-28 i-4 p-74 d-27 v-17 o-48 t-28 i-4 p-36 d-27 v-17 o-96 t-28 i-4 p-55 d-18 v-17 o-96 t-28 i-4 p-59 d-18 v-17 o-96 t-28 i-4 p-62 d-18 v-17 o-96 t-28 i-4 p-69 d-18 v-17 o-96 t-28 i-4 p-31 d-18 v-17 o-120 t-28 i-4 p-62 d-18 v-17 o-120 t-28 i-4 p-66 d-18 v-17 o-120 t-28 i-4 p-69 d-18 v-17 o-120 t-28 i-4 p-71 d-18 v-17 o-120 t-28 i-4 p-38 d-18 v-17 o-144 t-28 i-4 p-60 d-27 v-17 o-144 t-28 i-4 p-64 d-27 v-17 o-144 t-28 i-4 p-67 d-27 v-17 o-144 t-28 i-4 p-74 d-27 v-17 o-144 t-28 i-4 p-36 d-27 v-17 o-192 t-28 i-4 p-55 d-18 v-17 o-192 t-28 i-4 p-59 d-18 v-17 o-192 t-28 i-4 p-62 d-18 v-17 o-192 t-28 i-4 p-69 d-18 v-17 o-192 t-28 i-4 p-31 d-18 v-17 o-216 t-28 i-4 p-62 d-18 v-17 o-216 t-28 i-4 p-66 d-18 v-17 o-216 t-28 i-4 p-69 d-18 v-17 o-216 t-28 i-4 p-71 d-18 v-17 o-216 t-28 i-4 p-38 d-18 v-17 o-240 t-28 i-4 p-60 d-27 v-17 o-240 t-28 i-4 p-64 d-27 v-17 o-240 t-28 i-4 p-67 d-27 v-17 o-240 t-28 i-4 p-74 d-27 v-17 o-240 t-28 i-4 p-36 d-27 v-17 o-288 t-28 i-4 p-55 d-18 v-17 o-288 t-28 i-4 p-59 d-18 v-17 o-288 t-28 i-4 p-62 d-18 v-17 o-288 t-28 i-4 p-69 d-18 v-17 o-288 t-28 i-4 p-31 d-18 v-17 o-312 t-28 i-4 p-62 d-18 v-17 o-312 t-28 i-4 p-66 d-18 v-17 o-312 t-28 i-4 p-69 d-18 v-17 o-312 t-28 i-4 p-71 d-18 v-17 o-312 t-28 i-4 p-38 d-18 v-17 o-336 t-28 i-4 p-60 d-27 v-17 o-336 t-28 i-4 p-64 d-27 v-17 o-336 t-28 i-4 p-67 d-27 v-17 o-336 t-28 i-4 p-74 d-27 v-17 o-336 t-28 i-4 p-36 d-27 v-17 o-384 t-28 i-4 p-55 d-18 v-17 o-384 t-28 i-4 p-59 d-18 v-17 o-384 t-28 i-4 p-62 d-18 v-17 o-384 t-28 i-4 p-69 d-18 v-17 o-384 t-28 i-4 p-31 d-18 v-17 o-408 t-28 i-4 p-62 d-18 v-17 o-408 t-28 i-4 p-66 d-18 v-17 o-408 t-28 i-4 p-69 d-18 v-17 o-408 t-28 i-4 p-71 d-18 v-17 o-408 t-28 i-4 p-38 d-18 v-17 o-432 t-28 i-4 p-60 d-27 v-17 o-432 t-28 i-4 p-64 d-27 v-17 o-432 t-28 i-4 p-67 d-27 v-17 o-432 t-28 i-4 p-74 d-27 v-17 o-432 t-28 i-4 p-36 d-27 v-17 o-480 t-28 i-4 p-55 d-18 v-17 o-480 t-28 i-4 p-59 d-18 v-17 o-480 t-28 i-4 p-62 d-18 v-17 o-480 t-28 i-4 p-69 d-18 v-17 o-480 t-28 i-4 p-31 d-18 v-17 o-504 t-28 i-4 p-62 d-18 v-17 o-504 t-28 i-4 p-66 d-18 v-17 o-504 t-28 i-4 p-69 d-18 v-17 o-504 t-28 i-4 p-71 d-18 v-17 o-504 t-28 i-4 p-38 d-18 v-17 o-528 t-28 i-4 p-60 d-27 v-17 o-528 t-28 i-4 p-64 d-27 v-17 o-528 t-28 i-4 p-67 d-27 v-17 o-528 t-28 i-4 p-74 d-27 v-17 o-528 t-28 i-4 p-36 d-27 v-17 o-576 t-28 i-4 p-55 d-18 v-17 o-576 t-28 i-4 p-59 d-18 v-17 o-576 t-28 i-4 p-62 d-18 v-17 o-576 t-28 i-4 p-69 d-18 v-17 o-576 t-28 i-4 p-31 d-18 v-17 o-600 t-28 i-4 p-62 d-18 v-17 o-600 t-28 i-4 p-66 d-18 v-17 o-600 t-28 i-4 p-69 d-18 v-17 o-600 t-28 i-4 p-71 d-18 v-17 o-600 t-28 i-4 p-38 d-18 v-17 o-624 t-28 i-4 p-60 d-27 v-17 o-624 t-28 i-4 p-64 d-27 v-17 o-624 t-28 i-4 p-67 d-27 v-17 o-624 t-28 i-4 p-74 d-27 v-17 o-624 t-28 i-4 p-36 d-27 v-17 o-672 t-28 i-4 p-55 d-18 v-17 o-672 t-28 i-4 p-59 d-18 v-17 o-672 t-28 i-4 p-62 d-18 v-17 o-672 t-28 i-4 p-69 d-18 v-17 o-672 t-28 i-4 p-31 d-18 v-17 o-696 t-28 i-4 p-62 d-18 v-17 o-696 t-28 i-4 p-66 d-18 v-17 o-696 t-28 i-4 p-69 d-18 v-17 o-696 t-28 i-4 p-71 d-18 v-17 o-696 t-28 i-4 p-38 d-18 v-17 o-720 t-28 i-4 p-60 d-27 v-17 o-720 t-28 i-4 p-64 d-27 v-17 o-720 t-28 i-4 p-67 d-27 v-17 o-720 t-28 i-4 p-74 d-27 v-17 o-720 t-28 i-4 p-36 d-27 v-17"
# model_result_str = "s-9 o-6 t-30 i-27 p-65 d-6 v-20 o-12 t-30 i-27 p-65 d-6 v-20 o-18 t-30 i-27 p-65 d-6 v-20 o-24 t-30 i-27 p-65 d-6 v-20 o-30 t-30 i-27 p-67 d-6 v-20 o-36 t-30 i-27 p-70 d-6 v-20 o-42 t-30 i-27 p-70 d-12 v-20 o-54 t-30 i-27 p-67 d-6 v-20 o-60 t-30 i-27 p-65 d-6 v-20 o-66 t-30 i-27 p-70 d-12 v-20 o-78 t-30 i-27 p-67 d-6 v-20 o-90 t-30 i-27 p-70 d-6 v-20 o-96 t-30 i-27 p-70 d-6 v-20 o-102 t-30 i-27 p-67 d-6 v-20 o-108 t-30 i-27 p-65 d-6 v-20 o-114 t-30 i-27 p-70 d-12 v-20 o-126 t-30 i-27 p-67 d-6 v-20 o-132 t-30 i-27 p-65 d-6 v-20 o-138 t-30 i-27 p-70 d-12 v-20 o-150 t-30 i-27 p-67 d-6 v-20 o-156 t-30 i-27 p-65 d-6 v-20 o-162 t-30 i-27 p-70 d-12 v-20 o-174 t-30 i-27 p-67 d-6 v-20 o-186 t-30 i-27 p-65 d-6 v-20 o-192 t-30 i-27 p-70 d-6 v-20 o-198 t-30 i-27 p-72 d-6 v-20 o-204 t-30 i-27 p-74 d-6 v-20 o-210 t-30 i-27 p-72 d-12 v-20 o-222 t-30 i-27 p-67 d-6 v-20 o-228 t-30 i-27 p-65 d-6 v-20 o-234 t-30 i-27 p-65 d-12 v-20 o-246 t-30 i-27 p-67 d-3 v-20 o-249 t-30 i-27 p-70 d-9 v-20 o-258 t-30 i-27 p-70 d-12 v-20 o-270 t-30 i-27 p-70 d-6 v-20 o-282 t-30 i-27 p-70 d-6 v-20 o-288 t-30 i-27 p-70 d-6 v-20 o-294 t-30 i-27 p-67 d-6 v-20 o-300 t-30 i-27 p-65 d-6 v-20 o-306 t-30 i-27 p-70 d-12 v-20 o-318 t-30 i-27 p-67 d-6 v-20 o-324 t-30 i-27 p-65 d-6 v-20 o-330 t-30 i-27 p-70 d-12 v-20 o-342 t-30 i-27 p-67 d-6 v-20 o-348 t-30 i-27 p-65 d-6 v-20 o-354 t-30 i-27 p-70 d-12 v-20 o-366 t-30 i-27 p-67 d-6 v-20 o-378 t-30 i-27 p-65 d-6 v-20 o-384 t-30 i-27 p-70 d-6 v-20 o-390 t-30 i-27 p-72 d-6 v-20 o-396 t-30 i-27 p-74 d-6 v-20 o-402 t-30 i-27 p-72 d-12 v-20 o-414 t-30 i-27 p-67 d-6 v-20 o-420 t-30 i-27 p-65 d-6 v-20 o-426 t-30 i-27 p-65 d-12 v-20 o-438 t-30 i-27 p-67 d-3 v-20 o-441 t-30 i-27 p-70 d-9 v-20 o-450 t-30 i-27 p-70 d-6 v-20 o-456 t-30 i-27 p-70 d-6 v-20 o-462 t-30 i-27 p-70 d-6 v-20 o-468 t-30 i-27 p-70 d-3 v-20 o-471 t-30 i-27 p-67 d-3 v-20 o-474 t-30 i-27 p-65 d-6 v-20 o-486 t-30 i-27 p-65 d-3 v-20 o-489 t-30 i-27 p-67 d-3 v-20 o-492 t-30 i-27 p-70 d-6 v-20 o-498 t-30 i-27 p-70 d-6 v-20 o-504 t-30 i-27 p-70 d-6 v-20 o-510 t-30 i-27 p-70 d-6 v-20 o-516 t-30 i-27 p-70 d-3 v-20 o-519 t-30 i-27 p-67 d-3 v-20 o-522 t-30 i-27 p-65 d-6 v-20 o-534 t-30 i-27 p-65 d-3 v-20 o-537 t-30 i-27 p-67 d-3 v-20 o-540 t-30 i-27 p-70 d-6 v-20 o-546 t-30 i-27 p-70 d-6 v-20 o-552 t-30 i-27 p-70 d-6 v-20 o-558 t-30 i-27 p-70 d-6 v-20 o-564 t-30 i-27 p-70 d-3 v-20 o-567 t-30 i-27 p-67 d-3 v-20 o-570 t-30 i-27 p-65 d-6 v-20 o-576 t-30 i-27 p-70 d-6 v-20 o-582 t-30 i-27 p-72 d-6 v-20 o-588 t-30 i-27 p-74 d-6 v-20 o-594 t-30 i-27 p-72 d-12 v-20 o-606 t-30 i-27 p-67 d-6 v-20 o-612 t-30 i-27 p-65 d-6 v-20 o-618 t-30 i-27 p-65 d-12 v-20 o-630 t-30 i-27 p-67 d-3 v-20 o-633 t-30 i-27 p-70 d-9 v-20 o-0 t-30 i-41 p-67 d-18 v-17 o-0 t-30 i-41 p-70 d-18 v-17 o-0 t-30 i-41 p-74 d-18 v-17 o-0 t-30 i-41 p-43 d-18 v-17 o-24 t-30 i-41 p-63 d-12 v-17 o-24 t-30 i-41 p-67 d-12 v-17 o-24 t-30 i-41 p-70 d-12 v-17 o-24 t-30 i-41 p-39 d-12 v-17 o-48 t-30 i-41 p-58 d-27 v-17 o-48 t-30 i-41 p-62 d-27 v-17 o-48 t-30 i-41 p-65 d-27 v-17 o-48 t-30 i-41 p-34 d-27 v-17 o-96 t-30 i-41 p-58 d-18 v-17 o-96 t-30 i-41 p-62 d-18 v-17 o-96 t-30 i-41 p-65 d-18 v-17 o-96 t-30 i-41 p-34 d-18 v-17 o-120 t-30 i-41 p-60 d-18 v-17 o-120 t-30 i-41 p-63 d-18 v-17 o-120 t-30 i-41 p-67 d-18 v-17 o-120 t-30 i-41 p-36 d-18 v-17 o-144 t-30 i-41 p-67 d-27 v-17 o-144 t-30 i-41 p-70 d-27 v-17 o-144 t-30 i-41 p-74 d-27 v-17 o-144 t-30 i-41 p-43 d-27 v-17 o-192 t-30 i-41 p-67 d-18 v-17 o-192 t-30 i-41 p-70 d-18 v-17 o-192 t-30 i-41 p-74 d-18 v-17 o-192 t-30 i-41 p-43 d-18 v-17 o-216 t-30 i-41 p-63 d-18 v-17 o-216 t-30 i-41 p-67 d-18 v-17 o-216 t-30 i-41 p-70 d-18 v-17 o-216 t-30 i-41 p-39 d-18 v-17 o-240 t-30 i-41 p-58 d-27 v-17 o-240 t-30 i-41 p-62 d-27 v-17 o-240 t-30 i-41 p-65 d-27 v-17 o-240 t-30 i-41 p-34 d-27 v-17 o-288 t-30 i-41 p-58 d-18 v-17 o-288 t-30 i-41 p-62 d-18 v-17 o-288 t-30 i-41 p-65 d-18 v-17 o-288 t-30 i-41 p-34 d-18 v-17 o-312 t-30 i-41 p-60 d-18 v-17 o-312 t-30 i-41 p-63 d-18 v-17 o-312 t-30 i-41 p-67 d-18 v-17 o-312 t-30 i-41 p-36 d-18 v-17 o-336 t-30 i-41 p-67 d-27 v-17 o-336 t-30 i-41 p-70 d-27 v-17 o-336 t-30 i-41 p-74 d-27 v-17 o-336 t-30 i-41 p-43 d-27 v-17 o-384 t-30 i-41 p-67 d-18 v-17 o-384 t-30 i-41 p-70 d-18 v-17 o-384 t-30 i-41 p-74 d-18 v-17 o-384 t-30 i-41 p-43 d-18 v-17 o-408 t-30 i-41 p-63 d-18 v-17 o-408 t-30 i-41 p-67 d-18 v-17 o-408 t-30 i-41 p-70 d-18 v-17 o-408 t-30 i-41 p-39 d-18 v-17 o-432 t-30 i-41 p-58 d-27 v-17 o-432 t-30 i-41 p-62 d-27 v-17 o-432 t-30 i-41 p-65 d-27 v-17 o-432 t-30 i-41 p-34 d-27 v-17 o-480 t-30 i-41 p-58 d-18 v-17 o-480 t-30 i-41 p-62 d-18 v-17 o-480 t-30 i-41 p-65 d-18 v-17 o-480 t-30 i-41 p-34 d-18 v-17 o-504 t-30 i-41 p-60 d-18 v-17 o-504 t-30 i-41 p-63 d-18 v-17 o-504 t-30 i-41 p-67 d-18 v-17 o-504 t-30 i-41 p-36 d-18 v-17 o-528 t-30 i-41 p-67 d-27 v-17 o-528 t-30 i-41 p-70 d-27 v-17 o-528 t-30 i-41 p-74 d-27 v-17 o-528 t-30 i-41 p-43 d-27 v-17 o-576 t-30 i-41 p-67 d-18 v-17 o-576 t-30 i-41 p-70 d-18 v-17 o-576 t-30 i-41 p-74 d-18 v-17 o-576 t-30 i-41 p-43 d-18 v-17 o-600 t-30 i-41 p-63 d-18 v-17 o-600 t-30 i-41 p-67 d-18 v-17 o-600 t-30 i-41 p-70 d-18 v-17 o-600 t-30 i-41 p-39 d-18 v-17 o-624 t-30 i-41 p-58 d-15 v-17 o-624 t-30 i-41 p-62 d-15 v-17 o-624 t-30 i-41 p-65 d-15 v-17 o-624 t-30 i-41 p-34 d-15 v-17 o-642 t-30 i-41 p-58 d-21 v-17 o-642 t-30 i-41 p-62 d-21 v-17 o-642 t-30 i-41 p-65 d-21 v-17 o-642 t-30 i-41 p-72 d-21 v-17 o-642 t-30 i-41 p-34 d-21 v-17"
# model_result_str = "s-9 o-6 t-30 i-27 p-65 d-6 v-20 o-12 t-30 i-27 p-65 d-6 v-20 o-18 t-30 i-27 p-65 d-6 v-20 o-24 t-30 i-27 p-65 d-6 v-20 o-30 t-30 i-27 p-67 d-6 v-20 o-36 t-30 i-27 p-70 d-6 v-20 o-42 t-30 i-27 p-70 d-12 v-20 o-54 t-30 i-27 p-67 d-6 v-20 o-60 t-30 i-27 p-65 d-6 v-20 o-66 t-30 i-27 p-70 d-12 v-20 o-78 t-30 i-27 p-67 d-6 v-20 o-90 t-30 i-27 p-70 d-6 v-20 o-96 t-30 i-27 p-70 d-6 v-20 o-102 t-30 i-27 p-67 d-6 v-20 o-108 t-30 i-27 p-65 d-6 v-20 o-114 t-30 i-27 p-70 d-12 v-20 o-126 t-30 i-27 p-67 d-6 v-20 o-132 t-30 i-27 p-65 d-6 v-20 o-138 t-30 i-27 p-70 d-12 v-20 o-150 t-30 i-27 p-67 d-6 v-20 o-156 t-30 i-27 p-65 d-6 v-20 o-162 t-30 i-27 p-70 d-12 v-20 o-174 t-30 i-27 p-67 d-6 v-20 o-186 t-30 i-27 p-65 d-6 v-20 o-192 t-30 i-27 p-70 d-6 v-20 o-198 t-30 i-27 p-72 d-6 v-20 o-204 t-30 i-27 p-74 d-6 v-20 o-210 t-30 i-27 p-72 d-12 v-20 o-222 t-30 i-27 p-67 d-6 v-20 o-228 t-30 i-27 p-65 d-6 v-20 o-234 t-30 i-27 p-65 d-12 v-20 o-246 t-30 i-27 p-67 d-3 v-20 o-249 t-30 i-27 p-70 d-9 v-20 o-258 t-30 i-27 p-70 d-12 v-20 o-270 t-30 i-27 p-70 d-6 v-20 o-282 t-30 i-27 p-70 d-6 v-20 o-288 t-30 i-27 p-70 d-6 v-20 o-294 t-30 i-27 p-67 d-6 v-20 o-300 t-30 i-27 p-65 d-6 v-20 o-306 t-30 i-27 p-70 d-12 v-20 o-318 t-30 i-27 p-67 d-6 v-20 o-324 t-30 i-27 p-65 d-6 v-20 o-330 t-30 i-27 p-70 d-12 v-20 o-342 t-30 i-27 p-67 d-6 v-20 o-348 t-30 i-27 p-65 d-6 v-20 o-354 t-30 i-27 p-70 d-12 v-20 o-366 t-30 i-27 p-67 d-6 v-20 o-378 t-30 i-27 p-65 d-6 v-20 o-384 t-30 i-27 p-70 d-6 v-20 o-390 t-30 i-27 p-72 d-6 v-20 o-396 t-30 i-27 p-74 d-6 v-20 o-402 t-30 i-27 p-72 d-12 v-20 o-414 t-30 i-27 p-67 d-6 v-20 o-420 t-30 i-27 p-65 d-6 v-20 o-426 t-30 i-27 p-65 d-12 v-20 o-438 t-30 i-27 p-67 d-3 v-20 o-441 t-30 i-27 p-70 d-9 v-20 o-450 t-30 i-27 p-70 d-6 v-20 o-456 t-30 i-27 p-70 d-6 v-20 o-462 t-30 i-27 p-70 d-6 v-20 o-468 t-30 i-27 p-70 d-3 v-20 o-471 t-30 i-27 p-67 d-3 v-20 o-474 t-30 i-27 p-65 d-6 v-20 o-486 t-30 i-27 p-65 d-3 v-20 o-489 t-30 i-27 p-67 d-3 v-20 o-492 t-30 i-27 p-70 d-6 v-20 o-498 t-30 i-27 p-70 d-6 v-20 o-504 t-30 i-27 p-70 d-6 v-20 o-510 t-30 i-27 p-70 d-6 v-20 o-516 t-30 i-27 p-70 d-3 v-20 o-519 t-30 i-27 p-67 d-3 v-20 o-522 t-30 i-27 p-65 d-6 v-20 o-534 t-30 i-27 p-65 d-3 v-20 o-537 t-30 i-27 p-67 d-3 v-20 o-540 t-30 i-27 p-70 d-6 v-20 o-546 t-30 i-27 p-70 d-6 v-20 o-552 t-30 i-27 p-70 d-6 v-20 o-558 t-30 i-27 p-70 d-6 v-20 o-564 t-30 i-27 p-70 d-3 v-20 o-567 t-30 i-27 p-67 d-3 v-20 o-570 t-30 i-27 p-65 d-6 v-20 o-576 t-30 i-27 p-70 d-6 v-20 o-582 t-30 i-27 p-72 d-6 v-20 o-588 t-30 i-27 p-74 d-6 v-20 o-594 t-30 i-27 p-72 d-12 v-20 o-606 t-30 i-27 p-67 d-6 v-20 o-612 t-30 i-27 p-65 d-6 v-20 o-618 t-30 i-27 p-65 d-12 v-20 o-630 t-30 i-27 p-67 d-3 v-20 o-633 t-30 i-27 p-70 d-9 v-20 o-0 t-30 i-41 p-67 d-18 v-17 o-0 t-30 i-41 p-70 d-18 v-17 o-0 t-30 i-41 p-74 d-18 v-17 o-0 t-30 i-41 p-43 d-18 v-17 o-24 t-30 i-41 p-63 d-12 v-17 o-24 t-30 i-41 p-67 d-12 v-17 o-24 t-30 i-41 p-70 d-12 v-17 o-24 t-30 i-41 p-39 d-12 v-17 o-48 t-30 i-41 p-58 d-27 v-17 o-48 t-30 i-41 p-62 d-27 v-17 o-48 t-30 i-41 p-65 d-27 v-17 o-48 t-30 i-41 p-34 d-27 v-17 o-96 t-30 i-41 p-58 d-18 v-17 o-96 t-30 i-41 p-62 d-18 v-17 o-96 t-30 i-41 p-65 d-18 v-17 o-96 t-30 i-41 p-34 d-18 v-17 o-120 t-30 i-41 p-60 d-18 v-17 o-120 t-30 i-41 p-63 d-18 v-17 o-120 t-30 i-41 p-67 d-18 v-17 o-120 t-30 i-41 p-36 d-18 v-17 o-144 t-30 i-41 p-67 d-27 v-17 o-144 t-30 i-41 p-70 d-27 v-17 o-144 t-30 i-41 p-74 d-27 v-17 o-144 t-30 i-41 p-43 d-27 v-17 o-192 t-30 i-41 p-67 d-18 v-17 o-192 t-30 i-41 p-70 d-18 v-17 o-192 t-30 i-41 p-74 d-18 v-17 o-192 t-30 i-41 p-43 d-18 v-17 o-216 t-30 i-41 p-63 d-18 v-17 o-216 t-30 i-41 p-67 d-18 v-17 o-216 t-30 i-41 p-70 d-18 v-17 o-216 t-30 i-41 p-39 d-18 v-17 o-240 t-30 i-41 p-58 d-27 v-17 o-240 t-30 i-41 p-62 d-27 v-17 o-240 t-30 i-41 p-65 d-27 v-17 o-240 t-30 i-41 p-34 d-27 v-17 o-288 t-30 i-41 p-58 d-18 v-17 o-288 t-30 i-41 p-62 d-18 v-17 o-288 t-30 i-41 p-65 d-18 v-17 o-288 t-30 i-41 p-34 d-18 v-17 o-312 t-30 i-41 p-60 d-18 v-17 o-312 t-30 i-41 p-63 d-18 v-17 o-312 t-30 i-41 p-67 d-18 v-17 o-312 t-30 i-41 p-36 d-18 v-17 o-336 t-30 i-41 p-67 d-27 v-17 o-336 t-30 i-41 p-70 d-27 v-17 o-336 t-30 i-41 p-74 d-27 v-17 o-336 t-30 i-41 p-43 d-27 v-17 o-384 t-30 i-41 p-67 d-18 v-17 o-384 t-30 i-41 p-70 d-18 v-17 o-384 t-30 i-41 p-74 d-18 v-17 o-384 t-30 i-41 p-43 d-18 v-17 o-408 t-30 i-41 p-63 d-18 v-17 o-408 t-30 i-41 p-67 d-18 v-17 o-408 t-30 i-41 p-70 d-18 v-17 o-408 t-30 i-41 p-39 d-18 v-17 o-432 t-30 i-41 p-58 d-27 v-17 o-432 t-30 i-41 p-62 d-27 v-17 o-432 t-30 i-41 p-65 d-27 v-17 o-432 t-30 i-41 p-34 d-27 v-17 o-480 t-30 i-41 p-58 d-18 v-17 o-480 t-30 i-41 p-62 d-18 v-17 o-480 t-30 i-41 p-65 d-18 v-17 o-480 t-30 i-41 p-34 d-18 v-17 o-504 t-30 i-41 p-60 d-18 v-17 o-504 t-30 i-41 p-63 d-18 v-17 o-504 t-30 i-41 p-67 d-18 v-17 o-504 t-30 i-41 p-36 d-18 v-17 o-528 t-30 i-41 p-67 d-27 v-17 o-528 t-30 i-41 p-70 d-27 v-17 o-528 t-30 i-41 p-74 d-27 v-17 o-528 t-30 i-41 p-43 d-27 v-17 o-576 t-30 i-41 p-67 d-18 v-17 o-576 t-30 i-41 p-70 d-18 v-17 o-576 t-30 i-41 p-74 d-18 v-17 o-576 t-30 i-41 p-43 d-18 v-17 o-600 t-30 i-41 p-63 d-18 v-17 o-600 t-30 i-41 p-67 d-18 v-17 o-600 t-30 i-41 p-70 d-18 v-17 o-600 t-30 i-41 p-39 d-18 v-17 o-624 t-30 i-41 p-58 d-15 v-17 o-624 t-30 i-41 p-62 d-15 v-17 o-624 t-30 i-41 p-65 d-15 v-17 o-624 t-30 i-41 p-34 d-15 v-17 o-642 t-30 i-41 p-58 d-21 v-17 o-642 t-30 i-41 p-62 d-21 v-17 o-642 t-30 i-41 p-65 d-21 v-17 o-642 t-30 i-41 p-72 d-21 v-17 o-642 t-30 i-41 p-34 d-21 v-17"
# model_result_str = "s-9 o-42 t-26 i-30 p-77 d-6 v-20 o-48 t-26 i-30 p-75 d-24 v-20 o-90 t-26 i-30 p-77 d-3 v-20 o-93 t-26 i-30 p-75 d-16 v-20 o-114 t-26 i-30 p-73 d-3 v-20 o-117 t-26 i-30 p-73 d-16 v-20 o-144 t-26 i-30 p-77 d-9 v-20 o-153 t-26 i-30 p-78 d-9 v-20 o-162 t-26 i-30 p-77 d-12 v-20 o-174 t-26 i-30 p-82 d-6 v-20 o-180 t-26 i-30 p-82 d-6 v-20 o-186 t-26 i-30 p-80 d-25 v-20 o-237 t-26 i-30 p-82 d-3 v-20 o-240 t-26 i-30 p-82 d-6 v-20 o-246 t-26 i-30 p-80 d-3 v-20 o-249 t-26 i-30 p-80 d-9 v-20 o-258 t-26 i-30 p-78 d-3 v-20 o-261 t-26 i-30 p-78 d-9 v-20 o-270 t-26 i-30 p-77 d-3 v-20 o-273 t-26 i-30 p-76 d-6 v-20 o-279 t-26 i-30 p-77 d-6 v-20 o-285 t-26 i-30 p-80 d-9 v-20 o-294 t-26 i-30 p-78 d-3 v-20 o-297 t-26 i-30 p-78 d-16 v-20 o-318 t-26 i-30 p-80 d-6 v-20 o-330 t-26 i-30 p-77 d-3 v-20 o-333 t-26 i-30 p-77 d-3 v-20 o-336 t-26 i-30 p-78 d-6 v-20 o-342 t-26 i-30 p-77 d-6 v-20 o-348 t-26 i-30 p-70 d-6 v-20 o-354 t-26 i-30 p-73 d-6 v-20 o-360 t-26 i-30 p-77 d-6 v-20 o-366 t-26 i-30 p-75 d-3 v-20 o-369 t-26 i-30 p-75 d-9 v-20 o-378 t-26 i-30 p-73 d-3 v-20 o-381 t-26 i-30 p-73 d-24 v-20 o-426 t-26 i-30 p-77 d-6 v-20 o-432 t-26 i-30 p-78 d-6 v-20 o-438 t-26 i-30 p-77 d-6 v-20 o-444 t-26 i-30 p-70 d-6 v-20 o-450 t-26 i-30 p-73 d-6 v-20 o-456 t-26 i-30 p-77 d-18 v-20 o-483 t-26 i-30 p-75 d-9 v-20 o-492 t-26 i-30 p-73 d-24 v-20 o-0 t-26 i-0 p-49 d-27 v-17 o-0 t-26 i-0 p-53 d-27 v-17 o-0 t-26 i-0 p-56 d-27 v-17 o-0 t-26 i-0 p-25 d-27 v-17 o-48 t-26 i-0 p-65 d-27 v-17 o-48 t-26 i-0 p-69 d-27 v-17 o-48 t-26 i-0 p-72 d-27 v-17 o-48 t-26 i-0 p-75 d-27 v-17 o-48 t-26 i-0 p-41 d-27 v-17 o-96 t-26 i-0 p-58 d-27 v-17 o-96 t-26 i-0 p-61 d-27 v-17 o-96 t-26 i-0 p-65 d-27 v-17 o-96 t-26 i-0 p-68 d-27 v-17 o-96 t-26 i-0 p-34 d-27 v-17 o-144 t-26 i-0 p-56 d-15 v-17 o-144 t-26 i-0 p-59 d-15 v-17 o-144 t-26 i-0 p-63 d-15 v-17 o-144 t-26 i-0 p-66 d-15 v-17 o-144 t-26 i-0 p-32 d-15 v-17 o-162 t-26 i-0 p-61 d-21 v-17 o-162 t-26 i-0 p-65 d-21 v-17 o-162 t-26 i-0 p-68 d-21 v-17 o-162 t-26 i-0 p-71 d-21 v-17 o-162 t-26 i-0 p-37 d-21 v-17 o-192 t-26 i-0 p-54 d-18 v-17 o-192 t-26 i-0 p-56 d-18 v-17 o-192 t-26 i-0 p-61 d-18 v-17 o-192 t-26 i-0 p-30 d-18 v-17 o-216 t-26 i-0 p-56 d-18 v-17 o-216 t-26 i-0 p-60 d-18 v-17 o-216 t-26 i-0 p-63 d-18 v-17 o-216 t-26 i-0 p-66 d-18 v-17 o-216 t-26 i-0 p-32 d-18 v-17 o-240 t-26 i-0 p-53 d-18 v-17 o-240 t-26 i-0 p-56 d-18 v-17 o-240 t-26 i-0 p-60 d-18 v-17 o-240 t-26 i-0 p-63 d-18 v-17 o-240 t-26 i-0 p-29 d-18 v-17 o-264 t-26 i-0 p-58 d-18 v-17 o-264 t-26 i-0 p-62 d-18 v-17 o-264 t-26 i-0 p-65 d-18 v-17 o-264 t-26 i-0 p-68 d-18 v-17 o-264 t-26 i-0 p-34 d-18 v-17 o-288 t-26 i-0 p-51 d-27 v-17 o-288 t-26 i-0 p-54 d-27 v-17 o-288 t-26 i-0 p-56 d-27 v-17 o-288 t-26 i-0 p-58 d-27 v-17 o-288 t-26 i-0 p-61 d-27 v-17 o-288 t-26 i-0 p-27 d-27 v-17 o-336 t-26 i-0 p-56 d-27 v-17 o-336 t-26 i-0 p-61 d-27 v-17 o-336 t-26 i-0 p-63 d-27 v-17 o-336 t-26 i-0 p-66 d-27 v-17 o-336 t-26 i-0 p-32 d-27 v-17 o-384 t-26 i-0 p-49 d-18 v-17 o-384 t-26 i-0 p-53 d-18 v-17 o-384 t-26 i-0 p-56 d-18 v-17 o-384 t-26 i-0 p-25 d-18 v-17 o-408 t-26 i-0 p-58 d-18 v-17 o-408 t-26 i-0 p-61 d-18 v-17 o-408 t-26 i-0 p-65 d-18 v-17 o-408 t-26 i-0 p-68 d-18 v-17 o-408 t-26 i-0 p-34 d-18 v-17 o-432 t-26 i-0 p-51 d-18 v-17 o-432 t-26 i-0 p-54 d-18 v-17 o-432 t-26 i-0 p-56 d-18 v-17 o-432 t-26 i-0 p-58 d-18 v-17 o-432 t-26 i-0 p-61 d-18 v-17 o-432 t-26 i-0 p-27 d-18 v-17 o-456 t-26 i-0 p-56 d-18 v-17 o-456 t-26 i-0 p-60 d-18 v-17 o-456 t-26 i-0 p-63 d-18 v-17 o-456 t-26 i-0 p-66 d-18 v-17 o-456 t-26 i-0 p-70 d-18 v-17 o-456 t-26 i-0 p-73 d-18 v-17 o-456 t-26 i-0 p-32 d-18 v-17 o-480 t-26 i-0 p-51 d-18 v-17 o-480 t-26 i-0 p-54 d-18 v-17 o-480 t-26 i-0 p-58 d-18 v-17 o-480 t-26 i-0 p-61 d-18 v-17 o-480 t-26 i-0 p-27 d-18 v-17 o-504 t-26 i-0 p-68 d-18 v-17 o-504 t-26 i-0 p-60 d-18 v-17 o-504 t-26 i-0 p-63 d-18 v-17 o-504 t-26 i-0 p-36 d-18 v-17"
# model_result_str = "s-9 o-0 t-30 i-25 p-56 d-3 v-20 o-3 t-30 i-25 p-54 d-6 v-20 o-9 t-30 i-25 p-56 d-6 v-20 o-15 t-30 i-25 p-54 d-6 v-20 o-21 t-30 i-25 p-56 d-6 v-20 o-27 t-30 i-25 p-54 d-6 v-20 o-33 t-30 i-25 p-56 d-6 v-20 o-39 t-30 i-25 p-54 d-6 v-20 o-45 t-30 i-25 p-58 d-6 v-20 o-51 t-30 i-25 p-54 d-6 v-20 o-57 t-30 i-25 p-51 d-9 v-20 o-93 t-30 i-25 p-54 d-3 v-20 o-96 t-30 i-25 p-56 d-3 v-20 o-99 t-30 i-25 p-54 d-6 v-20 o-105 t-30 i-25 p-56 d-6 v-20 o-111 t-30 i-25 p-54 d-6 v-20 o-117 t-30 i-25 p-56 d-6 v-20 o-123 t-30 i-25 p-54 d-6 v-20 o-129 t-30 i-25 p-58 d-6 v-20 o-135 t-30 i-25 p-54 d-6 v-20 o-141 t-30 i-25 p-51 d-3 v-20 o-144 t-30 i-25 p-49 d-4 v-20 o-148 t-30 i-25 p-49 d-4 v-20 o-152 t-30 i-25 p-49 d-4 v-20 o-156 t-30 i-25 p-49 d-6 v-20 o-162 t-30 i-25 p-48 d-3 v-20 o-165 t-30 i-25 p-48 d-6 v-20 o-171 t-30 i-25 p-48 d-3 v-20 o-174 t-30 i-25 p-48 d-6 v-20 o-180 t-30 i-25 p-48 d-6 v-20 o-189 t-30 i-25 p-54 d-3 v-20 o-192 t-30 i-25 p-56 d-6 v-20 o-198 t-30 i-25 p-54 d-3 v-20 o-201 t-30 i-25 p-56 d-6 v-20 o-207 t-30 i-25 p-54 d-6 v-20 o-213 t-30 i-25 p-56 d-6 v-20 o-219 t-30 i-25 p-54 d-6 v-20 o-225 t-30 i-25 p-56 d-6 v-20 o-231 t-30 i-25 p-54 d-6 v-20 o-237 t-30 i-25 p-58 d-6 v-20 o-243 t-30 i-25 p-54 d-6 v-20 o-249 t-30 i-25 p-51 d-9 v-20 o-285 t-30 i-25 p-54 d-3 v-20 o-288 t-30 i-25 p-56 d-6 v-20 o-294 t-30 i-25 p-54 d-3 v-20 o-297 t-30 i-25 p-56 d-6 v-20 o-303 t-30 i-25 p-54 d-6 v-20 o-309 t-30 i-25 p-56 d-6 v-20 o-315 t-30 i-25 p-54 d-6 v-20 o-321 t-30 i-25 p-58 d-6 v-20 o-327 t-30 i-25 p-54 d-6 v-20 o-333 t-30 i-25 p-51 d-13 v-20 o-372 t-30 i-25 p-63 d-6 v-20 o-378 t-30 i-25 p-65 d-6 v-20 o-384 t-30 i-25 p-66 d-12 v-20 o-396 t-30 i-25 p-65 d-6 v-20 o-402 t-30 i-25 p-63 d-6 v-20 o-408 t-30 i-25 p-65 d-12 v-20 o-438 t-30 i-25 p-63 d-3 v-20 o-447 t-30 i-25 p-61 d-3 v-20 o-450 t-30 i-25 p-63 d-3 v-20 o-453 t-30 i-25 p-61 d-3 v-20 o-456 t-30 i-25 p-63 d-3 v-20 o-459 t-30 i-25 p-61 d-3 v-20 o-462 t-30 i-25 p-58 d-3 v-20 o-465 t-30 i-25 p-56 d-3 v-20 o-468 t-30 i-25 p-56 d-3 v-20 o-471 t-30 i-25 p-53 d-3 v-20 o-474 t-30 i-25 p-56 d-6 v-20 o-480 t-30 i-25 p-58 d-6 v-20 o-498 t-30 i-25 p-46 d-3 v-20 o-501 t-30 i-25 p-46 d-6 v-20 o-507 t-30 i-25 p-46 d-3 v-20 o-510 t-30 i-25 p-46 d-3 v-20 o-513 t-30 i-25 p-46 d-6 v-20 o-519 t-30 i-25 p-46 d-6 v-20 o-564 t-30 i-25 p-63 d-6 v-20 o-570 t-30 i-25 p-65 d-6 v-20 o-576 t-30 i-25 p-66 d-12 v-20 o-588 t-30 i-25 p-63 d-12 v-20 o-600 t-30 i-25 p-65 d-15 v-20 o-630 t-30 i-25 p-63 d-3 v-20 o-639 t-30 i-25 p-61 d-3 v-20 o-642 t-30 i-25 p-63 d-3 v-20 o-645 t-30 i-25 p-61 d-3 v-20 o-648 t-30 i-25 p-63 d-3 v-20 o-651 t-30 i-25 p-61 d-3 v-20 o-654 t-30 i-25 p-58 d-3 v-20 o-657 t-30 i-25 p-56 d-3 v-20 o-660 t-30 i-25 p-56 d-6 v-20 o-666 t-30 i-25 p-56 d-6 v-20 o-672 t-30 i-25 p-58 d-12 v-20 o-693 t-30 i-25 p-46 d-3 v-20 o-696 t-30 i-25 p-46 d-4 v-20 o-700 t-30 i-25 p-46 d-4 v-20 o-704 t-30 i-25 p-46 d-4 v-20 o-708 t-30 i-25 p-46 d-6 v-20 o-738 t-30 i-25 p-61 d-3 v-20 o-744 t-30 i-25 p-59 d-12 v-20 o-756 t-30 i-25 p-58 d-3 v-20 o-762 t-30 i-25 p-54 d-6 v-20 o-0 t-30 i-3 p-71 d-15 v-17 o-0 t-30 i-3 p-63 d-15 v-17 o-0 t-30 i-3 p-66 d-15 v-17 o-0 t-30 i-3 p-39 d-15 v-17 o-18 t-30 i-3 p-70 d-21 v-17 o-18 t-30 i-3 p-62 d-21 v-17 o-18 t-30 i-3 p-65 d-21 v-17 o-18 t-30 i-3 p-38 d-21 v-17 o-48 t-30 i-3 p-51 d-15 v-17 o-48 t-30 i-3 p-54 d-15 v-17 o-48 t-30 i-3 p-56 d-15 v-17 o-48 t-30 i-3 p-58 d-15 v-17 o-48 t-30 i-3 p-27 d-15 v-17 o-66 t-30 i-3 p-72 d-21 v-17 o-66 t-30 i-3 p-63 d-21 v-17 o-66 t-30 i-3 p-66 d-21 v-17 o-66 t-30 i-3 p-39 d-21 v-17 o-96 t-30 i-3 p-71 d-15 v-17 o-96 t-30 i-3 p-63 d-15 v-17 o-96 t-30 i-3 p-66 d-15 v-17 o-96 t-30 i-3 p-39 d-15 v-17 o-114 t-30 i-3 p-70 d-21 v-17 o-114 t-30 i-3 p-62 d-21 v-17 o-114 t-30 i-3 p-65 d-21 v-17 o-114 t-30 i-3 p-38 d-21 v-17 o-144 t-30 i-3 p-51 d-15 v-17 o-144 t-30 i-3 p-54 d-15 v-17 o-144 t-30 i-3 p-56 d-15 v-17 o-144 t-30 i-3 p-58 d-15 v-17 o-144 t-30 i-3 p-27 d-15 v-17 o-162 t-30 i-3 p-72 d-21 v-17 o-162 t-30 i-3 p-63 d-21 v-17 o-162 t-30 i-3 p-66 d-21 v-17 o-162 t-30 i-3 p-39 d-21 v-17 o-192 t-30 i-3 p-71 d-15 v-17 o-192 t-30 i-3 p-63 d-15 v-17 o-192 t-30 i-3 p-66 d-15 v-17 o-192 t-30 i-3 p-39 d-15 v-17 o-210 t-30 i-3 p-70 d-21 v-17 o-210 t-30 i-3 p-62 d-21 v-17 o-210 t-30 i-3 p-65 d-21 v-17 o-210 t-30 i-3 p-38 d-21 v-17 o-240 t-30 i-3 p-51 d-15 v-17 o-240 t-30 i-3 p-54 d-15 v-17 o-240 t-30 i-3 p-56 d-15 v-17 o-240 t-30 i-3 p-58 d-15 v-17 o-240 t-30 i-3 p-27 d-15 v-17 o-258 t-30 i-3 p-72 d-21 v-17 o-258 t-30 i-3 p-63 d-21 v-17 o-258 t-30 i-3 p-66 d-21 v-17 o-258 t-30 i-3 p-39 d-21 v-17 o-288 t-30 i-3 p-71 d-15 v-17 o-288 t-30 i-3 p-63 d-15 v-17 o-288 t-30 i-3 p-66 d-15 v-17 o-288 t-30 i-3 p-39 d-15 v-17 o-306 t-30 i-3 p-70 d-21 v-17 o-306 t-30 i-3 p-62 d-21 v-17 o-306 t-30 i-3 p-65 d-21 v-17 o-306 t-30 i-3 p-38 d-21 v-17 o-336 t-30 i-3 p-51 d-15 v-17 o-336 t-30 i-3 p-54 d-15 v-17 o-336 t-30 i-3 p-56 d-15 v-17 o-336 t-30 i-3 p-58 d-15 v-17 o-336 t-30 i-3 p-27 d-15 v-17 o-354 t-30 i-3 p-72 d-21 v-17 o-354 t-30 i-3 p-63 d-21 v-17 o-354 t-30 i-3 p-66 d-21 v-17 o-354 t-30 i-3 p-39 d-21 v-17 o-384 t-30 i-3 p-71 d-15 v-17 o-384 t-30 i-3 p-63 d-15 v-17 o-384 t-30 i-3 p-66 d-15 v-17 o-384 t-30 i-3 p-39 d-15 v-17 o-402 t-30 i-3 p-70 d-21 v-17 o-402 t-30 i-3 p-62 d-21 v-17 o-402 t-30 i-3 p-65 d-21 v-17 o-402 t-30 i-3 p-38 d-21 v-17 o-432 t-30 i-3 p-51 d-15 v-17 o-432 t-30 i-3 p-54 d-15 v-17 o-432 t-30 i-3 p-56 d-15 v-17 o-432 t-30 i-3 p-58 d-15 v-17 o-432 t-30 i-3 p-27 d-15 v-17 o-450 t-30 i-3 p-72 d-21 v-17 o-450 t-30 i-3 p-63 d-21 v-17 o-450 t-30 i-3 p-66 d-21 v-17 o-450 t-30 i-3 p-39 d-21 v-17 o-480 t-30 i-3 p-71 d-15 v-17 o-480 t-30 i-3 p-63 d-15 v-17 o-480 t-30 i-3 p-66 d-15 v-17 o-480 t-30 i-3 p-39 d-15 v-17 o-498 t-30 i-3 p-70 d-21 v-17 o-498 t-30 i-3 p-62 d-21 v-17 o-498 t-30 i-3 p-65 d-21 v-17 o-498 t-30 i-3 p-38 d-21 v-17 o-528 t-30 i-3 p-51 d-15 v-17 o-528 t-30 i-3 p-54 d-15 v-17 o-528 t-30 i-3 p-56 d-15 v-17 o-528 t-30 i-3 p-58 d-15 v-17 o-528 t-30 i-3 p-27 d-15 v-17 o-546 t-30 i-3 p-72 d-21 v-17 o-546 t-30 i-3 p-63 d-21 v-17 o-546 t-30 i-3 p-66 d-21 v-17 o-546 t-30 i-3 p-39 d-21 v-17 o-576 t-30 i-3 p-71 d-15 v-17 o-576 t-30 i-3 p-63 d-15 v-17 o-576 t-30 i-3 p-66 d-15 v-17 o-576 t-30 i-3 p-39 d-15 v-17 o-594 t-30 i-3 p-70 d-21 v-17 o-594 t-30 i-3 p-62 d-21 v-17 o-594 t-30 i-3 p-65 d-21 v-17 o-594 t-30 i-3 p-38 d-21 v-17 o-624 t-30 i-3 p-51 d-15 v-17 o-624 t-30 i-3 p-54 d-15 v-17 o-624 t-30 i-3 p-56 d-15 v-17 o-624 t-30 i-3 p-58 d-15 v-17 o-624 t-30 i-3 p-27 d-15 v-17 o-642 t-30 i-3 p-72 d-21 v-17 o-642 t-30 i-3 p-63 d-21 v-17 o-642 t-30 i-3 p-66 d-21 v-17 o-642 t-30 i-3 p-39 d-21 v-17 o-672 t-30 i-3 p-71 d-15 v-17 o-672 t-30 i-3 p-63 d-15 v-17 o-672 t-30 i-3 p-66 d-15 v-17 o-672 t-30 i-3 p-39 d-15 v-17 o-690 t-30 i-3 p-70 d-21 v-17 o-690 t-30 i-3 p-62 d-21 v-17 o-690 t-30 i-3 p-65 d-21 v-17 o-690 t-30 i-3 p-38 d-21 v-17 o-720 t-30 i-3 p-51 d-15 v-17 o-720 t-30 i-3 p-54 d-15 v-17 o-720 t-30 i-3 p-56 d-15 v-17 o-720 t-30 i-3 p-58 d-15 v-17 o-720 t-30 i-3 p-27 d-15 v-17 o-738 t-30 i-3 p-72 d-21 v-17 o-738 t-30 i-3 p-63 d-21 v-17 o-738 t-30 i-3 p-66 d-21 v-17 o-738 t-30 i-3 p-39 d-21 v-17"
# model_result_str = "s-9 o-30 t-36 i-24 p-75 d-6 v-20 o-36 t-36 i-24 p-74 d-6 v-20 o-42 t-36 i-24 p-70 d-6 v-20 o-48 t-36 i-24 p-70 d-9 v-20 o-57 t-36 i-24 p-72 d-3 v-20 o-63 t-36 i-24 p-67 d-9 v-20 o-72 t-36 i-24 p-65 d-6 v-20 o-78 t-36 i-24 p-63 d-6 v-20 o-84 t-36 i-24 p-65 d-6 v-20 o-90 t-36 i-24 p-72 d-6 v-20 o-96 t-36 i-24 p-70 d-6 v-20 o-102 t-36 i-24 p-65 d-3 v-20 o-105 t-36 i-24 p-67 d-6 v-20 o-111 t-36 i-24 p-63 d-3 v-20 o-114 t-36 i-24 p-65 d-6 v-20 o-120 t-36 i-24 p-63 d-6 v-20 o-132 t-36 i-24 p-63 d-6 v-20 o-138 t-36 i-24 p-70 d-6 v-20 o-144 t-36 i-24 p-68 d-6 v-20 o-150 t-36 i-24 p-67 d-6 v-20 o-156 t-36 i-24 p-65 d-6 v-20 o-162 t-36 i-24 p-63 d-6 v-20 o-168 t-36 i-24 p-62 d-6 v-20 o-174 t-36 i-24 p-63 d-6 v-20 o-180 t-36 i-24 p-65 d-6 v-20 o-186 t-36 i-24 p-68 d-6 v-20 o-192 t-36 i-24 p-67 d-6 v-20 o-198 t-36 i-24 p-65 d-3 v-20 o-201 t-36 i-24 p-67 d-6 v-20 o-207 t-36 i-24 p-72 d-9 v-20 o-216 t-36 i-24 p-70 d-6 v-20 o-228 t-36 i-24 p-67 d-6 v-20 o-234 t-36 i-24 p-70 d-6 v-20 o-240 t-36 i-24 p-72 d-9 v-20 o-249 t-36 i-24 p-68 d-9 v-20 o-258 t-36 i-24 p-67 d-6 v-20 o-264 t-36 i-24 p-65 d-6 v-20 o-270 t-36 i-24 p-63 d-6 v-20 o-276 t-36 i-24 p-65 d-6 v-20 o-282 t-36 i-24 p-72 d-6 v-20 o-288 t-36 i-24 p-70 d-6 v-20 o-294 t-36 i-24 p-72 d-3 v-20 o-297 t-36 i-24 p-67 d-3 v-20 o-303 t-36 i-24 p-63 d-3 v-20 o-306 t-36 i-24 p-65 d-6 v-20 o-312 t-36 i-24 p-63 d-6 v-20 o-324 t-36 i-24 p-60 d-6 v-20 o-330 t-36 i-24 p-62 d-6 v-20 o-336 t-36 i-24 p-63 d-6 v-20 o-342 t-36 i-24 p-60 d-6 v-20 o-348 t-36 i-24 p-63 d-6 v-20 o-354 t-36 i-24 p-65 d-6 v-20 o-360 t-36 i-24 p-67 d-6 v-20 o-366 t-36 i-24 p-63 d-6 v-20 o-372 t-36 i-24 p-67 d-6 v-20 o-378 t-36 i-24 p-72 d-6 v-20 o-384 t-36 i-24 p-71 d-12 v-20 o-402 t-36 i-24 p-63 d-6 v-20 o-408 t-36 i-24 p-63 d-6 v-20 o-414 t-36 i-24 p-75 d-6 v-20 o-420 t-36 i-24 p-74 d-6 v-20 o-426 t-36 i-24 p-70 d-6 v-20 o-432 t-36 i-24 p-70 d-9 v-20 o-441 t-36 i-24 p-72 d-3 v-20 o-447 t-36 i-24 p-67 d-9 v-20 o-456 t-36 i-24 p-65 d-6 v-20 o-462 t-36 i-24 p-63 d-6 v-20 o-468 t-36 i-24 p-65 d-6 v-20 o-474 t-36 i-24 p-72 d-6 v-20 o-480 t-36 i-24 p-70 d-6 v-20 o-486 t-36 i-24 p-72 d-3 v-20 o-489 t-36 i-24 p-67 d-3 v-20 o-495 t-36 i-24 p-63 d-3 v-20 o-498 t-36 i-24 p-65 d-6 v-20 o-504 t-36 i-24 p-63 d-6 v-20 o-516 t-36 i-24 p-63 d-6 v-20 o-522 t-36 i-24 p-70 d-6 v-20 o-528 t-36 i-24 p-68 d-6 v-20 o-534 t-36 i-24 p-67 d-6 v-20 o-540 t-36 i-24 p-65 d-6 v-20 o-546 t-36 i-24 p-63 d-6 v-20 o-552 t-36 i-24 p-62 d-6 v-20 o-558 t-36 i-24 p-63 d-6 v-20 o-564 t-36 i-24 p-65 d-6 v-20 o-570 t-36 i-24 p-68 d-6 v-20 o-576 t-36 i-24 p-67 d-6 v-20 o-582 t-36 i-24 p-65 d-3 v-20 o-585 t-36 i-24 p-67 d-6 v-20 o-591 t-36 i-24 p-72 d-9 v-20 o-600 t-36 i-24 p-70 d-6 v-20 o-612 t-36 i-24 p-67 d-6 v-20 o-618 t-36 i-24 p-70 d-6 v-20 o-624 t-36 i-24 p-72 d-9 v-20 o-633 t-36 i-24 p-68 d-9 v-20 o-642 t-36 i-24 p-67 d-6 v-20 o-648 t-36 i-24 p-65 d-6 v-20 o-654 t-36 i-24 p-74 d-6 v-20 o-660 t-36 i-24 p-72 d-6 v-20 o-666 t-36 i-24 p-70 d-6 v-20 o-672 t-36 i-24 p-70 d-6 v-20 o-678 t-36 i-24 p-72 d-6 v-20 o-684 t-36 i-24 p-74 d-6 v-20 o-690 t-36 i-24 p-75 d-6 v-20 o-702 t-36 i-24 p-67 d-6 v-20 o-708 t-36 i-24 p-65 d-6 v-20 o-714 t-36 i-24 p-63 d-6 v-20 o-726 t-36 i-24 p-60 d-6 v-20 o-732 t-36 i-24 p-63 d-6 v-20 o-738 t-36 i-24 p-68 d-3 v-20 o-741 t-36 i-24 p-67 d-6 v-20 o-750 t-36 i-24 p-63 d-6 v-20 o-756 t-36 i-24 p-65 d-6 v-20 o-762 t-36 i-24 p-63 d-6 v-20 o-48 t-36 i-21 p-56 d-18 v-17 o-48 t-36 i-21 p-60 d-18 v-17 o-48 t-36 i-21 p-63 d-18 v-17 o-48 t-36 i-21 p-67 d-18 v-17 o-48 t-36 i-21 p-32 d-18 v-17 o-72 t-36 i-21 p-58 d-18 v-17 o-72 t-36 i-21 p-62 d-18 v-17 o-72 t-36 i-21 p-65 d-18 v-17 o-72 t-36 i-21 p-68 d-18 v-17 o-72 t-36 i-21 p-34 d-18 v-17 o-96 t-36 i-21 p-55 d-18 v-17 o-96 t-36 i-21 p-58 d-18 v-17 o-96 t-36 i-21 p-62 d-18 v-17 o-96 t-36 i-21 p-65 d-18 v-17 o-96 t-36 i-21 p-31 d-18 v-17 o-120 t-36 i-21 p-48 d-18 v-17 o-120 t-36 i-21 p-51 d-18 v-17 o-120 t-36 i-21 p-55 d-18 v-17 o-120 t-36 i-21 p-24 d-18 v-17 o-144 t-36 i-21 p-56 d-18 v-17 o-144 t-36 i-21 p-60 d-18 v-17 o-144 t-36 i-21 p-63 d-18 v-17 o-144 t-36 i-21 p-67 d-18 v-17 o-144 t-36 i-21 p-32 d-18 v-17 o-168 t-36 i-21 p-55 d-18 v-17 o-168 t-36 i-21 p-59 d-18 v-17 o-168 t-36 i-21 p-62 d-18 v-17 o-168 t-36 i-21 p-65 d-18 v-17 o-168 t-36 i-21 p-31 d-18 v-17 o-192 t-36 i-21 p-48 d-18 v-17 o-192 t-36 i-21 p-51 d-18 v-17 o-192 t-36 i-21 p-55 d-18 v-17 o-192 t-36 i-21 p-24 d-18 v-17 o-216 t-36 i-21 p-79 d-12 v-17 o-216 t-36 i-21 p-82 d-12 v-17 o-216 t-36 i-21 p-73 d-12 v-17 o-216 t-36 i-21 p-49 d-12 v-17 o-228 t-36 i-21 p-63 d-12 v-17 o-228 t-36 i-21 p-67 d-12 v-17 o-228 t-36 i-21 p-70 d-12 v-17 o-228 t-36 i-21 p-73 d-12 v-17 o-228 t-36 i-21 p-39 d-12 v-17 o-240 t-36 i-21 p-56 d-18 v-17 o-240 t-36 i-21 p-60 d-18 v-17 o-240 t-36 i-21 p-63 d-18 v-17 o-240 t-36 i-21 p-67 d-18 v-17 o-240 t-36 i-21 p-32 d-18 v-17 o-264 t-36 i-21 p-58 d-18 v-17 o-264 t-36 i-21 p-62 d-18 v-17 o-264 t-36 i-21 p-65 d-18 v-17 o-264 t-36 i-21 p-68 d-18 v-17 o-264 t-36 i-21 p-34 d-18 v-17 o-288 t-36 i-21 p-55 d-18 v-17 o-288 t-36 i-21 p-58 d-18 v-17 o-288 t-36 i-21 p-62 d-18 v-17 o-288 t-36 i-21 p-65 d-18 v-17 o-288 t-36 i-21 p-31 d-18 v-17 o-312 t-36 i-21 p-48 d-18 v-17 o-312 t-36 i-21 p-51 d-18 v-17 o-312 t-36 i-21 p-55 d-18 v-17 o-312 t-36 i-21 p-24 d-18 v-17 o-336 t-36 i-21 p-56 d-27 v-17 o-336 t-36 i-21 p-60 d-27 v-17 o-336 t-36 i-21 p-63 d-27 v-17 o-336 t-36 i-21 p-67 d-27 v-17 o-336 t-36 i-21 p-32 d-27 v-17 o-384 t-36 i-21 p-55 d-18 v-17 o-384 t-36 i-21 p-59 d-18 v-17 o-384 t-36 i-21 p-62 d-18 v-17 o-384 t-36 i-21 p-65 d-18 v-17 o-384 t-36 i-21 p-31 d-18 v-17 o-408 t-36 i-21 p-55 d-18 v-17 o-408 t-36 i-21 p-59 d-18 v-17 o-408 t-36 i-21 p-62 d-18 v-17 o-408 t-36 i-21 p-65 d-18 v-17 o-408 t-36 i-21 p-31 d-18 v-17 o-432 t-36 i-21 p-56 d-18 v-17 o-432 t-36 i-21 p-60 d-18 v-17 o-432 t-36 i-21 p-63 d-18 v-17 o-432 t-36 i-21 p-67 d-18 v-17 o-432 t-36 i-21 p-32 d-18 v-17 o-456 t-36 i-21 p-58 d-18 v-17 o-456 t-36 i-21 p-62 d-18 v-17 o-456 t-36 i-21 p-65 d-18 v-17 o-456 t-36 i-21 p-68 d-18 v-17 o-456 t-36 i-21 p-34 d-18 v-17 o-480 t-36 i-21 p-55 d-18 v-17 o-480 t-36 i-21 p-58 d-18 v-17 o-480 t-36 i-21 p-62 d-18 v-17 o-480 t-36 i-21 p-65 d-18 v-17 o-480 t-36 i-21 p-31 d-18 v-17 o-504 t-36 i-21 p-48 d-18 v-17 o-504 t-36 i-21 p-51 d-18 v-17 o-504 t-36 i-21 p-55 d-18 v-17 o-504 t-36 i-21 p-24 d-18 v-17 o-528 t-36 i-21 p-56 d-18 v-17 o-528 t-36 i-21 p-60 d-18 v-17 o-528 t-36 i-21 p-63 d-18 v-17 o-528 t-36 i-21 p-67 d-18 v-17 o-528 t-36 i-21 p-32 d-18 v-17 o-552 t-36 i-21 p-55 d-18 v-17 o-552 t-36 i-21 p-59 d-18 v-17 o-552 t-36 i-21 p-62 d-18 v-17 o-552 t-36 i-21 p-65 d-18 v-17 o-552 t-36 i-21 p-31 d-18 v-17 o-576 t-36 i-21 p-48 d-18 v-17 o-576 t-36 i-21 p-51 d-18 v-17 o-576 t-36 i-21 p-55 d-18 v-17 o-576 t-36 i-21 p-24 d-18 v-17 o-600 t-36 i-21 p-79 d-12 v-17 o-600 t-36 i-21 p-82 d-12 v-17 o-600 t-36 i-21 p-73 d-12 v-17 o-600 t-36 i-21 p-49 d-12 v-17 o-612 t-36 i-21 p-63 d-12 v-17 o-612 t-36 i-21 p-67 d-12 v-17 o-612 t-36 i-21 p-70 d-12 v-17 o-612 t-36 i-21 p-73 d-12 v-17 o-612 t-36 i-21 p-39 d-12 v-17 o-624 t-36 i-21 p-56 d-18 v-17 o-624 t-36 i-21 p-60 d-18 v-17 o-624 t-36 i-21 p-63 d-18 v-17 o-624 t-36 i-21 p-67 d-18 v-17 o-624 t-36 i-21 p-32 d-18 v-17 o-648 t-36 i-21 p-58 d-18 v-17 o-648 t-36 i-21 p-62 d-18 v-17 o-648 t-36 i-21 p-65 d-18 v-17 o-648 t-36 i-21 p-68 d-18 v-17 o-648 t-36 i-21 p-34 d-18 v-17 o-672 t-36 i-21 p-67 d-18 v-17 o-672 t-36 i-21 p-59 d-18 v-17 o-672 t-36 i-21 p-62 d-18 v-17 o-672 t-36 i-21 p-65 d-18 v-17 o-672 t-36 i-21 p-35 d-18 v-17 o-696 t-36 i-21 p-48 d-12 v-17 o-696 t-36 i-21 p-51 d-12 v-17 o-696 t-36 i-21 p-55 d-12 v-17 o-696 t-36 i-21 p-24 d-12 v-17 o-708 t-36 i-21 p-48 d-12 v-17 o-708 t-36 i-21 p-51 d-12 v-17 o-708 t-36 i-21 p-55 d-12 v-17 o-708 t-36 i-21 p-58 d-12 v-17 o-708 t-36 i-21 p-24 d-12 v-17 o-720 t-36 i-21 p-56 d-16 v-17 o-720 t-36 i-21 p-60 d-16 v-17 o-720 t-36 i-21 p-63 d-16 v-17 o-720 t-36 i-21 p-67 d-16 v-17 o-720 t-36 i-21 p-32 d-16 v-17 o-741 t-36 i-21 p-58 d-12 v-17 o-741 t-36 i-21 p-62 d-12 v-17 o-741 t-36 i-21 p-65 d-12 v-17 o-741 t-36 i-21 p-34 d-12 v-17"
# model_result_str = "s-9 o-0 t-41 i-42 p-48 d-2 v-20 o-6 t-41 i-42 p-48 d-6 v-20 p-52 d-2 v-20 o-6 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-42 p-48 d-6 v-20 b-1 s-8 o-0 t-41 i-42 p-48 d-6 v-20 o-8 t-41 i-0 p-48 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-52 d-6 v-20 o-21 t-41 i-0 p-60 d-11 v-20 o-12 t-41 i-42 p-60 d-2 v-20 o-20 t-41 i-42 p-50 d-3 v-20 o-4 t-41 i-42 p-53 d-14 v-20 o-20 t-41 i-42 p-55 d-3 v-20 o-8 t-41 i-42 p-52 d-17 v-20 o-32 t-41 i-42 p-48 d-3 v-20 b-1 s-8 o-0 t-41 i-42 p-55 d-6 v-20 b-1 s-8 o-0 t-41 i-42 p-48 d-3 v-20 o-32 t-41 i-42 p-43 d-6 v-20 b-1 s-9 o-0 t-41 i-42 p-52 d-1 v-20 o-24 t-41 i-42 p-53 d-3 v-20 o-32 t-41 i-42 p-50 d-1 v-20 p-52 d-11 v-20 i-42 p-43 d-6 v-20 o-32 t-41 i-42 p-43 d-11 v-20 o-16 t-41 i-42 p-48 d-4 v-20 o-8 t-41 i-42 p-53 d-3 v-20 o-20 t-41 i-42 p-52 d-3 v-20 o-12 t-41 i-42 p-52 d-1 v-20 o-20 t-41 i-42 p-50 d-2 v-20 o-32 t-41 i-42 p-43 d-6 v-20 o-16 t-41 i-42 p-50 d-3 v-20 o-8 t-41 i-42 p-50 d-3 v-20 o-28 t-41 i-42 p-42 d-2 v-20 o-32 t-41 i-42 p-47 d-1 v-20 o-32 t-41 i-42 p-45 d-1 v-20 o-20 t-41 i-42 p-48 d-1 v-20 o-32 t-41 i-42 p-42 d-2 v-20 o-20 t-41 i-42 p-57 d-1 v-20 o-16 t-41 i-42 p-52 d-1 v-20 o-16 t-41 i-42 p-48 d-1 v-20 o-32 t-41 i-42 p-43 d-3 v-20 o-8 t-41 i-42 p-52 d-2 v-20 o-20 t-41 i-42 p-50 d-1 v-20 o-20 t-41 i-42 p-45 d-1 v-20 o-20 t-41 i-42 p-36 d-6 v-20 o-20 t-41 i-42 p-43 d-1 v-20 o-20 t-41 i-42 p-44 d-2 v-20 o-20 t-41 i-42 p-50 d-2 v-20 o-28 t-41 i-42 p-48 d-1 v-20 i-42 p-52 d-1 v-20 o-20 t-41 i-42 p-48 d-2 v-20 o-16 t-41 i-42 p-52 d-2 v-20 o-16 t-41 i-42 p-44 d-3 v-20 o-20 t-41 i-42 p-38 d-3 v-20 o-20 t-41 i-42 p-52 d-1 v-20 o-8 t-41 i-42 p-42 S4_19_2 i-42 p-45 d-1 v-20 i-42 p-36 d-2 v-20 o-32 t-41 i-42 p-52 d-1 v-20 o-8 t-41 i-42 p-52 d-1 v-20 o-32 t-41 i-42 p-52 d-1 v-20 o-8 t-41 i-42 p-36 d-1 v-20 o-8 t-41 i-42 p-42 d-1 v-20 o-8 t-41 i-42 p-48 d-1 v-20 o-8 t-41 i-42 p-42 d-3 v-25 i-42 p-44 d-2 v-20 o-32 t-41 i-42 p-52 d-2 v-25 o-16 t-41 i-42 p-45 d-1 v-25 i-42 p-43 d-1 v-20 o-16 t-41 i-42 p-42 d-1 v-20 o-16 t-41 i-42 p-48 d-2 v-20 o-8 t-41 i-42 p-50 d-1 v-20 o-8 t-41 i-42 p-42 d-3 v-20 o-8 t-41 i-1 o-16 t-41 i-42 p-57 d-2 v-20 o-16 t-41 i-42 p-52 d-2 v-20 o-32 t-41 i-42"
# model_result_str = "s-9 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-69 d-11 v-20 o-24 t-41 i-0 p-65 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-12 t-41 i-0 p-67 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-67 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-12 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-59 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0"
# model_result_str = "s-9 o-0 t-41 i-0 p-62 d-11 v-20 o-12 t-41 i-0 p-67 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-17 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-65 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-59 d-11 v-20 o-24 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-67 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-59 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-24 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-59 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 o-12 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-41 i-0"
# model_result_str = "s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-62 d-12 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 o-12 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11"
# model_result_str = "s-19 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-59 d-6 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-60 d-11 v-20 o-12 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-59 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-59 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-65 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-59 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-59 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-62 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-45 v-20 b-1 s-8 o-0 t-35 i-0 p-67 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11 v-20 o-24 t-35 i-0 p-60 d-11 v-20 b-1 s-8 o-0 t-35 i-0 p-60 d-11"
model_result_str = "s-19 o-0 t-35 i-0 p-64 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-67 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-12 t-35 i-0 p-62 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-64 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-11 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 b-1 s-9 o-0 t-35 i-0 p-60 d-6 v-20 o-24 t-35 i-0 p-60 d-6 v-20 b-1"

In [6]:
mid_file_name_prefix = "model_result"
# mid_file_name_prefix = "check_hook_data"

1: p-20 d-10 v-15
2: p-x d-5 v-y
3: p-25 d-15 v-10




In [7]:
now = datetime.now()

dd = int(now.strftime("%d"))
mm = int(now.strftime("%m"))
yyyy = int(now.strftime("%Y"))
hh = int(now.strftime("%H"))


am_pm = None
if hh > 12:
    hh = hh - 12
    am_pm = "pm"
else:
    am_pm = "am"

# Make dd, mm, hh 2 digits
dd = f"{'0' * (2 - len(str(dd)))}{dd}"
mm = f"{'0' * (2 - len(str(mm)))}{mm}"
hh = f"{'0' * (2 - len(str(hh)))}{hh}"

print(f"{dd} {mm} {yyyy} {hh}{am_pm}")

23 06 2024 06pm


In [8]:
log_date_folder_path = f"{log_dir}/{yyyy}_{mm}_{dd}"
log_time_folder_path = f"{log_date_folder_path}/{hh}{am_pm}"

print(log_date_folder_path)
print(log_time_folder_path)

# Create the folders if it does not exist
if not os.path.exists(log_date_folder_path):
    os.mkdir(log_date_folder_path)

if not os.path.exists(log_time_folder_path):
    os.mkdir(log_time_folder_path)

/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/logs/model_result_log/2024_06_23
/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/logs/model_result_log/2024_06_23/06pm


In [9]:
# Count the number of folders in the log_date_folder_path (excluding the .DS_Store file)
log_date_folders = [f for f in os.listdir(log_date_folder_path) if not f.startswith('.')]

n_log_time_folders = 0
# Count log time folders in each log date folder
for log_date_folder in log_date_folders:
    log_time_folder = f"{log_date_folder_path}/{log_date_folder}"
    
    # Count the number of folders in the log_time_folder (excluding the .DS_Store file)
    log_time_folders = [f for f in os.listdir(log_time_folder) if not f.startswith('.')]
    n_log_time_folders += len(log_time_folders)

midi_index = n_log_time_folders

print(f"midi_index: {midi_index}")

midi_index: 0


In [10]:
log_index_folder_path = f"{log_time_folder_path}/{midi_index}"
print(log_index_folder_path)

if not os.path.exists(log_index_folder_path):
    os.mkdir(log_index_folder_path)

/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/logs/model_result_log/2024_06_23/06pm/0


In [11]:
file_name_for_the_str_above = f"{mid_file_name_prefix}_{yyyy}_{mm}_{dd}_{hh}{am_pm}_{midi_index}.mid"

In [12]:
abbreviations = {
    "b": "bar",
    "o": "position",
    "s": "time_signature",
    "t": "tempo",
    "i": "instrument",
    "p": "pitch",
    "d": "duration",
    "v": "velocity",
    "n": "pitch_name",
    "c": "pitch_octave",
    "f": "family",
    "e": "special"
}

In [30]:
reversed_abbrs = {v: k for k, v in abbreviations.items()}
print(reversed_abbrs)

{'bar': 'b', 'position': 'o', 'time_signature': 's', 'tempo': 't', 'instrument': 'i', 'pitch': 'p', 'duration': 'd', 'velocity': 'v', 'pitch_name': 'n', 'pitch_octave': 'c', 'family': 'f', 'special': 'e'}


In [13]:
def midi_number_to_note_name(midi_number):
    '''
        Convert a midi number to a note name
    '''
    notes = [
        "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"
    ]

    note = notes[midi_number % 12]
    octave = midi_number // 12 - 1

    return f"{note}{octave}"

In [14]:
def print_key_value_line(key_value_line):
    printed_lines = ""

    if key_value_line == None or key_value_line == []:
        print("None or Empty")
        print(key_value_line)

        printed_lines += "None or Empty\n"
        printed_lines += str(key_value_line)
        return printed_lines
    
    n_space_for_digits = 5

    n_letter_per_print_time_for_abbrs = {
        x : len(abbreviations[x]) + n_space_for_digits
        for x in abbreviations
    }

    for pair in key_value_line:
        key, value = pair

        converted_value = None

        if abbreviations[key] == 'pitch':
            converted_value = int(value)
            converted_value = midi_number_to_note_name(int(value))
        else:
            converted_value = ''

        abbr_meaning = abbreviations[key]

        pair_str = f"{abbr_meaning}: {value} {converted_value}"
        end_spaces = ' ' * (n_letter_per_print_time_for_abbrs[key] - len(abbr_meaning) - len(str(value)))

        if abbr_meaning in ['position', 'time_signature']:
            print()
            printed_lines += "\n"

        print(pair_str, end=end_spaces)

        printed_lines += pair_str + end_spaces

    return printed_lines

In [15]:
model_result_tokens_list = musecoco_line_str_to_token_list_converter(
    model_result_str
)

print(model_result_tokens_list)

[('s', 19), ('o', 0), ('t', 35), ('i', 0), ('p', 64), ('d', 6), ('v', 20), ('o', 12), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('o', 12), ('t', 35), ('i', 0), ('p', 64), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 64), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('o', 24), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('o', 12), ('t', 35), ('i', 0), ('p', 60), ('d', 6), ('v', 20), ('o',

In [16]:
# Manually Fill
# model_result_tokens_list = [('s', 9), ('o', 0), ('t', 41), ('i', 42), ('p', 48), ('d', 2), ('v', 20), ('o', 6), ('t', 41), ('i', 42), ('p', 48), ('d', 6), ('v', 20), ('p', 52), ('d', 2), ('v', 20), ('o', 6), ('t', 41), ('i', 0), ('p', 62), ('d', 11), ('v', 20), ('o', 24), ('t', 41), ('i', 42), ('p', 48), ('d', 6), ('v', 20), ('b', 1), ('s', 8), ('o', 0), ('t', 41), ('i', 42), ('p', 48), ('d', 6), ('v', 20), ('o', 8), ('t', 41), ('i', 0), ('p', 48), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 41), ('i', 42), ('p', 52), ('d', 6), ('v', 20), ('o', 21), ('t', 41), ('i', 0), ('p', 60), ('d', 11), ('v', 20), ('o', 12), ('t', 41), ('i', 42), ('p', 60), ('d', 2), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 50), ('d', 3), ('v', 20), ('o', 4), ('t', 41), ('i', 42), ('p', 53), ('d', 14), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 55), ('d', 3), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 52), ('d', 17), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 48), ('d', 3), ('v', 20), ('b', 1), ('s', 8), ('o', 0), ('t', 41), ('i', 42), ('p', 55), ('d', 6), ('v', 20), ('b', 1), ('s', 8), ('o', 0), ('t', 41), ('i', 42), ('p', 48), ('d', 3), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 43), ('d', 6), ('v', 20), ('b', 1), ('s', 9), ('o', 0), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 24), ('t', 41), ('i', 42), ('p', 53), ('d', 3), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 50), ('d', 1), ('v', 20), ('p', 52), ('d', 11), ('v', 20), ('i', 42), ('p', 43), ('d', 6), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 43), ('d', 11), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 48), ('d', 4), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 53), ('d', 3), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 52), ('d', 3), ('v', 20), ('o', 12), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 50), ('d', 2), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 43), ('d', 6), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 50), ('d', 3), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 50), ('d', 3), ('v', 20), ('o', 28), ('t', 41), ('i', 42), ('p', 42), ('d', 2), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 47), ('d', 1), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 45), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 48), ('d', 1), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 42), ('d', 2), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 57), ('d', 1), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 48), ('d', 1), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 43), ('d', 3), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 52), ('d', 2), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 50), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 45), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 36), ('d', 6), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 43), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 44), ('d', 2), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 50), ('d', 2), ('v', 20), ('o', 28), ('t', 41), ('i', 42), ('p', 48), ('d', 1), ('v', 20), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 48), ('d', 2), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 52), ('d', 2), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 44), ('d', 3), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 38), ('d', 3), ('v', 20), ('o', 20), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 42), ('d', 1), ('v', 20), ('i', 42), ('p', 45), ('d', 1), ('v', 20), ('i', 42), ('p', 36), ('d', 2), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 52), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 36), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 42), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 48), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 42), ('d', 3), ('v', 25), ('i', 42), ('p', 44), ('d', 2), ('v', 20), ('o', 32), ('t', 41), ('i', 42), ('p', 52), ('d', 2), ('v', 25), ('o', 16), ('t', 41), ('i', 42), ('p', 45), ('d', 1), ('v', 25), ('i', 42), ('p', 43), ('d', 1), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 42), ('d', 1), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 48), ('d', 2), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 50), ('d', 1), ('v', 20), ('o', 8), ('t', 41), ('i', 42), ('p', 42), ('d', 3), ('v', 20), ('o', 8), ('t', 41), ('i', 1), ('p', 42), ('d', 3), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 57), ('d', 2), ('v', 20), ('o', 16), ('t', 41), ('i', 42), ('p', 52), ('d', 2), ('v', 20), ('o', 32), ('t', 41), ('i', 42)]

In [17]:
# for k, v in model_result_tokens_list:
#     if v > 127:
#         print(f"Error: {k} {v}")
#     else:
#         pass

In [18]:

printed_lines = print_key_value_line(model_result_tokens_list)

original_structure_log_file_path = f"{log_index_folder_path}/original_structure.txt"

with open(original_structure_log_file_path, "w") as f:
    f.write(printed_lines)

print(original_structure_log_file_path)


time_signature: 19    
position: 0     tempo: 35    instrument: 0     pitch: 64 E4   duration: 6     velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 60 C4   duration: 6     velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 64 E4   duration: 6     velocity: 20    bar: 1     
time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 64 E4   duration: 6     velocity: 20    bar: 1     
time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 60 C4   duration: 6     velocity: 20    bar: 1     
time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 60 C4   duration: 6     velocity: 20    bar: 1     
time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 60 C4   duration: 6     velocity: 20    
position: 24    tempo: 35    instrument: 0     pitch: 60 C4   duration: 6     velocity: 20    bar: 1     
time_signature: 9     
position: 0     tempo: 35    instrument: 0    

In [19]:
# Count the number of folders in the log_date_folder_path
log_date_folders = os.listdir(log_date_folder_path)

n_log_time_folders = 0
# Count log time folders in each log date folder
for log_date_folder in log_date_folders:
    log_time_folder = f"{log_date_folder_path}/{log_date_folder}"
    if os.path.isdir(log_time_folder):
        n_log_time_folders += 1

midi_index = n_log_time_folders

print(f"midi_index: {midi_index}")

midi_index: 1


In [20]:
# CODE SNIPPET: Convert token list to MIDI

output_midi_file_name = file_name_for_the_str_above

output_midi_file_path = f"{output_dir}/{output_midi_file_name}"

model_result_tokens_list, printed_lines = remove_structure_errors(model_result_tokens_list)

# for k, v in model_result_tokens_list:
#     if v > 127:
#         print(f"Error: {k} {v}")
#     else:
#         pass

midi_dto = musecoco_line_to_midi_dto_converter(
    model_result_tokens_list,
    tick_per_beat=480
)

midi_obj = midi_dto_to_midi_file_object_converter(midi_dto)

write_midi_file(midi_obj, output_midi_file_path)

printed_lines = f"{output_midi_file_path}\n{printed_lines}"

remove_structure_errors: NO STRUCTURE ERRORS FOUND
MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output/model_result_2024_06_23_06pm_0.mid


In [21]:
wrong_structure_log_file_path = f"{log_index_folder_path}/wrong_structure_log.txt"

with open(wrong_structure_log_file_path, "w") as f:
    f.write(printed_lines)

print(wrong_structure_log_file_path)

/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/hooktheory/output/logs/model_result_log/2024_06_23/06pm/0/wrong_structure_log.txt


In [22]:
raise Exception("STOP HERE")

Exception: STOP HERE

In [ ]:
topk15_t1_0_ngram0_output_dir = f"{output_dir}/topk15-t1.0-ngram0"

# Create the output directory if it does not exist
if not os.path.exists(topk15_t1_0_ngram0_output_dir):
    os.makedirs(topk15_t1_0_ngram0_output_dir)
else:
    pass

In [ ]:
# Listing all 0.txt and 1.txt files in the topk15_t1_0_ngram0 directory recursively
topk15_t1_0_ngram0_txt_files = []

for root, dirs, files in os.walk(topk15_t1_0_ngram0_dir):
    for file in files:
        if file.endswith("0.txt") or file.endswith("1.txt"):
            topk15_t1_0_ngram0_txt_files.append(os.path.join(root, file))

# Sort by alphabetical order
topk15_t1_0_ngram0_txt_files = sorted(topk15_t1_0_ngram0_txt_files)

print(len(topk15_t1_0_ngram0_txt_files))

135


In [ ]:
n_success = 0

error_files = []

success_file_path_indices = []

for file_path in topk15_t1_0_ngram0_txt_files:
    print(f"PROCESSING: {file_path}")

    file_path_idx = file_path.split('/')[-3]
    
    with open(file_path, "r") as file:
        lines = file.readlines()

        for line in lines:
            line = line.split("<sep>")[1].strip()
            
            tokens = line.split(" ")
            tokens = list(filter(lambda x: "-" in x, tokens))
            tokens = list(map(lambda x: tuple([x.split("-")[0], int(x.split("-")[1])]), tokens))
            
            # tokens = remove_structure_errors(tokens)
            
            try:
                midi_dto = musecoco_line_to_midi_dto_converter(
                    tokens,
                    tick_per_beat=480
                )

                midi_obj = midi_dto_to_midi_file_object_converter(midi_dto)
                
                file_name = file_path.split('/')[-1].split('.')[0]
                grand_parent_dir_name = file_path.split('/')[-3]
                output_file_path = f"{topk15_t1_0_ngram0_output_dir}/{grand_parent_dir_name}_{file_name}.mid"
                
                write_midi_file(midi_obj, output_file_path)
                
                n_success += 1

                success_file_path_indices.append(file_path_idx)
                
            except Exception as e:
                print_key_value_line(tokens)     
                print()
                print(f"ERROR MESSAGE: {e}")
                
                print(f"ERROR: {file_path}")
                
                error_files.append(file_path)
                

            print()

print()
print(f"SUCCESS: {n_success}/{len(topk15_t1_0_ngram0_txt_files)}")  
print(success_file_path_indices)

PROCESSING: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0/1/remi/0.txt
item: ('v', 20)

time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 37 C#2   duration: 17    velocity: 20    pitch: 52 E3   duration: 6     velocity: 20    
position: 42    tempo: 35    instrument: 12    velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 55 G3   duration: 6     velocity: 20    
position: 6     tempo: 35    instrument: 0     pitch: 71 B4   duration: 6     velocity: 20    
position: 24    tempo: 35    
position: 18    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6     velocity: 20    pitch: 62 D4   duration: 6     velocity: 20    pitch: 55 G3   duration: 6     velocity: 20    pitch: 64 E4   duration: 6     velocity: 20    pitch: 52 E3   duration: 18    velocity: 20    
position: 24    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6     velocity: 2

PROCESSING: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0/11/remi/0.txt
MIDI file saved successfully: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output/topk15-t1.0-ngram0/11_0.mid

PROCESSING: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0/110/remi/0.txt

time_signature: 8     
position: 0     tempo: 35    instrument: 0     pitch: 72 C5   duration: 18    velocity: 20    pitch: 60 C4   duration: 6     velocity: 20    
position: 42    tempo: 35    instrument: 0     pitch: 67 G4   duration: 6     velocity: 20    
position: 9     tempo: 35    instrument: 52    pitch: 52 E3   duration: 15    velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 86 D6   duration: 11    velocity: 20    instrument: 60    pitch: 60 C4   duration: 3     velocity: 20    pitch: 62 

In [ ]:
n_success = 0

for file_path in error_files:
    print(f"PROCESSING: {file_path}")
    
    with open(file_path, "r") as file:
        lines = file.readlines()

        for line in lines:
            line = line.split("<sep>")[1].strip()
            
            tokens = line.split(" ")
            tokens = list(filter(lambda x: "-" in x, tokens))
            tokens = list(map(lambda x: tuple([x.split("-")[0], int(x.split("-")[1])]), tokens))
            
            print("BEFORE remove_structure_errors")
            print_key_value_line(tokens)     
            
            tokens = remove_structure_errors(tokens)
            
            print("AFTER remove_structure_errors")
            print_key_value_line(tokens)
            
            try:
                midi_dto = musecoco_line_to_midi_dto_converter(
                    tokens,
                    tick_per_beat=480
                )

                midi_obj = midi_dto_to_midi_file_object_converter(midi_dto)
                
                file_name = file_path.split('/')[-1].split('.')[0]
                grand_parent_dir_name = file_path.split('/')[-3]
                output_file_path = f"{topk15_t1_0_ngram0_output_dir}/{grand_parent_dir_name}_{file_name}.mid"
                
                write_midi_file(midi_obj, output_file_path)
                
                n_success += 1
            except Exception as e:
                print_key_value_line(tokens)     
                print()
                print(f"ERROR MESSAGE: {e}")
                
            print()

print()
print(f"SUCCESS: {n_success}/{len(error_files)}")                

PROCESSING: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/simple_convert_flow_testing_data/topk15-t1.0-ngram0/1/remi/0.txt
BEFORE remove_structure_errors

time_signature: 9     
position: 0     tempo: 35    instrument: 0     pitch: 37 C#2   duration: 17    velocity: 20    pitch: 52 E3   duration: 6     velocity: 20    
position: 42    tempo: 35    instrument: 12    velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 55 G3   duration: 6     velocity: 20    
position: 6     tempo: 35    instrument: 0     pitch: 71 B4   duration: 6     velocity: 20    
position: 24    tempo: 35    
position: 18    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6     velocity: 20    pitch: 62 D4   duration: 6     velocity: 20    pitch: 55 G3   duration: 6     velocity: 20    pitch: 64 E4   duration: 6     velocity: 20    pitch: 52 E3   duration: 18    velocity: 20    
position: 24    tempo: 35    instrument: 0     pitch: 62 D4   duration: 6 

   tempo: 35    instrument: 52    pitch: 62 D4   duration: 6     velocity: 20    
position: 36    tempo: 35    instrument: 0     pitch: 59 B3   duration: 11    velocity: 20    instrument: 58    pitch: 48 C3   duration: 12    velocity: 20    
position: 24    tempo: 35    instrument: 0     pitch: 74 D5   duration: 6     velocity: 20    bar: 1     
time_signature: 8     
position: 0     tempo: 35    instrument: 58    pitch: 46 A#2   duration: 12    velocity: 20    
position: 12    tempo: 35    instrument: 0     pitch: 74 D5   duration: 6     velocity: 20    
position: 9     tempo: 35    instrument: 0     pitch: 75 D#5   duration: 3     velocity: 20    
position: 18    tempo: 35    instrument: 0     pitch: 59 B3   duration: 12    velocity: 20    pitch: 50 D3   duration: 17    velocity: 20    pitch: 43 G2   duration: 6     velocity: 20    bar: 1     
time_signature: 8     
position: 0     tempo: 35    instrument: 0     pitch: 77 F5   duration: 23    velocity: 20    
position: 42    tempo: 3